# Question
```
You need to create a Company Funding Management Tool (CFMT). The tool requires you to
have a UI hosted on any server of your choice, (Flash, Django etc). CFMT should be a Web
App and it should be able to perform the following operations:
1. Create a new entry
2. Read a new entry
3. Delete an entry
4. Update an entry
Try to use database as Firebase, which will keep all the information of company funding.

Tool should also possess these extra functionalities:
1. Show a pie chart on all the data ingested which depicts the total funding raised
category wise.
2. Show a pie chart on all the data ingested which depicts the total funding raised state
wise.
3. Bar chart depicting total funding raised date wise.
4. Show and downloadable data in csv format contains all data (columns), round wise
(a/b/c/seed etc) separately. i.e return all the rows which has gone under &quot;seed&quot;
funding.
5. Return the name of the company(s) which has made their way till &quot;round e&quot; funding.
Similarly for other rounds as well.
For e.g. For your reference sequence is: seed, angel, a, b, c, d, e

Sample data of close to 1.4k records will be provided to you as separate attachment, Kindly
use that data load into your database (Firebase) and perform the analysis. You can decide
on the schema of your table after you go through the attachment.
```
- source: https://docs.google.com/spreadsheets/d/1l7-S39a-lAMQ_DiYp0aocWkBBgw7zq0_ulKRALOQ47M/edit#gid=191978290

## Importing the data into MySQL database.

In [5]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode


DB_NAME = 'cf'

TABLES = {}
TABLES['cf_tbl'] = (
    "CREATE TABLE `cf_tbl` ("
    "  `permalink` VARCHAR(38),"
    "  `company` VARCHAR(37),"
    "  `numEmps` INTEGER,"
    "  `category` VARCHAR(10),"
    "  `city` VARCHAR(19),"
    "  `state` VARCHAR(10),"
    "  `fundedDate` DATETIME,"
    "  `raisedAmt` INTEGER,"
    "  `raisedCurrency` VARCHAR(8),"
    "  `roundx` VARCHAR(13)"
    ") ENGINE=InnoDB")

cnx = mysql.connector.connect(user='root',host='localhost', password='asdert123',auth_plugin='mysql_native_password')
cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
        cursor.execute("SET GLOBAL local_infile=1")
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
    cursor.execute("SET GLOBAL local_infile=1")
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

        
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


cursor.close()
cnx.close()

Database cf does not exists.
Database cf created successfully.
Creating table cf_tbl: OK


In [8]:
from __future__ import print_function
from datetime import date, datetime, timedelta
import mysql.connector

cnx = mysql.connector.connect(user='root', database='cf', host='localhost', password='asdert123',auth_plugin='mysql_native_password')
cursor = cnx.cursor()

cf_tbl = ("INSERT INTO cf_tbl"
    "(`permalink`, `company`, `numEmps`, `category`, `city`, `state`, `fundedDate`, `raisedAmt`, `raisedCurrency`, `roundx`)"
" VALUES ('lifelock', 'LifeLock', 0, 'web', 'Tempe', 'AZ', '2007-05-01 00:00:00', 6850000, 'USD', 'b'),\
    ('lifelock', 'LifeLock', 0, 'web', 'Tempe', 'AZ', '2006-10-01 00:00:00', 6000000, 'USD', 'a'),\
    ('lifelock', 'LifeLock', 0, 'web', 'Tempe', 'AZ', '2008-01-01 00:00:00', 25000000, 'USD', 'c'),\
    ('mycityfaces', 'MyCityFaces', 7, 'web', 'Scottsdale', 'AZ', '2008-01-01 00:00:00', 50000, 'USD', 'seed'),\
    ('flypaper', 'Flypaper', 0, 'web', 'Phoenix', 'AZ', '2008-02-01 00:00:00', 3000000, 'USD', 'a'),\
    ('infusionsoft', 'Infusionsoft', 105, 'software', 'Gilbert', 'AZ', '2007-10-01 00:00:00', 9000000, 'USD', 'a'),\
    ('gauto', 'gAuto', 4, 'web', 'Scottsdale', 'AZ', '2008-01-01 00:00:00', 250000, 'USD', 'seed'),\
    ('chosenlist-com', 'ChosenList.com', 5, 'web', 'Scottsdale', 'AZ', '2006-10-01 00:00:00', 140000, 'USD', 'seed'),\
    ('chosenlist-com', 'ChosenList.com', 5, 'web', 'Scottsdale', 'AZ', '2008-01-25 00:00:00', 233750, 'USD', 'unknown'),\
    ('digg', 'Digg', 60, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 8500000, 'USD', 'b'),\
    ('digg', 'Digg', 60, 'web', 'San Francisco', 'CA', '2005-10-01 00:00:00', 2800000, 'USD', 'a'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2004-09-01 00:00:00', 500000, 'USD', 'angel'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2005-05-01 00:00:00', 12700000, 'USD', 'a'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2006-04-01 00:00:00', 27500000, 'USD', 'b'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2007-10-01 00:00:00', 300000000, 'USD', 'c'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2008-03-01 00:00:00', 40000000, 'USD', 'c'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2008-01-15 00:00:00', 15000000, 'USD', 'c'),\
    ('facebook', 'Facebook', 450, 'web', 'Palo Alto', 'CA', '2008-05-01 00:00:00', 100000000, 'USD', 'debt_round'),\
    ('photobucket', 'Photobucket', 60, 'web', 'Palo Alto', 'CA', '2006-05-01 00:00:00', 10500000, 'USD', 'b'),\
    ('photobucket', 'Photobucket', 60, 'web', 'Palo Alto', 'CA', '2005-03-01 00:00:00', 3000000, 'USD', 'a'),\
    ('omnidrive', 'Omnidrive', 0, 'web', 'Palo Alto', 'CA', '2006-12-01 00:00:00', 800000, 'USD', 'angel'),\
    ('geni', 'Geni', 18, 'web', 'West Hollywood', 'CA', '2007-01-01 00:00:00', 1500000, 'USD', 'a'),\
    ('geni', 'Geni', 18, 'web', 'West Hollywood', 'CA', '2007-03-01 00:00:00', 10000000, 'USD', 'b'),\
    ('twitter', 'Twitter', 17, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 5400000, 'USD', 'b'),\
    ('twitter', 'Twitter', 17, 'web', 'San Francisco', 'CA', '2008-05-01 00:00:00', 15000000, 'USD', 'c'),\
    ('stumbleupon', 'StumbleUpon', 0, 'web', 'San Francisco', 'CA', '2005-12-01 00:00:00', 1500000, 'USD', 'seed'),\
    ('gizmoz', 'Gizmoz', 0, 'web', 'Menlo Park', 'CA', '2007-05-01 00:00:00', 6300000, 'USD', 'a'),\
    ('gizmoz', 'Gizmoz', 0, 'web', 'Menlo Park', 'CA', '2008-03-16 00:00:00', 6500000, 'USD', 'b'),\
    ('scribd', 'Scribd', 14, 'web', 'San Francisco', 'CA', '2006-06-01 00:00:00', 12000, 'USD', 'seed'),\
    ('scribd', 'Scribd', 14, 'web', 'San Francisco', 'CA', '2007-01-01 00:00:00', 40000, 'USD', 'angel'),\
    ('scribd', 'Scribd', 14, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 3710000, 'USD', 'a'),\
    ('slacker', 'Slacker', 0, 'web', 'San Diego', 'CA', '2007-06-01 00:00:00', 40000000, 'USD', 'b'),\
    ('slacker', 'Slacker', 0, 'web', 'San Diego', 'CA', '2007-06-01 00:00:00', 13500000, 'USD', 'a'),\
    ('lala', 'Lala', 0, 'web', 'Palo Alto', 'CA', '2007-05-01 00:00:00', 9000000, 'USD', 'a'),\
    ('plaxo', 'Plaxo', 50, 'web', 'Mountain View', 'CA', '2002-11-01 00:00:00', 3800000, 'USD', 'a'),\
    ('plaxo', 'Plaxo', 50, 'web', 'Mountain View', 'CA', '2003-07-01 00:00:00', 8500000, 'USD', 'b'),\
    ('plaxo', 'Plaxo', 50, 'web', 'Mountain View', 'CA', '2004-04-01 00:00:00', 7000000, 'USD', 'c'),\
    ('plaxo', 'Plaxo', 50, 'web', 'Mountain View', 'CA', '2007-02-01 00:00:00', 9000000, 'USD', 'd'),\
    ('powerset', 'Powerset', 60, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 12500000, 'USD', 'a'),\
    ('technorati', 'Technorati', 25, 'web', 'San Francisco', 'CA', '2006-06-01 00:00:00', 10520000, 'USD', 'c'),\
    ('technorati', 'Technorati', 25, 'web', 'San Francisco', 'CA', '2004-09-01 00:00:00', 6500000, 'USD', 'b'),\
    ('technorati', 'Technorati', 25, 'web', 'San Francisco', 'CA', '2008-06-13 00:00:00', 7500000, 'USD', 'd'),\
    ('technorati', 'Technorati', 25, 'web', 'San Francisco', 'CA', '2007-05-10 00:00:00', 1000000, 'USD', 'c'),\
    ('mahalo', 'Mahalo', 40, 'web', 'Santa Monica', 'CA', '2006-01-01 00:00:00', 5000000, 'USD', 'a'),\
    ('mahalo', 'Mahalo', 40, 'web', 'Santa Monica', 'CA', '2007-01-01 00:00:00', 16000000, 'USD', 'b'),\
    ('kyte', 'Kyte', 40, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 2250000, 'USD', 'a'),\
    ('kyte', 'Kyte', 40, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 15000000, 'USD', 'b'),\
    ('kyte', 'Kyte', 40, 'web', 'San Francisco', 'CA', '2008-03-10 00:00:00', 6100000, 'USD', 'b'),\
    ('veoh', 'Veoh', 0, 'web', 'San Diego', 'CA', '2006-04-01 00:00:00', 12500000, 'USD', 'b'),\
    ('veoh', 'Veoh', 0, 'web', 'San Diego', 'CA', '2007-08-01 00:00:00', 25000000, 'USD', 'c'),\
    ('veoh', 'Veoh', 0, 'web', 'San Diego', 'CA', '2005-07-01 00:00:00', 2250000, 'USD', 'a'),\
    ('veoh', 'Veoh', 0, 'web', 'San Diego', 'CA', '2008-06-03 00:00:00', 30000000, 'USD', 'd'),\
    ('jingle-networks', 'Jingle Networks', 0, 'web', 'Menlo Park', 'CA', '2006-10-01 00:00:00', 30000000, 'USD', 'c'),\
    ('jingle-networks', 'Jingle Networks', 0, 'web', 'Menlo Park', 'CA', '2005-12-01 00:00:00', 5000000, 'USD', 'a'),\
    ('jingle-networks', 'Jingle Networks', 0, 'web', 'Menlo Park', 'CA', '2006-04-01 00:00:00', 26000000, 'USD', 'b'),\
    ('jingle-networks', 'Jingle Networks', 0, 'web', 'Menlo Park', 'CA', '2005-10-01 00:00:00', 400000, 'USD', 'angel'),\
    ('jingle-networks', 'Jingle Networks', 0, 'web', 'Menlo Park', 'CA', '2008-01-01 00:00:00', 13000000, 'USD', 'c'),\
    ('ning', 'Ning', 41, 'web', 'Palo Alto', 'CA', '2007-07-01 00:00:00', 44000000, 'USD', 'c'),\
    ('ning', 'Ning', 41, 'web', 'Palo Alto', 'CA', '2008-04-01 00:00:00', 60000000, 'USD', 'd'),\
    ('jotspot', 'JotSpot', 0, 'web', 'Palo Alto', 'CA', '2004-08-01 00:00:00', 5200000, 'USD', 'a'),\
    ('mercora', 'Mercora', 0, 'web', 'Sunnyvale', 'CA', '2005-01-01 00:00:00', 5000000, 'USD', 'a'),\
    ('wesabe', 'Wesabe', 0, 'web', 'San Francisco', 'CA', '2007-02-01 00:00:00', 700000, 'USD', 'seed'),\
    ('wesabe', 'Wesabe', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 4000000, 'USD', 'a'),\
    ('jangl', 'Jangl', 22, 'web', 'Pleasanton', 'CA', '2006-07-01 00:00:00', 7000000, 'USD', 'b'),\
    ('hyphen8', 'Hyphen 8', 4, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 100000, 'USD', 'angel'),\
    ('prosper', 'Prosper', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 20000000, 'USD', 'c'),\
    ('prosper', 'Prosper', 0, 'web', 'San Francisco', 'CA', '2005-04-01 00:00:00', 7500000, 'USD', 'a'),\
    ('prosper', 'Prosper', 0, 'web', 'San Francisco', 'CA', '2006-02-01 00:00:00', 12500000, 'USD', 'b'),\
    ('google', 'Google', 20000, 'web', 'Mountain View', 'CA', '1999-06-01 00:00:00', 25000000, 'USD', 'a'),\
    ('jajah', 'Jajah', 80, 'web', 'Mountain View', 'CA', '2007-05-01 00:00:00', 20000000, 'USD', 'c'),\
    ('jajah', 'Jajah', 80, 'web', 'Mountain View', 'CA', '2006-03-01 00:00:00', 3000000, 'USD', 'a'),\
    ('jajah', 'Jajah', 80, 'web', 'Mountain View', 'CA', '2006-04-01 00:00:00', 5000000, 'USD', 'b'),\
    ('youtube', 'YouTube', 0, 'web', 'San Bruno', 'CA', '2005-11-01 00:00:00', 3500000, 'USD', 'a'),\
    ('youtube', 'YouTube', 0, 'web', 'San Bruno', 'CA', '2006-04-01 00:00:00', 8000000, 'USD', 'b'),\
    ('ustream', 'Ustream', 0, 'web', 'Mountain View', 'CA', '2007-12-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('ustream', 'Ustream', 0, 'web', 'Mountain View', 'CA', '2008-04-10 00:00:00', 11100000, 'USD', 'a'),\
    ('gizmoproject', 'GizmoProject', 0, 'web', 'San Diego', 'CA', '2006-02-01 00:00:00', 6000000, 'USD', 'a'),\
    ('adap-tv', 'Adap.tv', 15, 'web', 'San Mateo', 'CA', '2007-07-01 00:00:00', 10000000, 'USD', 'a'),\
    ('topix', 'Topix', 0, 'web', 'Palo Alto', 'CA', '2006-11-01 00:00:00', 15000000, 'USD', 'b'),\
    ('revision3', 'Revision3', 0, 'web', 'San Francisco', 'CA', '2006-09-01 00:00:00', 1000000, 'USD', 'a'),\
    ('revision3', 'Revision3', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 8000000, 'USD', 'b'),\
    ('aggregateknowledge', 'Aggregate Knowledge', 0, 'web', 'San Mateo', 'CA', '2006-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('aggregateknowledge', 'Aggregate Knowledge', 0, 'web', 'San Mateo', 'CA', '2007-04-01 00:00:00', 20000000, 'USD', 'b'),\
    ('sugarinc', 'Sugar Inc', 0, 'web', 'San Francisco', 'CA', '2006-10-01 00:00:00', 5000000, 'USD', 'a'),\
    ('sugarinc', 'Sugar Inc', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 10000000, 'USD', 'b'),\
    ('zing', 'Zing', 0, 'web', 'Mountain View', 'CA', '2007-01-01 00:00:00', 13000000, 'USD', 'a'),\
    ('criticalmetrics', 'CriticalMetrics', 4, 'web', 'unknown', 'CA', '2007-01-01 00:00:00', 100000, 'USD', 'angel'),\
    ('spock', 'Spock', 30, 'web', 'Redwood City', 'CA', '2006-12-01 00:00:00', 7000000, 'USD', 'a'),\
    ('wize', 'Wize', 0, 'web', 'unknown', 'CA', '2007-01-01 00:00:00', 4000000, 'USD', 'a'),\
    ('sodahead', 'SodaHead', 0, 'web', 'unknown', 'CA', '2007-01-01 00:00:00', 4250000, 'USD', 'a'),\
    ('sodahead', 'SodaHead', 0, 'web', 'unknown', 'CA', '2008-06-25 00:00:00', 8400000, 'USD', 'b'),\
    ('casttv', 'CastTV', 0, 'web', 'San Francisco', 'CA', '2007-04-01 00:00:00', 3100000, 'USD', 'a'),\
    ('buzznet', 'BuzzNet', 0, 'web', 'Hollywood', 'CA', '2007-05-01 00:00:00', 6000000, 'USD', 'a'),\
    ('buzznet', 'BuzzNet', 0, 'web', 'Hollywood', 'CA', '2008-04-07 00:00:00', 25000000, 'USD', 'b'),\
    ('funny-or-die', 'Funny Or Die', 0, 'web', 'Palo Alto', 'CA', '2007-12-01 00:00:00', 15000000, 'USD', 'b'),\
    ('sphere', 'Sphere', 11, 'web', 'San Francisco', 'CA', '2006-05-01 00:00:00', 3000000, 'USD', 'b'),\
    ('sphere', 'Sphere', 11, 'web', 'San Francisco', 'CA', '2005-04-01 00:00:00', 500000, 'USD', 'a'),\
    ('meevee', 'MeeVee', 0, 'web', 'Burlingame', 'CA', '2006-02-01 00:00:00', 6500000, 'USD', 'b'),\
    ('meevee', 'MeeVee', 0, 'web', 'Burlingame', 'CA', '2006-08-01 00:00:00', 8000000, 'USD', 'c'),\
    ('meevee', 'MeeVee', 0, 'web', 'Burlingame', 'CA', '2005-02-01 00:00:00', 7000000, 'USD', 'a'),\
    ('meevee', 'MeeVee', 0, 'web', 'Burlingame', 'CA', '2007-09-01 00:00:00', 3500000, 'USD', 'd'),\
    ('mashery', 'Mashery', 0, 'web', 'San Francisco', 'CA', '2008-06-26 00:00:00', 2000000, 'USD', 'c'),\
    ('yelp', 'Yelp', 0, 'web', 'San Francisco', 'CA', '2006-10-01 00:00:00', 10000000, 'USD', 'c'),\
    ('yelp', 'Yelp', 0, 'web', 'San Francisco', 'CA', '2005-10-01 00:00:00', 5000000, 'USD', 'b'),\
    ('yelp', 'Yelp', 0, 'web', 'San Francisco', 'CA', '2004-07-01 00:00:00', 1000000, 'USD', 'a'),\
    ('yelp', 'Yelp', 0, 'web', 'San Francisco', 'CA', '2008-02-26 00:00:00', 15000000, 'USD', 'd'),\
    ('spotplex', 'Spotplex', 3, 'web', 'Santa Clara', 'CA', '2007-01-01 00:00:00', 450000, 'USD', 'angel'),\
    ('coghead', 'Coghead', 21, 'web', 'Redwood City', 'CA', '2006-03-01 00:00:00', 3200000, 'USD', 'a'),\
    ('coghead', 'Coghead', 21, 'web', 'Redwood City', 'CA', '2007-03-01 00:00:00', 8000000, 'USD', 'b'),\
    ('zooomr', 'Zooomr', 0, 'web', 'San Francisco', 'CA', '2006-02-01 00:00:00', 50000, 'USD', 'angel'),\
    ('sidestep', 'SideStep', 75, 'web', 'Santa Clara', 'CA', '2007-02-01 00:00:00', 15000000, 'USD', 'c'),\
    ('sidestep', 'SideStep', 75, 'web', 'Santa Clara', 'CA', '2004-01-01 00:00:00', 8000000, 'USD', 'b'),\
    ('sidestep', 'SideStep', 75, 'web', 'Santa Clara', 'CA', '1999-12-01 00:00:00', 2200000, 'USD', 'a'),\
    ('sidestep', 'SideStep', 75, 'web', 'Santa Clara', 'CA', '2000-10-01 00:00:00', 6800000, 'USD', 'a'),\
    ('rockyou', 'RockYou', 0, 'web', 'San Mateo', 'CA', '2007-01-01 00:00:00', 1500000, 'USD', 'a'),\
    ('rockyou', 'RockYou', 0, 'web', 'San Mateo', 'CA', '2007-03-01 00:00:00', 15000000, 'USD', 'b'),\
    ('rockyou', 'RockYou', 0, 'web', 'San Mateo', 'CA', '2008-06-09 00:00:00', 35000000, 'USD', 'c'),\
    ('rockyou', 'RockYou', 0, 'web', 'San Mateo', 'CA', '2008-05-27 00:00:00', 1000000, 'USD', 'unattributed'),\
    ('pageflakes', 'Pageflakes', 20, 'web', 'San Francisco', 'CA', '2006-05-01 00:00:00', 1300000, 'USD', 'a'),\
    ('swivel', 'Swivel', 0, 'web', 'San Francisco', 'CA', '2006-09-01 00:00:00', 1000000, 'USD', 'a'),\
    ('swivel', 'Swivel', 0, 'web', 'San Francisco', 'CA', '2007-04-01 00:00:00', 1000000, 'USD', 'a'),\
    ('slide', 'Slide', 64, 'web', 'San Francisco', 'CA', '2005-07-01 00:00:00', 8000000, 'USD', 'b'),\
    ('slide', 'Slide', 64, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 50000000, 'USD', 'd'),\
    ('bebo', 'Bebo', 0, 'web', 'San Francisco', 'CA', '2006-05-01 00:00:00', 15000000, 'USD', 'a'),\
    ('freebase', 'freebase', 0, 'web', 'San Francisco', 'CA', '2006-03-01 00:00:00', 15000000, 'USD', 'a'),\
    ('freebase', 'freebase', 0, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 42500000, 'USD', 'b'),\
    ('metawebtechnologies', 'Metaweb Technologies', 0, 'web', 'San Francisco', 'CA', '2008-01-15 00:00:00', 42000000, 'USD', 'b'),\
    ('glammedia', 'Glam Media', 0, 'web', 'Brisbane', 'CA', '2004-07-01 00:00:00', 1100000, 'USD', 'a'),\
    ('glammedia', 'Glam Media', 0, 'web', 'Brisbane', 'CA', '2005-07-01 00:00:00', 10000000, 'USD', 'b'),\
    ('glammedia', 'Glam Media', 0, 'web', 'Brisbane', 'CA', '2006-12-01 00:00:00', 18500000, 'USD', 'c'),\
    ('glammedia', 'Glam Media', 0, 'web', 'Brisbane', 'CA', '2008-02-01 00:00:00', 65000000, 'USD', 'd'),\
    ('glammedia', 'Glam Media', 0, 'web', 'Brisbane', 'CA', '2008-02-01 00:00:00', 20000000, 'USD', 'd'),\
    ('thefind', 'TheFind', 0, 'web', 'Mountain View', 'CA', '2005-02-01 00:00:00', 7000000, 'USD', 'a'),\
    ('thefind', 'TheFind', 0, 'web', 'Mountain View', 'CA', '2006-10-01 00:00:00', 4000000, 'USD', 'b'),\
    ('thefind', 'TheFind', 0, 'web', 'Mountain View', 'CA', '2007-07-01 00:00:00', 15000000, 'USD', 'c'),\
    ('zazzle', 'Zazzle', 0, 'web', 'Redwood City', 'CA', '2005-07-01 00:00:00', 16000000, 'USD', 'a'),\
    ('zazzle', 'Zazzle', 0, 'web', 'Redwood City', 'CA', '2007-10-01 00:00:00', 30000000, 'USD', 'b'),\
    ('dogster', 'Dogster', 0, 'web', 'San Francisco', 'CA', '2006-09-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('pandora', 'Pandora', 80, 'web', 'Oakland', 'CA', '2004-01-01 00:00:00', 7800000, 'USD', 'b'),\
    ('pandora', 'Pandora', 80, 'web', 'Oakland', 'CA', '2005-10-01 00:00:00', 12000000, 'USD', 'c'),\
    ('pandora', 'Pandora', 80, 'web', 'Oakland', 'CA', '2000-03-02 00:00:00', 1500000, 'USD', 'a'),\
    ('cafepress', 'Cafepress', 300, 'web', 'Foster City', 'CA', '2000-03-01 00:00:00', 1200000, 'USD', 'a'),\
    ('cafepress', 'Cafepress', 300, 'web', 'Foster City', 'CA', '2001-05-01 00:00:00', 300000, 'USD', 'a'),\
    ('cafepress', 'Cafepress', 300, 'web', 'Foster City', 'CA', '2005-02-01 00:00:00', 14000000, 'USD', 'b'),\
    ('pbwiki', 'pbwiki', 15, 'web', 'unknown', 'CA', '2006-09-11 00:00:00', 350000, 'USD', 'a'),\
    ('pbwiki', 'pbwiki', 15, 'web', 'unknown', 'CA', '2007-02-01 00:00:00', 2100000, 'USD', 'b'),\
    ('adbrite', 'AdBrite', 0, 'web', 'San Francisco', 'CA', '2004-09-01 00:00:00', 4000000, 'USD', 'a'),\
    ('adbrite', 'AdBrite', 0, 'web', 'San Francisco', 'CA', '2006-02-01 00:00:00', 8000000, 'USD', 'b'),\
    ('adbrite', 'AdBrite', 0, 'web', 'San Francisco', 'CA', '2007-11-01 00:00:00', 23000000, 'USD', 'c'),\
    ('loomia', 'Loomia', 0, 'web', 'San Francisco', 'CA', '2005-06-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('loomia', 'Loomia', 0, 'web', 'San Francisco', 'CA', '2008-04-01 00:00:00', 5000000, 'USD', 'a'),\
    ('meebo', 'Meebo', 40, 'web', 'Mountain View', 'CA', '2005-12-01 00:00:00', 3500000, 'USD', 'a'),\
    ('meebo', 'Meebo', 40, 'web', 'Mountain View', 'CA', '2007-01-01 00:00:00', 9000000, 'USD', 'b'),\
    ('meebo', 'Meebo', 40, 'web', 'Mountain View', 'CA', '2008-04-30 00:00:00', 25000000, 'USD', 'c'),\
    ('eventbrite', 'Eventbrite', 13, 'web', 'San Francisco', 'CA', '2006-11-01 00:00:00', 100000, 'USD', 'angel'),\
    ('linkedin', 'LinkedIn', 0, 'web', 'Mountain View', 'CA', '2003-11-01 00:00:00', 4700000, 'USD', 'a'),\
    ('linkedin', 'LinkedIn', 0, 'web', 'Mountain View', 'CA', '2004-10-01 00:00:00', 10000000, 'USD', 'b'),\
    ('linkedin', 'LinkedIn', 0, 'web', 'Mountain View', 'CA', '2007-01-01 00:00:00', 12800000, 'USD', 'c'),\
    ('linkedin', 'LinkedIn', 0, 'web', 'Mountain View', 'CA', '2008-06-17 00:00:00', 53000000, 'USD', 'd'),\
    ('flickim', 'FlickIM', 0, 'web', 'Berkeley', 'CA', '2007-03-01 00:00:00', 1600000, 'USD', 'a'),\
    ('terabitz', 'Terabitz', 12, 'web', 'Palo Alto', 'CA', '2007-02-01 00:00:00', 10000000, 'USD', 'a'),\
    ('healthline', 'Healthline', 0, 'web', 'San Francisco', 'CA', '2006-01-01 00:00:00', 14000000, 'USD', 'a'),\
    ('healthline', 'Healthline', 0, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 21000000, 'USD', 'b'),\
    ('box-net', 'Box.net', 19, 'web', 'Palo Alto', 'CA', '2006-10-01 00:00:00', 1500000, 'USD', 'a'),\
    ('box-net', 'Box.net', 19, 'web', 'Palo Alto', 'CA', '2008-01-23 00:00:00', 6000000, 'USD', 'b'),\
    ('conduit', 'Conduit', 0, 'web', 'Redwood Shores', 'CA', '2006-07-01 00:00:00', 1800000, 'USD', 'a'),\
    ('conduit', 'Conduit', 0, 'web', 'Redwood Shores', 'CA', '2008-01-01 00:00:00', 8000000, 'USD', 'b'),\
    ('edgeio', 'Edgeio', 0, 'web', 'Palo Alto', 'CA', '2007-07-01 00:00:00', 5000000, 'USD', 'a'),\
    ('spotrunner', 'Spot Runner', 0, 'web', 'Los Angeles', 'CA', '2006-01-01 00:00:00', 10000000, 'USD', 'a'),\
    ('spotrunner', 'Spot Runner', 0, 'web', 'Los Angeles', 'CA', '2006-10-01 00:00:00', 40000000, 'USD', 'b'),\
    ('spotrunner', 'Spot Runner', 0, 'web', 'Los Angeles', 'CA', '2008-05-07 00:00:00', 51000000, 'USD', 'c'),\
    ('kaboodle', 'Kaboodle', 0, 'web', 'Santa Clara', 'CA', '2005-03-01 00:00:00', 1500000, 'USD', 'a'),\
    ('kaboodle', 'Kaboodle', 0, 'web', 'Santa Clara', 'CA', '2006-03-01 00:00:00', 2000000, 'USD', 'b'),\
    ('kaboodle', 'Kaboodle', 0, 'web', 'Santa Clara', 'CA', '2007-03-01 00:00:00', 1500000, 'USD', 'c'),\
    ('giga-omni-media', 'Giga Omni Media', 18, 'web', 'San Francisco', 'CA', '2006-06-01 00:00:00', 325000, 'USD', 'a'),\
    ('visiblepath', 'Visible Path', 0, 'web', 'Foster City', 'CA', '2006-03-01 00:00:00', 17000000, 'USD', 'b'),\
    ('visiblepath', 'Visible Path', 0, 'web', 'Foster City', 'CA', '2006-03-01 00:00:00', 7700000, 'USD', 'a'),\
    ('wink', 'Wink', 0, 'web', 'Mountain View', 'CA', '2005-01-01 00:00:00', 6200000, 'USD', 'a'),\
    ('seesmic', 'seesmic', 0, 'web', 'San Francisco', 'CA', '2007-11-01 00:00:00', 6000000, 'USD', 'a'),\
    ('seesmic', 'seesmic', 0, 'web', 'San Francisco', 'CA', '2008-06-20 00:00:00', 6000000, 'USD', 'b'),\
    ('zvents', 'Zvents', 0, 'web', 'San Mateo', 'CA', '2006-11-07 00:00:00', 7000000, 'USD', 'a'),\
    ('zvents', 'Zvents', 0, 'web', 'San Mateo', 'CA', '2005-10-05 00:00:00', 200000, 'USD', 'seed'),\
    ('eventful', 'Eventful', 35, 'web', 'San Diego', 'CA', '2006-09-01 00:00:00', 7500000, 'USD', 'b'),\
    ('eventful', 'Eventful', 35, 'web', 'San Diego', 'CA', '2005-03-01 00:00:00', 2100000, 'USD', 'a'),\
    ('oodle', 'Oodle', 0, 'web', 'San Mateo', 'CA', '2006-05-01 00:00:00', 5000000, 'USD', 'a'),\
    ('oodle', 'Oodle', 0, 'web', 'San Mateo', 'CA', '2007-07-01 00:00:00', 11000000, 'USD', 'b'),\
    ('odesk', 'oDesk', 0, 'web', 'Menlo Park', 'CA', '2006-04-01 00:00:00', 6000000, 'USD', 'a'),\
    ('odesk', 'oDesk', 0, 'web', 'Menlo Park', 'CA', '2006-09-01 00:00:00', 8000000, 'USD', 'b'),\
    ('odesk', 'oDesk', 0, 'web', 'Menlo Park', 'CA', '2008-06-01 00:00:00', 15000000, 'USD', 'c'),\
    ('simplyhired', 'SimplyHired', 0, 'web', 'Mountain View', 'CA', '2006-03-01 00:00:00', 13500000, 'USD', 'b'),\
    ('simplyhired', 'SimplyHired', 0, 'web', 'Mountain View', 'CA', '2005-08-05 00:00:00', 3000000, 'USD', 'a'),\
    ('ooma', 'ooma', 47, 'web', 'Palo Alto', 'CA', '2005-01-01 00:00:00', 8000000, 'USD', 'a'),\
    ('ooma', 'ooma', 47, 'web', 'Palo Alto', 'CA', '2006-12-01 00:00:00', 18000000, 'USD', 'b'),\
    ('goingon', 'GoingOn', 3, 'web', 'Woodside', 'CA', '2005-08-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('flixster', 'Flixster', 17, 'web', 'San Francisco', 'CA', '2007-02-01 00:00:00', 2000000, 'USD', 'a'),\
    ('flixster', 'Flixster', 17, 'web', 'San Francisco', 'CA', '2008-04-04 00:00:00', 5000000, 'USD', 'b'),\
    ('piczo', 'Piczo', 0, 'web', 'San Francisco', 'CA', '2007-01-01 00:00:00', 11000000, 'USD', 'c'),\
    ('socialtext', 'Socialtext', 0, 'web', 'Palo Alto', 'CA', '2004-06-01 00:00:00', 300000, 'USD', 'angel'),\
    ('socialtext', 'Socialtext', 0, 'web', 'Palo Alto', 'CA', '2005-06-01 00:00:00', 3100000, 'USD', 'b'),\
    ('socialtext', 'Socialtext', 0, 'web', 'Palo Alto', 'CA', '2007-11-01 00:00:00', 9500000, 'USD', 'c'),\
    ('powerreviews', 'PowerReview', 0, 'web', 'Millbrae', 'CA', '2005-12-01 00:00:00', 6250000, 'USD', 'a'),\
    ('powerreviews', 'PowerReview', 0, 'web', 'Millbrae', 'CA', '2007-09-01 00:00:00', 15000000, 'USD', 'b'),\
    ('hi5', 'hi5', 100, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 20000000, 'USD', 'a'),\
    ('hi5', 'hi5', 100, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 15000000, 'USD', 'debt_round'),\
    ('tagged', 'Tagged', 0, 'web', 'San Francisco', 'CA', '2005-09-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('tagged', 'Tagged', 0, 'web', 'San Francisco', 'CA', '2005-12-01 00:00:00', 7000000, 'USD', 'a'),\
    ('tagged', 'Tagged', 0, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 15000000, 'USD', 'b'),\
    ('jaxtr', 'Jaxtr', 0, 'web', 'Menlo Park', 'CA', '2007-07-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('jaxtr', 'Jaxtr', 0, 'web', 'Menlo Park', 'CA', '2007-08-01 00:00:00', 10000000, 'USD', 'a'),\
    ('jaxtr', 'Jaxtr', 0, 'web', 'Menlo Park', 'CA', '2008-06-23 00:00:00', 10000000, 'USD', 'b'),\
    ('m', 'Me', 0, 'web', 'San Jose', 'CA', '2007-05-01 00:00:00', 1500000, 'USD', 'seed'),\
    ('intronetworks', 'introNetworks', 9, 'web', 'Santa Barbara', 'CA', '2007-05-01 00:00:00', 2700000, 'USD', 'a'),\
    ('leveragesoftware', 'Leverage Software', 21, 'web', 'San Francisco', 'CA', '2005-05-14 00:00:00', 6000000, 'USD', 'a'),\
    ('lithiumtechnologies', 'Lithium Technologies', 42, 'web', 'Emeryville', 'CA', '2007-04-17 00:00:00', 9000000, 'USD', 'a'),\
    ('lithiumtechnologies', 'Lithium Technologies', 42, 'web', 'Emeryville', 'CA', '2008-06-01 00:00:00', 12000000, 'USD', 'b'),\
    ('genius', 'Genius', 0, 'web', 'San Mateo', 'CA', '2006-03-01 00:00:00', 5100000, 'USD', 'a'),\
    ('genius', 'Genius', 0, 'web', 'San Mateo', 'CA', '2007-01-01 00:00:00', 10000000, 'USD', 'b'),\
    ('genius', 'Genius', 0, 'web', 'San Mateo', 'CA', '2008-02-05 00:00:00', 19000000, 'USD', 'c'),\
    ('respectance', 'Respectance', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 250000, 'USD', 'seed'),\
    ('respectance', 'Respectance', 0, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 1500000, 'USD', 'a'),\
    ('curse', 'Curse', 0, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 5000000, 'USD', 'a'),\
    ('licketyship', 'LicketyShip', 0, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('grockit', 'Grockit', 8, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 2300000, 'USD', 'a'),\
    ('grockit', 'Grockit', 8, 'web', 'San Francisco', 'CA', '2008-05-30 00:00:00', 8000000, 'USD', 'b'),\
    ('peerme', 'PeerMe', 0, 'web', 'Mountain View', 'CA', '2005-01-01 00:00:00', 5000000, 'USD', 'a'),\
    ('kiptronic', 'Kiptronic', 20, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 4000000, 'USD', 'a'),\
    ('kiptronic', 'Kiptronic', 20, 'web', 'San Francisco', 'CA', '2005-11-01 00:00:00', 300000, 'USD', 'angel'),\
    ('kiptronic', 'Kiptronic', 20, 'web', 'San Francisco', 'CA', '2008-06-05 00:00:00', 3000000, 'USD', 'b'),\
    ('vuze', 'Vuze', 0, 'web', 'Palo Alto', 'CA', '2006-12-01 00:00:00', 12000000, 'USD', 'b'),\
    ('vuze', 'Vuze', 0, 'web', 'Palo Alto', 'CA', '2007-12-01 00:00:00', 20000000, 'USD', 'c'),\
    ('phonezoo', 'Phonezoo', 0, 'web', 'Sunnyvale', 'CA', '2006-06-01 00:00:00', 560000, 'USD', 'a'),\
    ('phonezoo', 'Phonezoo', 0, 'web', 'Sunnyvale', 'CA', '2007-02-01 00:00:00', 1500000, 'USD', 'a'),\
    ('droplettechnology', 'Droplet Technology', 0, 'web', 'Menlo Park', 'CA', '2006-10-01 00:00:00', 3500000, 'USD', 'a'),\
    ('podtech', 'PodTech', 0, 'web', 'Palo Alto', 'CA', '2006-03-01 00:00:00', 5500000, 'USD', 'a'),\
    ('podtech', 'PodTech', 0, 'web', 'Palo Alto', 'CA', '2007-06-01 00:00:00', 2000000, 'USD', 'a'),\
    ('crackle', 'Crackle', 0, 'web', 'Mill Valley', 'CA', '2005-12-01 00:00:00', 1750000, 'USD', 'a'),\
    ('reddit', 'Reddit', 0, 'web', 'San Francisco', 'CA', '2005-06-01 00:00:00', 100000, 'USD', 'seed'),\
    ('wikia', 'Wikia', 0, 'web', 'San Mateo', 'CA', '2006-03-01 00:00:00', 4000000, 'USD', 'a'),\
    ('wikia', 'Wikia', 0, 'web', 'San Mateo', 'CA', '2006-12-01 00:00:00', 10000000, 'USD', 'b'),\
    ('retrevo', 'Retrevo', 0, 'web', 'Sunnyvale', 'CA', '2006-12-01 00:00:00', 3200000, 'USD', 'a'),\
    ('retrevo', 'Retrevo', 0, 'web', 'Sunnyvale', 'CA', '2006-02-01 00:00:00', 700000, 'USD', 'seed'),\
    ('retrevo', 'Retrevo', 0, 'web', 'Sunnyvale', 'CA', '2008-03-20 00:00:00', 8000000, 'USD', 'b'),\
    ('buxfer', 'Buxfer', 2, 'web', 'Mountain View', 'CA', '2007-01-01 00:00:00', 15000, 'USD', 'seed'),\
    ('buxfer', 'Buxfer', 2, 'web', 'Mountain View', 'CA', '2007-04-01 00:00:00', 300000, 'USD', 'angel'),\
    ('yousendit', 'YouSendIt', 0, 'web', 'Campbell', 'CA', '2007-04-24 00:00:00', 10000000, 'USD', 'b'),\
    ('yousendit', 'YouSendIt', 0, 'web', 'Campbell', 'CA', '2005-08-01 00:00:00', 10000000, 'USD', 'a'),\
    ('yousendit', 'YouSendIt', 0, 'web', 'Campbell', 'CA', '2008-07-15 00:00:00', 14000000, 'USD', 'c'),\
    ('tangler', 'Tangler', 0, 'web', 'Mountain View', 'CA', '2006-07-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('obopay', 'obopay', 0, 'web', 'Redwood City', 'CA', '2006-02-01 00:00:00', 10000000, 'USD', 'a'),\
    ('obopay', 'obopay', 0, 'web', 'Redwood City', 'CA', '2006-09-01 00:00:00', 7000000, 'USD', 'b'),\
    ('obopay', 'obopay', 0, 'web', 'Redwood City', 'CA', '2007-07-01 00:00:00', 29000000, 'USD', 'c'),\
    ('obopay', 'obopay', 0, 'web', 'Redwood City', 'CA', '2008-04-21 00:00:00', 20000000, 'USD', 'd'),\
    ('paypal', 'PayPal', 0, 'web', 'San Jose', 'CA', '2000-04-01 00:00:00', 100000000, 'USD', 'c'),\
    ('paypal', 'PayPal', 0, 'web', 'San Jose', 'CA', '2001-02-01 00:00:00', 90000000, 'USD', 'd'),\
    ('paypal', 'PayPal', 0, 'web', 'San Jose', 'CA', '1999-07-01 00:00:00', 4000000, 'USD', 'b'),\
    ('talkplus', 'TalkPlus', 0, 'web', 'San Mateo', 'CA', '2006-10-01 00:00:00', 5500000, 'USD', 'a'),\
    ('vudu', 'Vudu', 50, 'hardware', 'Santa Clara', 'CA', '2005-06-01 00:00:00', 21000000, 'USD', 'a'),\
    ('insiderpages', 'Insider Pages', 0, 'web', 'Redwood City', 'CA', '2006-03-01 00:00:00', 8500000, 'USD', 'a'),\
    ('snapfish', 'Snapfish', 0, 'web', 'San Francisco', 'CA', '1999-11-01 00:00:00', 7500000, 'USD', 'a'),\
    ('snapfish', 'Snapfish', 0, 'web', 'San Francisco', 'CA', '2000-05-01 00:00:00', 36000000, 'USD', 'b'),\
    ('affinitycircles', 'Affinity Circles', 22, 'web', 'Mountain View', 'CA', '2006-05-01 00:00:00', 4100000, 'USD', 'c'),\
    ('affinitycircles', 'Affinity Circles', 22, 'web', 'Mountain View', 'CA', '2005-06-01 00:00:00', 1000000, 'USD', 'b'),\
    ('affinitycircles', 'Affinity Circles', 22, 'web', 'Mountain View', 'CA', '2004-12-01 00:00:00', 440000, 'USD', 'a'),\
    ('imvu', 'IMVU', 0, 'web', 'Palo Alto', 'CA', '2006-02-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('nirvanix', 'Nirvanix', 0, 'web', 'San Diego', 'CA', '2007-12-19 00:00:00', 18000000, 'USD', 'a'),\
    ('fuzzwich', 'Habit Industries', 4, 'web', 'Palo Alto', 'CA', '2007-06-01 00:00:00', 15000, 'USD', 'seed'),\
    ('gaia', 'Gaia', 0, 'web', 'San Jose', 'CA', '2006-06-01 00:00:00', 8930000, 'USD', 'a'),\
    ('gaia', 'Gaia', 0, 'web', 'San Jose', 'CA', '2007-03-01 00:00:00', 12000000, 'USD', 'b'),\
    ('gaia', 'Gaia', 0, 'web', 'San Jose', 'CA', '2008-07-14 00:00:00', 11000000, 'USD', 'c'),\
    ('greatcall', 'GreatCall', 0, 'web', 'Del Mar', 'CA', '2007-08-01 00:00:00', 36600000, 'USD', 'a'),\
    ('revver', 'Revver', 0, 'web', 'Los Angeles', 'CA', '2005-11-01 00:00:00', 4000000, 'USD', 'a'),\
    ('revver', 'Revver', 0, 'web', 'Los Angeles', 'CA', '2006-04-01 00:00:00', 8700000, 'USD', 'b'),\
    ('metacafe', 'Metacafe', 0, 'web', 'Palo Alto', 'CA', '2006-07-01 00:00:00', 15000000, 'USD', 'b'),\
    ('metacafe', 'Metacafe', 0, 'web', 'Palo Alto', 'CA', '2007-08-01 00:00:00', 30000000, 'USD', 'c'),\
    ('flock', 'Flock', 40, 'web', 'Redwood City', 'CA', '2005-11-01 00:00:00', 3300000, 'USD', 'b'),\
    ('flock', 'Flock', 40, 'web', 'Redwood City', 'CA', '2006-06-01 00:00:00', 10000000, 'USD', 'c'),\
    ('flock', 'Flock', 40, 'web', 'Redwood City', 'CA', '2008-05-21 00:00:00', 15000000, 'USD', 'd'),\
    ('vmixmedia', 'VMIX Media', 42, 'web', 'San Diego', 'CA', '2006-03-01 00:00:00', 5000000, 'USD', 'a'),\
    ('vmixmedia', 'VMIX Media', 42, 'web', 'San Diego', 'CA', '2007-10-01 00:00:00', 16500000, 'USD', 'b'),\
    ('kontera', 'Kontera', 0, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 10300000, 'USD', 'b'),\
    ('kontera', 'Kontera', 0, 'web', 'San Francisco', 'CA', '2006-07-01 00:00:00', 7000000, 'USD', 'a'),\
    ('tokbox', 'Tokbox', 0, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 4000000, 'USD', 'a'),\
    ('tokbox', 'Tokbox', 0, 'web', 'San Francisco', 'CA', '2008-08-06 00:00:00', 10000000, 'USD', 'b'),\
    ('six-apart', 'Six Apart', 150, 'web', 'San Francisco', 'CA', '2004-10-01 00:00:00', 10000000, 'USD', 'b'),\
    ('six-apart', 'Six Apart', 150, 'web', 'San Francisco', 'CA', '2006-03-01 00:00:00', 12000000, 'USD', 'c'),\
    ('six-apart', 'Six Apart', 150, 'web', 'San Francisco', 'CA', '2003-04-23 00:00:00', 600000, 'USD', 'a'),\
    ('weebly', 'Weebly', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 650000, 'USD', 'seed'),\
    ('synthasite', 'Synthasite', 26, 'web', 'San Francisco', 'CA', '2007-11-01 00:00:00', 5000000, 'USD', 'a'),\
    ('bittorrent', 'BitTorrent', 29, 'web', 'San Francisco', 'CA', '2005-09-01 00:00:00', 8750000, 'USD', 'a'),\
    ('bittorrent', 'BitTorrent', 29, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 25000000, 'USD', 'b'),\
    ('kongregate', 'Kongregate', 16, 'web', 'San Francisco', 'CA', '2007-03-01 00:00:00', 1000000, 'USD', 'a'),\
    ('kongregate', 'Kongregate', 16, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 5000000, 'USD', 'b'),\
    ('kongregate', 'Kongregate', 16, 'web', 'San Francisco', 'CA', '2008-04-01 00:00:00', 3000000, 'USD', 'b'),\
    ('dancejam', 'DanceJam', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('dancejam', 'DanceJam', 0, 'web', 'San Francisco', 'CA', '2008-02-01 00:00:00', 3500000, 'USD', 'a'),\
    ('fathomonline', 'Fathom Online', 0, 'web', 'San Francisco', 'CA', '2004-07-01 00:00:00', 6000000, 'USD', 'a'),\
    ('zivity', 'Zivity', 16, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('zivity', 'Zivity', 16, 'web', 'San Francisco', 'CA', '2008-03-01 00:00:00', 7000000, 'USD', 'b'),\
    ('bluelithium', 'BlueLithium', 135, 'web', 'San Jose', 'CA', '2005-02-01 00:00:00', 11500000, 'USD', 'a'),\
    ('expresso', 'eXpresso', 0, 'web', 'Menlo Park', 'CA', '2007-10-01 00:00:00', 2000000, 'USD', 'a'),\
    ('doostang', 'Doostang', 0, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 3500000, 'USD', 'a'),\
    ('docstoc', 'Docstoc', 0, 'web', 'Beverly Hills', 'CA', '2007-11-01 00:00:00', 750000, 'USD', 'a'),\
    ('docstoc', 'Docstoc', 0, 'web', 'Beverly Hills', 'CA', '2008-04-28 00:00:00', 3250000, 'USD', 'b'),\
    ('mevio', 'Mevio', 0, 'web', 'San Francisco', 'CA', '2005-07-01 00:00:00', 8900000, 'USD', 'a'),\
    ('mevio', 'Mevio', 0, 'web', 'San Francisco', 'CA', '2006-09-01 00:00:00', 15000000, 'USD', 'b'),\
    ('mevio', 'Mevio', 0, 'web', 'San Francisco', 'CA', '2008-07-09 00:00:00', 15000000, 'USD', 'c'),\
    ('purevideo', 'PureVideo Networks', 0, 'web', 'Los Angeles', 'CA', '2005-12-01 00:00:00', 5600000, 'USD', 'a'),\
    ('purevideo', 'PureVideo Networks', 0, 'web', 'Los Angeles', 'CA', '2007-09-01 00:00:00', 2850000, 'USD', 'b'),\
    ('filmloop', 'FilmLoop', 0, 'web', 'Menlo Park', 'CA', '2005-02-01 00:00:00', 5600000, 'USD', 'a'),\
    ('bitpass', 'BitPass', 0, 'web', 'Mountain View', 'CA', '2003-07-01 00:00:00', 1500000, 'USD', 'a'),\
    ('bitpass', 'BitPass', 0, 'web', 'Mountain View', 'CA', '2004-09-01 00:00:00', 11800000, 'USD', 'b'),\
    ('art-com', 'Art.com', 500, 'web', 'Emeryville', 'CA', '2005-02-01 00:00:00', 30000000, 'USD', 'a'),\
    ('atomshockwave', 'Atom Shockwave', 0, 'web', 'San Francisco', 'CA', '2001-03-01 00:00:00', 22900000, 'USD', 'a'),\
    ('cinemanow', 'CinemaNow', 0, 'web', 'Marina Del Rey', 'CA', '2004-07-01 00:00:00', 11000000, 'USD', 'a'),\
    ('digitalchocolate', 'Digital Chocolate', 0, 'web', 'San Mateo', 'CA', '2003-12-01 00:00:00', 8400000, 'USD', 'a'),\
    ('digitalchocolate', 'Digital Chocolate', 0, 'web', 'San Mateo', 'CA', '2003-12-01 00:00:00', 13000000, 'USD', 'b'),\
    ('eharmony', 'eHarmony', 0, 'web', 'Pasadena', 'CA', '2000-06-01 00:00:00', 3000000, 'USD', 'a'),\
    ('eharmony', 'eHarmony', 0, 'web', 'Pasadena', 'CA', '2004-11-01 00:00:00', 110000000, 'USD', 'b'),\
    ('friendster', 'Friendster', 465, 'web', 'San Francisco', 'CA', '2002-12-01 00:00:00', 2400000, 'USD', 'a'),\
    ('friendster', 'Friendster', 465, 'web', 'San Francisco', 'CA', '2003-10-01 00:00:00', 13000000, 'USD', 'b'),\
    ('friendster', 'Friendster', 465, 'web', 'San Francisco', 'CA', '2006-08-01 00:00:00', 10000000, 'USD', 'c'),\
    ('friendster', 'Friendster', 465, 'web', 'San Francisco', 'CA', '2008-08-05 00:00:00', 20000000, 'USD', 'd'),\
    ('greystripe', 'Greystripe', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 8900000, 'USD', 'b'),\
    ('slingmedia', 'Sling Media', 0, 'web', 'Foster City', 'CA', '2004-11-01 00:00:00', 10500000, 'USD', 'a'),\
    ('slingmedia', 'Sling Media', 0, 'web', 'Foster City', 'CA', '2005-07-01 00:00:00', 4000000, 'USD', 'a'),\
    ('trulia', 'Trulia', 0, 'web', 'San Francisco', 'CA', '2005-12-01 00:00:00', 5700000, 'USD', 'b'),\
    ('trulia', 'Trulia', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 10000000, 'USD', 'c'),\
    ('trulia', 'Trulia', 0, 'web', 'San Francisco', 'CA', '2005-09-01 00:00:00', 2100000, 'USD', 'a'),\
    ('trulia', 'Trulia', 0, 'web', 'San Francisco', 'CA', '2008-07-10 00:00:00', 15000000, 'USD', 'd'),\
    ('lendingclub', 'Lending Club', 0, 'web', 'Sunnyvale', 'CA', '2007-05-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('lendingclub', 'Lending Club', 0, 'web', 'Sunnyvale', 'CA', '2007-08-01 00:00:00', 10260000, 'USD', 'a'),\
    ('tubemogul', 'tubemogul', 15, 'web', 'Emeryville', 'CA', '2008-02-07 00:00:00', 1500000, 'USD', 'a'),\
    ('gemini', 'Gemini', 0, 'web', 'San Mateo', 'CA', '2007-03-01 00:00:00', 20000000, 'USD', 'a'),\
    ('gemini', 'Gemini', 0, 'web', 'San Mateo', 'CA', '2007-05-01 00:00:00', 5000000, 'USD', 'b'),\
    ('multiverse', 'Multiverse', 0, 'web', 'Mountain View', 'CA', '2007-05-01 00:00:00', 4175000, 'USD', 'a'),\
    ('multiverse', 'Multiverse', 0, 'web', 'Mountain View', 'CA', '2007-04-17 00:00:00', 850000, 'USD', 'angel'),\
    ('limelife', 'LimeLife', 0, 'mobile', 'Menlo Park', 'CA', '2006-03-28 00:00:00', 10000000, 'USD', 'b'),\
    ('limelife', 'LimeLife', 0, 'mobile', 'Menlo Park', 'CA', '2005-08-01 00:00:00', 5000000, 'USD', 'a'),\
    ('limelife', 'LimeLife', 0, 'mobile', 'Menlo Park', 'CA', '2007-05-24 00:00:00', 3900000, 'USD', 'b'),\
    ('opentable', 'OpenTable', 0, 'web', 'San Francisco', 'CA', '1999-05-01 00:00:00', 2000000, 'USD', 'a'),\
    ('opentable', 'OpenTable', 0, 'web', 'San Francisco', 'CA', '2000-01-01 00:00:00', 10000000, 'USD', 'b'),\
    ('opentable', 'OpenTable', 0, 'web', 'San Francisco', 'CA', '2000-10-01 00:00:00', 36000000, 'USD', 'c'),\
    ('rooftopcomedy', 'RooftopComedy', 20, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 2500000, 'USD', 'a'),\
    ('federatedmedia', 'Federated Media', 0, 'web', 'Sausalito', 'CA', '2007-08-01 00:00:00', 4500000, 'USD', 'b'),\
    ('federatedmedia', 'Federated Media', 0, 'web', 'Sausalito', 'CA', '2008-04-01 00:00:00', 50000000, 'USD', 'c'),\
    ('deliveryagent', 'Delivery Agent', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 18500000, 'USD', 'c'),\
    ('deliveryagent', 'Delivery Agent', 0, 'web', 'San Francisco', 'CA', '2006-05-17 00:00:00', 11000000, 'USD', 'b'),\
    ('deliveryagent', 'Delivery Agent', 0, 'web', 'San Francisco', 'CA', '2005-03-29 00:00:00', 5500000, 'USD', 'a'),\
    ('reunion', 'Reunion', 100, 'web', 'Los Angeles', 'CA', '2007-04-01 00:00:00', 25000000, 'USD', 'a'),\
    ('mixercast', 'Mixercast', 0, 'web', 'San Mateo', 'CA', '2005-05-01 00:00:00', 2600000, 'USD', 'a'),\
    ('mixercast', 'Mixercast', 0, 'web', 'San Mateo', 'CA', '2008-01-01 00:00:00', 6000000, 'USD', 'b'),\
    ('mtraks', 'mTraks', 0, 'web', 'San Diego', 'CA', '2007-05-01 00:00:00', 550000, 'USD', 'angel'),\
    ('xora', 'Xora', 0, 'web', 'Mountain View', 'CA', '2001-05-01 00:00:00', 7000000, 'USD', 'a'),\
    ('xora', 'Xora', 0, 'web', 'Mountain View', 'CA', '2003-02-01 00:00:00', 4000000, 'USD', 'b'),\
    ('xora', 'Xora', 0, 'web', 'Mountain View', 'CA', '2007-06-01 00:00:00', 4000000, 'USD', 'debt_round'),\
    ('ooyala', 'Ooyala', 0, 'web', 'Mountain View', 'CA', '2008-01-01 00:00:00', 8500000, 'USD', 'b'),\
    ('saynow', 'SayNow', 14, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 7500000, 'USD', 'a'),\
    ('crunchyroll', 'Crunchyroll', 0, 'web', 'San Francsico', 'CA', '2008-02-01 00:00:00', 4050000, 'USD', 'a'),\
    ('circleup', 'CircleUp', 19, 'web', 'Newport Beach', 'CA', '2007-06-01 00:00:00', 3000000, 'USD', 'a'),\
    ('pixsense', 'PixSense', 0, 'web', 'Santa Clara', 'CA', '2006-12-01 00:00:00', 5400000, 'USD', 'a'),\
    ('pixsense', 'PixSense', 0, 'web', 'Santa Clara', 'CA', '2007-06-01 00:00:00', 2000000, 'USD', 'b'),\
    ('danal', 'Danal', 0, 'web', 'San Jose', 'CA', '2007-06-01 00:00:00', 9500000, 'USD', 'a'),\
    ('hulu', 'hulu', 0, 'web', 'Los Angeles', 'CA', '2007-08-01 00:00:00', 100000000, 'USD', 'a'),\
    ('ourstory', 'OurStory', 0, 'web', 'Mountain View', 'CA', '2006-01-01 00:00:00', 6300000, 'USD', 'a'),\
    ('specificmedia', 'Specificmedia', 0, 'web', 'Ir vine', 'CA', '2006-06-01 00:00:00', 10000000, 'USD', 'a'),\
    ('specificmedia', 'Specificmedia', 0, 'web', 'Ir vine', 'CA', '2007-11-01 00:00:00', 100000000, 'USD', 'b'),\
    ('admob', 'AdMob', 0, 'web', 'San Mateo', 'CA', '2007-03-01 00:00:00', 15000000, 'USD', 'b'),\
    ('mysql', 'MySQL', 0, 'web', 'Cupertino', 'CA', '2006-02-01 00:00:00', 18500000, 'USD', 'c'),\
    ('mysql', 'MySQL', 0, 'web', 'Cupertino', 'CA', '2003-06-01 00:00:00', 19500000, 'USD', 'b'),\
    ('attributor', 'Attributor', 0, 'web', 'Redwood City', 'CA', '2006-01-28 00:00:00', 2000000, 'USD', 'a'),\
    ('attributor', 'Attributor', 0, 'web', 'Redwood City', 'CA', '2008-04-02 00:00:00', 12000000, 'USD', 'c'),\
    ('attributor', 'Attributor', 0, 'web', 'Redwood City', 'CA', '2006-12-18 00:00:00', 8000000, 'USD', 'b'),\
    ('socialmedia', 'Social Media', 15, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 500000, 'USD', 'a'),\
    ('socialmedia', 'Social Media', 15, 'web', 'Palo Alto', 'CA', '2007-10-01 00:00:00', 3500000, 'USD', 'b'),\
    ('demandmedia', 'Demand Media', 0, 'web', 'Santa Monica', 'CA', '2006-05-01 00:00:00', 120000000, 'USD', 'a'),\
    ('demandmedia', 'Demand Media', 0, 'web', 'Santa Monica', 'CA', '2006-09-01 00:00:00', 100000000, 'USD', 'b'),\
    ('demandmedia', 'Demand Media', 0, 'web', 'Santa Monica', 'CA', '2007-09-01 00:00:00', 100000000, 'USD', 'c'),\
    ('demandmedia', 'Demand Media', 0, 'web', 'Santa Monica', 'CA', '2008-03-24 00:00:00', 35000000, 'USD', 'd'),\
    ('tripit', 'TripIt', 8, 'web', 'San Francisco', 'CA', '2007-04-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('pubmatic', 'PubMatic', 35, 'web', 'Palo Alto', 'CA', '2008-01-01 00:00:00', 7000000, 'USD', 'a'),\
    ('mint', 'Mint', 22, 'web', 'Mountain View', 'CA', '2007-10-01 00:00:00', 4700000, 'USD', 'a'),\
    ('mint', 'Mint', 22, 'web', 'Mountain View', 'CA', '2006-10-01 00:00:00', 325000, 'USD', 'seed'),\
    ('mint', 'Mint', 22, 'web', 'Mountain View', 'CA', '2008-03-05 00:00:00', 12000000, 'USD', 'b'),\
    ('app2you', 'app2you', 4, 'web', 'La Jolla', 'CA', '2007-11-20 00:00:00', 255000, 'USD', 'angel'),\
    ('app2you', 'app2you', 4, 'web', 'La Jolla', 'CA', '2006-10-01 00:00:00', 177000, 'USD', 'angel'),\
    ('yourstreet', 'YourStreet', 6, 'web', 'San Francisco', 'CA', '2006-10-01 00:00:00', 250000, 'USD', 'angel'),\
    ('tagworld', 'Tagworld', 0, 'web', 'Santa Monica', 'CA', '2006-03-01 00:00:00', 7500000, 'USD', 'a'),\
    ('tagworld', 'Tagworld', 0, 'web', 'Santa Monica', 'CA', '2006-12-01 00:00:00', 40000000, 'USD', 'b'),\
    ('rotohog', 'RotoHog', 0, 'web', 'Inglewood', 'CA', '2007-08-01 00:00:00', 6000000, 'USD', 'a'),\
    ('thisnext', 'ThisNext', 0, 'web', 'Santa Monica', 'CA', '2008-01-01 00:00:00', 5000000, 'USD', 'b'),\
    ('thisnext', 'ThisNext', 0, 'web', 'Santa Monica', 'CA', '2006-01-01 00:00:00', 2500000, 'USD', 'a'),\
    ('chesspark', 'ChessPark', 0, 'web', 'West Palm Beach', 'CA', '2007-07-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('break', 'Break', 0, 'web', 'Beverly Hills', 'CA', '2007-07-01 00:00:00', 21400000, 'USD', 'a'),\
    ('musestorm', 'MuseStorm', 0, 'web', 'Sunnyvale', 'CA', '2007-07-01 00:00:00', 1000000, 'USD', 'a'),\
    ('mediamachines', 'Media Machines', 0, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 9400000, 'USD', 'a'),\
    ('mig33', 'Ciihui', 0, 'web', 'Burlingame', 'CA', '2007-05-01 00:00:00', 10000000, 'USD', 'a'),\
    ('mig33', 'Ciihui', 0, 'web', 'Burlingame', 'CA', '2008-01-01 00:00:00', 13500000, 'USD', 'b'),\
    ('aliph', 'Aliph', 0, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 5000000, 'USD', 'a'),\
    ('mego', 'mEgo', 15, 'web', 'Los Angeles', 'CA', '2006-05-01 00:00:00', 1100000, 'USD', 'angel'),\
    ('mego', 'mEgo', 15, 'web', 'Los Angeles', 'CA', '2007-07-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('realius', 'Realius', 3, 'web', 'Berkeley', 'CA', '2007-05-01 00:00:00', 500000, 'USD', 'angel'),\
    ('xobni', 'Xobni', 0, 'software', 'San Francisco', 'CA', '2006-06-01 00:00:00', 12000, 'USD', 'seed'),\
    ('xobni', 'Xobni', 0, 'software', 'San Francisco', 'CA', '2006-11-01 00:00:00', 80000, 'USD', 'angel'),\
    ('xobni', 'Xobni', 0, 'software', 'San Francisco', 'CA', '2007-03-01 00:00:00', 4260000, 'USD', 'a'),\
    ('spoke', 'Spoke', 0, 'web', 'San Mateo', 'CA', '2003-11-18 00:00:00', 11700000, 'USD', 'c'),\
    ('spoke', 'Spoke', 0, 'web', 'San Mateo', 'CA', '2003-04-24 00:00:00', 9200000, 'USD', 'b'),\
    ('fixya', 'FixYa', 30, 'web', 'San Mateo', 'CA', '2007-01-01 00:00:00', 2000000, 'USD', 'a'),\
    ('fixya', 'FixYa', 30, 'web', 'San Mateo', 'CA', '2008-03-18 00:00:00', 6000000, 'USD', 'b'),\
    ('mozilla', 'Mozilla', 150, 'web', 'Mountain View', 'CA', '2003-07-15 00:00:00', 2000000, 'USD', 'unattributed'),\
    ('brightqube', 'BrightQube', 0, 'web', 'Carlsbad', 'CA', '2007-05-01 00:00:00', 200000, 'USD', 'angel'),\
    ('buzzdash', 'BuzzDash', 0, 'web', 'Marina del Rey', 'CA', '2006-09-01 00:00:00', 1200000, 'USD', 'angel'),\
    ('urbanmapping', 'Urban Mapping', 9, 'web', 'San Francisco', 'CA', '2006-05-01 00:00:00', 400000, 'USD', 'seed'),\
    ('consortemedia', 'Consorte Media', 0, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 7000000, 'USD', 'b'),\
    ('ffwd', 'ffwd', 0, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 1700000, 'USD', 'a'),\
    ('satisfaction', 'Get Satisfaction', 0, 'web', 'San Francisco', 'CA', '2007-09-01 00:00:00', 1300000, 'USD', 'a'),\
    ('ringcentral', 'RingCentral', 0, 'web', 'Redwood City', 'CA', '2007-09-01 00:00:00', 12000000, 'USD', 'a'),\
    ('ringcentral', 'RingCentral', 0, 'web', 'Redwood City', 'CA', '2008-02-01 00:00:00', 12000000, 'USD', 'b'),\
    ('3jam', '3Jam', 0, 'web', 'Menlo Park', 'CA', '2007-07-01 00:00:00', 4000000, 'USD', 'a'),\
    ('playspan', 'PlaySpan', 0, 'web', 'Santa Clara', 'CA', '2007-09-01 00:00:00', 6500000, 'USD', 'a'),\
    ('grouply', 'Grouply', 6, 'web', 'Redwood City', 'CA', '2007-06-01 00:00:00', 935000, 'USD', 'angel'),\
    ('grouply', 'Grouply', 6, 'web', 'Redwood City', 'CA', '2008-01-01 00:00:00', 365000, 'USD', 'angel'),\
    ('grouply', 'Grouply', 6, 'web', 'Redwood City', 'CA', '2008-01-14 00:00:00', 1300000, 'USD', 'a'),\
    ('graspr', 'Graspr', 0, 'web', 'Sunnyvale', 'CA', '2008-07-10 00:00:00', 2500000, 'USD', 'a'),\
    ('zannel', 'Zannel', 0, 'web', 'San Francisco', 'CA', '2008-06-02 00:00:00', 10000000, 'USD', 'b'),\
    ('deca-tv', 'deca.tv', 0, 'web', 'Santa Monica', 'CA', '2007-09-01 00:00:00', 5000000, 'USD', 'a'),\
    ('zimbra', 'Zimbra', 0, 'web', 'San Mateo', 'CA', '2006-04-01 00:00:00', 14500000, 'USD', 'c'),\
    ('friendfeed', 'FriendFeed', 0, 'web', 'Mountain View', 'CA', '2008-02-26 00:00:00', 5000000, 'USD', 'a'),\
    ('rupture', 'Rupture', 15, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 3000000, 'USD', 'angel'),\
    ('jibjab', 'JibJab', 0, 'web', 'Santa Monica', 'CA', '2007-10-01 00:00:00', 3000000, 'USD', 'b'),\
    ('rubiconproject', 'Rubicon Project', 0, 'web', 'Los Angeles', 'CA', '2007-10-01 00:00:00', 4000000, 'USD', 'a'),\
    ('rubiconproject', 'Rubicon Project', 0, 'web', 'Los Angeles', 'CA', '2007-10-01 00:00:00', 2000000, 'USD', 'debt_round'),\
    ('rubiconproject', 'Rubicon Project', 0, 'web', 'Los Angeles', 'CA', '2008-01-28 00:00:00', 15000000, 'USD', 'b'),\
    ('snaplayout', 'SnapLayout', 0, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 650000, 'USD', 'a'),\
    ('confabb', 'Confabb', 0, 'web', 'unknown', 'CA', '2007-08-01 00:00:00', 200000, 'USD', 'angel'),\
    ('uptake', 'UpTake', 0, 'web', 'Palo Alto', 'CA', '2007-12-18 00:00:00', 4000000, 'USD', 'a'),\
    ('23andme', '23andMe', 30, 'web', 'Mountain View', 'CA', '2007-05-01 00:00:00', 9000000, 'USD', 'a'),\
    ('yume', 'YuMe', 0, 'web', 'Redwood City', 'CA', '2007-10-01 00:00:00', 9000000, 'USD', 'b'),\
    ('mochimedia', 'Mochi Media', 0, 'web', 'San Francisco', 'CA', '2008-03-12 00:00:00', 4000000, 'USD', 'a'),\
    ('mochimedia', 'Mochi Media', 0, 'web', 'San Francisco', 'CA', '2008-06-18 00:00:00', 10000000, 'USD', 'b'),\
    ('blurb', 'Blurb', 0, 'web', 'San Francisco', 'CA', '2006-10-01 00:00:00', 12000000, 'USD', 'b'),\
    ('weatherbill', 'WeatherBill', 0, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 12600000, 'USD', 'a'),\
    ('automattic', 'Automattic', 20, 'web', 'San Francisco', 'CA', '2005-07-01 00:00:00', 1100000, 'USD', 'a'),\
    ('automattic', 'Automattic', 20, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 29500000, 'USD', 'b'),\
    ('twine', 'Radar Networks', 0, 'web', 'San Francisco', 'CA', '2006-04-01 00:00:00', 5000000, 'USD', 'a'),\
    ('twine', 'Radar Networks', 0, 'web', 'San Francisco', 'CA', '2008-02-01 00:00:00', 13000000, 'USD', 'b'),\
    ('veodia', 'Veodia', 15, 'web', 'San Mateo', 'CA', '2008-05-12 00:00:00', 8300000, 'USD', 'a'),\
    ('echosign', 'EchoSign', 0, 'web', 'Palo Alto', 'CA', '2007-10-01 00:00:00', 6000000, 'USD', 'a'),\
    ('rollbase', 'Rollbase', 0, 'web', 'Mountain View', 'CA', '2007-08-01 00:00:00', 400000, 'USD', 'angel'),\
    ('predictify', 'Predictify', 6, 'web', 'Redwood City', 'CA', '2008-03-25 00:00:00', 4300000, 'USD', 'a'),\
    ('trialpay', 'TrialPay', 50, 'web', 'San Francisco', 'CA', '2008-02-01 00:00:00', 12700000, 'USD', 'b'),\
    ('socializr', 'Socializr', 0, 'web', 'San Francisco', 'CA', '2007-09-01 00:00:00', 1500000, 'USD', 'a'),\
    ('eurekster', 'Eurekster', 0, 'web', 'unknown', 'CA', '2004-12-01 00:00:00', 1350000, 'USD', 'a'),\
    ('eurekster', 'Eurekster', 0, 'web', 'unknown', 'CA', '2007-03-13 00:00:00', 5500000, 'USD', 'b'),\
    ('gydget', 'Gydget', 0, 'web', 'San Francisco', 'CA', '2006-08-01 00:00:00', 1000000, 'USD', 'a'),\
    ('pickspal', 'PicksPal', 0, 'web', 'Mountain View', 'CA', '2007-10-01 00:00:00', 3000000, 'USD', 'c'),\
    ('gigya', 'Gigya', 0, 'web', 'Palo Alto', 'CA', '2007-02-01 00:00:00', 3000000, 'USD', 'a'),\
    ('gigya', 'Gigya', 0, 'web', 'Palo Alto', 'CA', '2008-03-09 00:00:00', 9500000, 'USD', 'b'),\
    ('songbird', 'Songbird', 0, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 8000000, 'USD', 'seed'),\
    ('guardiananalytics', 'Guardian Analytics', 0, 'web', 'Los Altos', 'CA', '2007-10-01 00:00:00', 4500000, 'USD', 'b'),\
    ('fora-tv', 'Fora.TV', 0, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 4000000, 'USD', 'a'),\
    ('disqus', 'Disqus', 3, 'web', 'San Francisco', 'CA', '2008-03-18 00:00:00', 500000, 'USD', 'a'),\
    ('sezwho', 'SezWho', 9, 'web', 'Los Altos', 'CA', '2007-10-01 00:00:00', 1000000, 'USD', 'a'),\
    ('yieldbuild', 'YieldBuild', 0, 'web', 'San Francisco', 'CA', '2008-03-03 00:00:00', 6000000, 'USD', 'b'),\
    ('akimbo', 'Akimbo', 0, 'web', 'San Mateo', 'CA', '2006-06-01 00:00:00', 15500000, 'USD', 'c'),\
    ('akimbo', 'Akimbo', 0, 'web', 'San Mateo', 'CA', '2004-07-01 00:00:00', 12000000, 'USD', 'b'),\
    ('akimbo', 'Akimbo', 0, 'web', 'San Mateo', 'CA', '2003-06-01 00:00:00', 4200000, 'USD', 'seed'),\
    ('akimbo', 'Akimbo', 0, 'web', 'San Mateo', 'CA', '2008-02-29 00:00:00', 8000000, 'USD', 'd'),\
    ('laszlosystems', 'Laszlo Systems', 60, 'software', 'San Mateo', 'CA', '2008-03-06 00:00:00', 14600000, 'USD', 'c'),\
    ('laszlosystems', 'Laszlo Systems', 60, 'software', 'San Mateo', 'CA', '2005-04-15 00:00:00', 6250000, 'USD', 'b'),\
    ('brightroll', 'BrightRoll', 0, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 5000000, 'USD', 'b'),\
    ('merchantcircle', 'MerchantCircle', 10, 'web', 'Los Altos', 'CA', '2007-09-01 00:00:00', 10000000, 'USD', 'b'),\
    ('minekey', 'Minekey', 0, 'web', 'Sunnyvale', 'CA', '2007-08-01 00:00:00', 3000000, 'USD', 'a'),\
    ('bookingangel', 'Booking Angel', 0, 'web', 'Hollywood', 'CA', '2007-08-01 00:00:00', 100000, 'USD', 'angel'),\
    ('loopt', 'Loopt', 0, 'web', 'Mountain View', 'CA', '2005-06-01 00:00:00', 6000, 'USD', 'seed'),\
    ('loopt', 'Loopt', 0, 'web', 'Mountain View', 'CA', '2007-07-23 00:00:00', 8250000, 'USD', 'b'),\
    ('fix8', 'Fix8', 0, 'web', 'Sherman Oaks', 'CA', '2007-10-01 00:00:00', 3000000, 'USD', 'a'),\
    ('fix8', 'Fix8', 0, 'web', 'Sherman Oaks', 'CA', '2008-04-03 00:00:00', 2000000, 'USD', 'b'),\
    ('mybuys', 'MyBuys', 0, 'web', 'Redwood Shores', 'CA', '2007-02-21 00:00:00', 2800000, 'USD', 'a'),\
    ('mybuys', 'MyBuys', 0, 'web', 'Redwood Shores', 'CA', '2007-10-08 00:00:00', 10000000, 'USD', 'b'),\
    ('blackarrow', 'BlackArrow', 35, 'web', 'San Mateo', 'CA', '2007-10-01 00:00:00', 12000000, 'USD', 'b'),\
    ('blackarrow', 'BlackArrow', 35, 'web', 'San Mateo', 'CA', '2006-11-01 00:00:00', 14750000, 'USD', 'a'),\
    ('peerflix', 'Peerflix', 0, 'web', 'Palo Alto', 'CA', '2005-10-01 00:00:00', 8000000, 'USD', 'b'),\
    ('peerflix', 'Peerflix', 0, 'web', 'Palo Alto', 'CA', '2005-03-01 00:00:00', 2000000, 'USD', 'a'),\
    ('fatdoor', 'fatdoor', 0, 'web', 'Palo Alto', 'CA', '2007-11-01 00:00:00', 5500000, 'USD', 'a'),\
    ('fatdoor', 'fatdoor', 0, 'web', 'Palo Alto', 'CA', '2007-02-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('fatdoor', 'fatdoor', 0, 'web', 'Palo Alto', 'CA', '2007-05-01 00:00:00', 500000, 'USD', 'debt_round'),\
    ('verimatrix', 'Verimatrix', 100, 'web', 'San Diego', 'CA', '2006-06-01 00:00:00', 8000000, 'USD', 'b'),\
    ('verimatrix', 'Verimatrix', 100, 'web', 'San Diego', 'CA', '2007-10-01 00:00:00', 5000000, 'USD', 'c'),\
    ('billeo', 'Billeo', 0, 'web', 'Santa Clara', 'CA', '2007-11-01 00:00:00', 7000000, 'USD', 'b'),\
    ('billeo', 'Billeo', 0, 'web', 'Santa Clara', 'CA', '2006-04-01 00:00:00', 4000000, 'USD', 'a'),\
    ('caring-com', 'Caring.com', 0, 'web', 'San Mateo', 'CA', '2007-09-01 00:00:00', 6000000, 'USD', 'a'),\
    ('ask-com', 'Ask.com', 475, 'web', 'Oakland', 'CA', '1999-03-01 00:00:00', 25000000, 'USD', 'a'),\
    ('projectplaylist', 'Project Playlist', 0, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 3000000, 'USD', 'a'),\
    ('blowtorch', 'Blowtorch', 0, 'web', 'Sausalito', 'CA', '2007-11-01 00:00:00', 50000000, 'USD', 'a'),\
    ('gamelayers', 'GameLayers', 0, 'web', 'San Francisco', 'CA', '2007-10-01 00:00:00', 500000, 'USD', 'seed'),\
    ('mogreet', 'MoGreet', 0, 'web', 'Venice', 'CA', '2007-08-01 00:00:00', 1200000, 'USD', 'a'),\
    ('mogreet', 'MoGreet', 0, 'web', 'Venice', 'CA', '2008-06-20 00:00:00', 2500000, 'USD', 'b'),\
    ('yourtrumanshow', 'YourTrumanShow', 0, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 1300000, 'USD', 'angel'),\
    ('apprema', 'Apprema', 0, 'web', 'Sunnyvale', 'CA', '2007-11-01 00:00:00', 800000, 'USD', 'debt_round'),\
    ('trustedid', 'TrustedID', 100, 'web', 'Redwood City', 'CA', '2007-01-01 00:00:00', 5000000, 'USD', 'a'),\
    ('trustedid', 'TrustedID', 100, 'web', 'Redwood City', 'CA', '2007-10-18 00:00:00', 10000000, 'USD', 'b'),\
    ('seeqpod', 'Seeqpod', 25, 'web', 'Emeryville', 'CA', '2008-04-01 00:00:00', 7000000, 'USD', 'angel'),\
    ('quantenna', 'Quantenna', 0, 'web', 'Sunnyvale', 'CA', '2007-11-01 00:00:00', 12700000, 'USD', 'b'),\
    ('qwaq', 'Qwaq', 0, 'web', 'Palo Alto', 'CA', '2007-11-01 00:00:00', 7000000, 'USD', 'b'),\
    ('uber', 'uber', 24, 'web', 'Beverly Hills', 'CA', '2008-05-26 00:00:00', 7600000, 'USD', 'b'),\
    ('cashview', 'CashView', 0, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 6500000, 'USD', 'b'),\
    ('crossloop', 'CrossLoop', 8, 'web', 'Monterey', 'CA', '2007-12-01 00:00:00', 3000000, 'USD', 'a'),\
    ('dapper', 'Dapper', 20, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 3000000, 'USD', 'a'),\
    ('anchorintelligence', 'Anchor Intelligence', 0, 'web', 'Mountain View', 'CA', '2007-01-01 00:00:00', 2000000, 'USD', 'a'),\
    ('anchorintelligence', 'Anchor Intelligence', 0, 'web', 'Mountain View', 'CA', '2007-09-01 00:00:00', 4000000, 'USD', 'b'),\
    ('edgecast', 'EdgeCast', 30, 'web', 'Los Angeles', 'CA', '2007-12-01 00:00:00', 6000000, 'USD', 'b'),\
    ('edgecast', 'EdgeCast', 30, 'web', 'Los Angeles', 'CA', '2007-06-01 00:00:00', 4000000, 'USD', 'a'),\
    ('kosmix', 'Kosmix', 0, 'web', 'Mountain View', 'CA', '2007-12-01 00:00:00', 10000000, 'USD', 'c'),\
    ('goodreads', 'GoodReads', 0, 'web', 'Santa Monica', 'CA', '2007-11-01 00:00:00', 750000, 'USD', 'angel'),\
    ('ribbit', 'Ribbit', 28, 'web', 'Mountain View', 'CA', '2007-12-01 00:00:00', 10000000, 'USD', 'b'),\
    ('ribbit', 'Ribbit', 28, 'web', 'Mountain View', 'CA', '2006-10-01 00:00:00', 3000000, 'USD', 'a'),\
    ('jobvite', 'Jobvite', 0, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 7200000, 'USD', 'a'),\
    ('juicewireless', 'Juice Wireless', 0, 'web', 'Los Angeles', 'CA', '2005-04-01 00:00:00', 500000, 'USD', 'a'),\
    ('juicewireless', 'Juice Wireless', 0, 'web', 'Los Angeles', 'CA', '2006-01-01 00:00:00', 1500000, 'USD', 'a'),\
    ('juicewireless', 'Juice Wireless', 0, 'web', 'Los Angeles', 'CA', '2006-07-01 00:00:00', 4000000, 'USD', 'a'),\
    ('juicewireless', 'Juice Wireless', 0, 'web', 'Los Angeles', 'CA', '2007-03-01 00:00:00', 3300000, 'USD', 'b'),\
    ('juicewireless', 'Juice Wireless', 0, 'web', 'Los Angeles', 'CA', '2007-12-01 00:00:00', 6000000, 'USD', 'c'),\
    ('qik', 'Qik', 22, 'web', 'Foster City', 'CA', '2008-04-09 00:00:00', 3000000, 'USD', 'b'),\
    ('playfirst', 'PlayFirst', 0, 'web', 'San Francisco', 'CA', '2007-12-01 00:00:00', 16500000, 'USD', 'c'),\
    ('playfirst', 'PlayFirst', 0, 'web', 'San Francisco', 'CA', '2006-01-01 00:00:00', 5000000, 'USD', 'b'),\
    ('mobissimo', 'Mobissimo', 15, 'web', 'San Francisco', 'CA', '2004-04-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('chumby', 'Chumby', 20, 'hardware', 'San Diego', 'CA', '2006-12-01 00:00:00', 5000000, 'USD', 'a'),\
    ('chumby', 'Chumby', 20, 'hardware', 'San Diego', 'CA', '2008-03-31 00:00:00', 12500000, 'USD', 'b'),\
    ('ugobe', 'UGOBE', 0, 'web', 'Emeryville', 'CA', '2006-10-01 00:00:00', 8000000, 'USD', 'b'),\
    ('ausra', 'Ausra', 4, 'web', 'Palo Alto', 'CA', '2007-09-01 00:00:00', 40000000, 'USD', 'a'),\
    ('causes', 'Causes', 0, 'web', 'Berkeley', 'CA', '2007-01-01 00:00:00', 2350000, 'USD', 'a'),\
    ('causes', 'Causes', 0, 'web', 'Berkeley', 'CA', '2008-03-01 00:00:00', 5000000, 'USD', 'b'),\
    ('nanosolar', 'Nanosolar', 0, 'web', 'San Jose', 'CA', '2005-05-01 00:00:00', 20000000, 'USD', 'b'),\
    ('nanosolar', 'Nanosolar', 0, 'web', 'San Jose', 'CA', '2006-06-01 00:00:00', 75000000, 'USD', 'c'),\
    ('shozu', 'Shozu', 0, 'web', 'San Francisco', 'CA', '2005-05-01 00:00:00', 12000000, 'USD', 'b'),\
    ('shozu', 'Shozu', 0, 'web', 'San Francisco', 'CA', '2008-01-29 00:00:00', 12000000, 'USD', 'c'),\
    ('tesla-motors', 'Tesla Motors', 270, 'hardware', 'San Carlos', 'CA', '2006-05-01 00:00:00', 40000000, 'USD', 'c'),\
    ('tesla-motors', 'Tesla Motors', 270, 'hardware', 'San Carlos', 'CA', '2007-05-01 00:00:00', 45000000, 'USD', 'd'),\
    ('tesla-motors', 'Tesla Motors', 270, 'hardware', 'San Carlos', 'CA', '2004-04-01 00:00:00', 7500000, 'USD', 'a'),\
    ('tesla-motors', 'Tesla Motors', 270, 'hardware', 'San Carlos', 'CA', '2005-02-01 00:00:00', 13000000, 'USD', 'b'),\
    ('tesla-motors', 'Tesla Motors', 270, 'hardware', 'San Carlos', 'CA', '2008-02-08 00:00:00', 40000000, 'USD', 'e'),\
    ('bunchball', 'Bunchball', 0, 'web', 'Redwood City', 'CA', '2006-10-01 00:00:00', 2000000, 'USD', 'a'),\
    ('bunchball', 'Bunchball', 0, 'web', 'Redwood City', 'CA', '2008-04-13 00:00:00', 4000000, 'USD', 'b'),\
    ('pinger', 'Pinger', 0, 'web', 'San Jose', 'CA', '2006-12-01 00:00:00', 8000000, 'USD', 'b'),\
    ('hooja', 'Hooja', 0, 'web', 'Palo Alto', 'CA', '2007-12-01 00:00:00', 1250000, 'USD', 'seed'),\
    ('travelmuse', 'TravelMuse', 11, 'web', 'Los Altos', 'CA', '2007-05-01 00:00:00', 3000000, 'USD', 'a'),\
    ('cooking-com', 'Cooking.com', 0, 'web', 'Santa Monica', 'CA', '2008-01-01 00:00:00', 7000000, 'USD', 'debt_round'),\
    ('cooking-com', 'Cooking.com', 0, 'web', 'Santa Monica', 'CA', '2000-05-01 00:00:00', 35000000, 'USD', 'e'),\
    ('cooking-com', 'Cooking.com', 0, 'web', 'Santa Monica', 'CA', '1999-04-01 00:00:00', 16000000, 'USD', 'b'),\
    ('cooking-com', 'Cooking.com', 0, 'web', 'Santa Monica', 'CA', '1999-10-01 00:00:00', 30000000, 'USD', 'c'),\
    ('meraki', 'Meraki', 0, 'web', 'Mountain View', 'CA', '2007-02-01 00:00:00', 5000000, 'USD', 'a'),\
    ('meraki', 'Meraki', 0, 'web', 'Mountain View', 'CA', '2008-01-01 00:00:00', 20000000, 'USD', 'b'),\
    ('sugarcrm', 'SugarCRM', 0, 'web', 'Cupertino', 'CA', '2008-01-01 00:00:00', 14500000, 'USD', 'd'),\
    ('sugarcrm', 'SugarCRM', 0, 'web', 'Cupertino', 'CA', '2005-10-01 00:00:00', 18770000, 'USD', 'c'),\
    ('sugarcrm', 'SugarCRM', 0, 'web', 'Cupertino', 'CA', '2004-08-01 00:00:00', 2000000, 'USD', 'a'),\
    ('sugarcrm', 'SugarCRM', 0, 'web', 'Cupertino', 'CA', '2005-02-01 00:00:00', 5750000, 'USD', 'b'),\
    ('puddingmedia', 'Pudding Media', 11, 'software', 'San Jose', 'CA', '2008-01-01 00:00:00', 8000000, 'USD', 'a'),\
    ('4homemedia', '4HomeMedia', 10, 'web', 'Sunnyvale', 'CA', '2007-01-01 00:00:00', 2850000, 'USD', 'a'),\
    ('pageonce', 'Pageonce', 0, 'web', 'Palo Alto', 'CA', '2008-01-01 00:00:00', 1500000, 'USD', 'a'),\
    ('bluepulse', 'bluepulse', 0, 'web', 'San Mateo', 'CA', '2007-04-01 00:00:00', 6000000, 'USD', 'a'),\
    ('mogad', 'Mogad', 2, 'web', 'San Francisco', 'CA', '2007-08-01 00:00:00', 500000, 'USD', 'a'),\
    ('devicevm', 'DeviceVM', 0, 'web', 'San Jose', 'CA', '2006-08-01 00:00:00', 10000000, 'USD', 'a'),\
    ('devicevm', 'DeviceVM', 0, 'web', 'San Jose', 'CA', '2007-10-01 00:00:00', 10000000, 'USD', 'b'),\
    ('outspark', 'Outspark', 0, 'web', 'San Francisco', 'CA', '2007-04-11 00:00:00', 4000000, 'USD', 'a'),\
    ('outspark', 'Outspark', 0, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 11000000, 'USD', 'b'),\
    ('engineyard', 'Engine Yard', 60, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 3500000, 'USD', 'a'),\
    ('engineyard', 'Engine Yard', 60, 'web', 'San Francisco', 'CA', '2008-07-13 00:00:00', 15000000, 'USD', 'b'),\
    ('plymedia', 'PLYmedia', 0, 'web', 'Palo Alto', 'CA', '2006-10-01 00:00:00', 2500000, 'USD', 'a'),\
    ('plymedia', 'PLYmedia', 0, 'web', 'Palo Alto', 'CA', '2008-07-01 00:00:00', 6000000, 'USD', 'b'),\
    ('fabrik', 'fabrik', 175, 'web', 'San Mateo', 'CA', '2005-09-01 00:00:00', 4100000, 'USD', 'a'),\
    ('fabrik', 'fabrik', 175, 'web', 'San Mateo', 'CA', '2006-06-01 00:00:00', 8000000, 'USD', 'b'),\
    ('fabrik', 'fabrik', 175, 'web', 'San Mateo', 'CA', '2007-02-01 00:00:00', 14300000, 'USD', 'c'),\
    ('fabrik', 'fabrik', 175, 'web', 'San Mateo', 'CA', '2007-05-01 00:00:00', 24900000, 'USD', 'd'),\
    ('widgetbox', 'Widgetbox', 0, 'web', 'San Francisco', 'CA', '2006-06-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('widgetbox', 'Widgetbox', 0, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 8000000, 'USD', 'b'),\
    ('widgetbox', 'Widgetbox', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('razorgator', 'RazorGator', 0, 'web', 'Los Angeles', 'CA', '2005-03-01 00:00:00', 26000000, 'USD', 'a'),\
    ('razorgator', 'RazorGator', 0, 'web', 'Los Angeles', 'CA', '2006-03-01 00:00:00', 22800000, 'USD', 'b'),\
    ('oversee', 'OverSee', 150, 'web', 'Los Angeles', 'CA', '2007-01-01 00:00:00', 60000000, 'USD', 'debt_round'),\
    ('oversee', 'OverSee', 150, 'web', 'Los Angeles', 'CA', '2008-01-01 00:00:00', 150000000, 'USD', 'a'),\
    ('zynga', 'Zynga', 90, 'other', 'San Francisco', 'CA', '2008-01-01 00:00:00', 10000000, 'USD', 'a'),\
    ('zynga', 'Zynga', 90, 'other', 'San Francisco', 'CA', '2008-07-22 00:00:00', 29000000, 'USD', 'b'),\
    ('smaato', 'Smaato', 0, 'web', 'San Mateo', 'CA', '2008-01-01 00:00:00', 3500000, 'USD', 'a'),\
    ('credit-karma', 'Credit Karma', 4, 'web', 'San Francisco', 'CA', '2007-05-01 00:00:00', 750000, 'USD', 'angel'),\
    ('greenplum', 'Greenplum', 0, 'web', 'San Mateo', 'CA', '2008-01-01 00:00:00', 27000000, 'USD', 'c'),\
    ('greenplum', 'Greenplum', 0, 'web', 'San Mateo', 'CA', '2007-02-01 00:00:00', 15000000, 'USD', 'b'),\
    ('greenplum', 'Greenplum', 0, 'web', 'San Mateo', 'CA', '2006-03-01 00:00:00', 15000000, 'USD', 'a'),\
    ('greenplum', 'Greenplum', 0, 'web', 'San Mateo', 'CA', '2007-02-01 00:00:00', 4000000, 'USD', 'debt_round'),\
    ('amobee', 'Amobee', 0, 'web', 'Redwood City', 'CA', '2006-11-01 00:00:00', 5000000, 'USD', 'a'),\
    ('amobee', 'Amobee', 0, 'web', 'Redwood City', 'CA', '2007-01-01 00:00:00', 15000000, 'USD', 'b'),\
    ('webmynd', 'WebMynd', 0, 'web', 'San Francisco', 'CA', '2008-07-01 00:00:00', 250000, 'USD', 'angel'),\
    ('currenttv', 'Current Media', 0, 'web', 'San Francisco', 'CA', '2005-07-29 00:00:00', 15000000, 'USD', 'a'),\
    ('heroku', 'Heroku', 3, 'web', 'San Francisco', 'CA', '2008-01-01 00:00:00', 20000, 'USD', 'seed'),\
    ('heroku', 'Heroku', 3, 'web', 'San Francisco', 'CA', '2008-05-08 00:00:00', 3000000, 'USD', 'a'),\
    ('lookery', 'Lookery', 9, 'web', 'San Francisco', 'CA', '2008-02-07 00:00:00', 900000, 'USD', 'seed'),\
    ('mill-river-labs', 'Mill River Labs', 5, 'web', 'Mountain View', 'CA', '2008-01-01 00:00:00', 900000, 'USD', 'angel'),\
    ('grouptivity', 'Grouptivity', 10, 'web', 'San Mateo', 'CA', '2006-05-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('aductions', 'Aductions', 5, 'web', 'San Jose', 'CA', '2007-07-05 00:00:00', 100000, 'USD', 'seed'),\
    ('venturebeat', 'VentureBeat', 0, 'web', 'unknown', 'CA', '2008-02-11 00:00:00', 320000, 'USD', 'seed'),\
    ('collarity', 'Collarity', 30, 'web', 'Palo Alto', 'CA', '2008-02-01 00:00:00', 7800000, 'USD', 'b'),\
    ('rocketon', 'RocketOn', 0, 'web', 'South San Francisco', 'CA', '2008-02-01 00:00:00', 5800000, 'USD', 'b'),\
    ('what-they-like', 'What They Like', 10, 'web', 'San Francisco', 'CA', '2007-08-29 00:00:00', 990000, 'USD', 'a'),\
    ('gumgum', 'GumGum', 5, 'web', 'Los Angeles', 'CA', '2007-12-01 00:00:00', 225000, 'USD', 'seed'),\
    ('gumgum', 'GumGum', 5, 'web', 'Los Angeles', 'CA', '2008-07-21 00:00:00', 1000000, 'USD', 'a'),\
    ('snap-technologies', 'Snap Technologies', 32, 'web', 'Pasadena', 'CA', '2005-07-01 00:00:00', 10000000, 'USD', 'a'),\
    ('twofish', 'TwoFish', 0, 'web', 'Redwood City', 'CA', '2007-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('three-rings', 'Three Rings', 30, 'web', 'San Francisco', 'CA', '2008-03-03 00:00:00', 3500000, 'USD', 'a'),\
    ('smalltown', 'Smalltown', 8, 'web', 'San Mateo', 'CA', '2005-11-01 00:00:00', 4000000, 'USD', 'a'),\
    ('sparkplay-media', 'Sparkplay Media', 0, 'web', 'Mill Valley', 'CA', '2008-02-01 00:00:00', 4250000, 'USD', 'a'),\
    ('mog', 'MOG', 0, 'web', 'Berkeley', 'CA', '2007-03-01 00:00:00', 1800000, 'USD', 'angel'),\
    ('mog', 'MOG', 0, 'web', 'Berkeley', 'CA', '2008-04-29 00:00:00', 2800000, 'USD', 'a'),\
    ('social-gaming-network', 'Social Gaming Network', 0, 'web', 'Palo Alto', 'CA', '2008-05-13 00:00:00', 15000000, 'USD', 'a'),\
    ('danger', 'Danger', 0, 'software', 'Palo Alto', 'CA', '2001-10-01 00:00:00', 36000000, 'USD', 'b'),\
    ('danger', 'Danger', 0, 'software', 'Palo Alto', 'CA', '2003-02-01 00:00:00', 35000000, 'USD', 'd'),\
    ('danger', 'Danger', 0, 'software', 'Palo Alto', 'CA', '2004-07-01 00:00:00', 37000000, 'USD', 'd'),\
    ('coverity', 'Coverity', 0, 'software', 'San Francisco', 'CA', '2008-02-01 00:00:00', 22000000, 'USD', 'a'),\
    ('genietown', 'GenieTown', 8, 'web', 'Palo Alto', 'CA', '2007-10-01 00:00:00', 2000000, 'USD', 'a'),\
    ('redux', 'Redux', 11, 'web', 'Berkeley', 'CA', '2007-03-01 00:00:00', 1650000, 'USD', 'seed'),\
    ('redux', 'Redux', 11, 'web', 'Berkeley', 'CA', '2008-04-07 00:00:00', 6500000, 'USD', 'a'),\
    ('evernote', 'Evernote', 0, 'software', 'Sunnyvale', 'CA', '2006-03-01 00:00:00', 6000000, 'USD', 'angel'),\
    ('evernote', 'Evernote', 0, 'software', 'Sunnyvale', 'CA', '2007-09-01 00:00:00', 3000000, 'USD', 'angel'),\
    ('numobiq', 'Numobiq', 0, 'mobile', 'Pleasanton', 'CA', '2008-02-08 00:00:00', 4500000, 'USD', 'a'),\
    ('goldspot-media', 'GoldSpot Media', 0, 'mobile', 'Sunnyvale', 'CA', '2008-01-23 00:00:00', 3000000, 'USD', 'a'),\
    ('mobixell', 'Mobixell', 0, 'mobile', 'Cupertino', 'CA', '2008-07-08 00:00:00', 6000000, 'USD', 'a'),\
    ('ad-infuse', 'Ad Infuse', 0, 'mobile', 'San Francisco', 'CA', '2008-01-23 00:00:00', 12000000, 'USD', 'b'),\
    ('ad-infuse', 'Ad Infuse', 0, 'mobile', 'San Francisco', 'CA', '2006-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('sendme', 'SendMe', 0, 'mobile', 'San Francisco', 'CA', '2006-12-01 00:00:00', 6000000, 'USD', 'b'),\
    ('sendme', 'SendMe', 0, 'mobile', 'San Francisco', 'CA', '2008-03-18 00:00:00', 15000000, 'USD', 'c'),\
    ('tiny-pictures', 'Tiny Pictures', 15, 'mobile', 'San Francisco', 'CA', '2007-08-01 00:00:00', 4000000, 'USD', 'a'),\
    ('tiny-pictures', 'Tiny Pictures', 15, 'mobile', 'San Francisco', 'CA', '2008-02-01 00:00:00', 7200000, 'USD', 'b'),\
    ('flurry', 'flurry', 0, 'mobile', 'San Francisco', 'CA', '2007-03-08 00:00:00', 3500000, 'USD', 'a'),\
    ('sharpcast', 'Sharpcast', 0, 'unknown', 'Palo Alto', 'CA', '2006-03-08 00:00:00', 13500000, 'USD', 'a'),\
    ('sharpcast', 'Sharpcast', 0, 'unknown', 'Palo Alto', 'CA', '2006-03-08 00:00:00', 3000000, 'USD', 'b'),\
    ('teneros', 'Teneros', 0, 'software', 'Mountain View', 'CA', '2004-07-01 00:00:00', 7000000, 'USD', 'a'),\
    ('teneros', 'Teneros', 0, 'software', 'Mountain View', 'CA', '2005-03-01 00:00:00', 17500000, 'USD', 'b'),\
    ('teneros', 'Teneros', 0, 'software', 'Mountain View', 'CA', '2006-04-01 00:00:00', 20000000, 'USD', 'c'),\
    ('teneros', 'Teneros', 0, 'software', 'Mountain View', 'CA', '2008-01-01 00:00:00', 40000000, 'USD', 'd'),\
    ('photocrank', 'PhotoCrank', 3, 'web', 'Palo Alto', 'CA', '2007-04-01 00:00:00', 250000, 'USD', 'seed'),\
    ('yodlee', 'Yodlee', 0, 'web', 'Redwood City', 'CA', '2008-06-04 00:00:00', 35000000, 'USD', 'unattributed'),\
    ('sliderocket', 'SlideRocket', 5, 'web', 'San Francisco', 'CA', '2007-12-31 00:00:00', 2000000, 'USD', 'a'),\
    ('surf-canyon', 'Surf Canyon', 3, 'software', 'Oakland', 'CA', '2007-07-31 00:00:00', 250000, 'USD', 'seed'),\
    ('surf-canyon', 'Surf Canyon', 3, 'software', 'Oakland', 'CA', '2008-05-08 00:00:00', 600000, 'USD', 'seed'),\
    ('central-desktop', 'Central Desktop', 0, 'web', 'Pasadena', 'CA', '2008-04-16 00:00:00', 7000000, 'USD', 'a'),\
    ('opendns', 'OpenDNS', 15, 'web', 'San Francisco', 'CA', '2006-06-01 00:00:00', 2000000, 'USD', 'a'),\
    ('coveo', 'Coveo', 0, 'web', 'Palo Alto', 'CA', '2008-03-06 00:00:00', 2500000, 'USD', 'a'),\
    ('vizu', 'Vizu', 0, 'web', 'San Francisco', 'CA', '2006-02-20 00:00:00', 1000000, 'USD', 'a'),\
    ('vizu', 'Vizu', 0, 'web', 'San Francisco', 'CA', '2007-01-31 00:00:00', 2900000, 'USD', 'b'),\
    ('taltopia', 'Taltopia', 0, 'web', 'Los Angeles', 'CA', '2008-03-08 00:00:00', 800000, 'USD', 'angel'),\
    ('kapow-technologies', 'Kapow Technologies', 0, 'web', 'Palo Alto', 'CA', '2008-03-06 00:00:00', 11600000, 'USD', 'c'),\
    ('kapow-technologies', 'Kapow Technologies', 0, 'web', 'Palo Alto', 'CA', '2005-02-01 00:00:00', 5100000, 'EUR', 'b'),\
    ('programmermeetdesigner-com', 'ProgrammerMeetDesigner.com', 4, 'web', 'Los Angeles', 'CA', '2007-12-12 00:00:00', 500000, 'USD', 'seed'),\
    ('liveops', 'LiveOps', 0, 'web', 'Santa Clara', 'CA', '2004-04-01 00:00:00', 22000000, 'USD', 'b'),\
    ('liveops', 'LiveOps', 0, 'web', 'Santa Clara', 'CA', '2007-02-13 00:00:00', 28000000, 'USD', 'c'),\
    ('clickpass', 'Clickpass', 0, 'web', 'San Francisco', 'CA', '2007-06-01 00:00:00', 20000, 'USD', 'seed'),\
    ('searchme', 'SearchMe', 52, 'web', 'Mountain View', 'CA', '2005-07-01 00:00:00', 400000, 'USD', 'a'),\
    ('searchme', 'SearchMe', 52, 'web', 'Mountain View', 'CA', '2006-01-01 00:00:00', 3600000, 'USD', 'b'),\
    ('searchme', 'SearchMe', 52, 'web', 'Mountain View', 'CA', '2007-06-01 00:00:00', 12000000, 'USD', 'c'),\
    ('searchme', 'SearchMe', 52, 'web', 'Mountain View', 'CA', '2007-10-01 00:00:00', 15000000, 'USD', 'd'),\
    ('searchme', 'SearchMe', 52, 'web', 'Mountain View', 'CA', '2008-05-01 00:00:00', 12600000, 'USD', 'e'),\
    ('accountnow', 'AccountNow', 0, 'web', 'San Ramon', 'CA', '2007-06-29 00:00:00', 12750000, 'USD', 'c'),\
    ('dailystrength', 'DailyStrength', 14, 'web', 'Los Angeles', 'CA', '2007-05-18 00:00:00', 4000000, 'USD', 'a'),\
    ('popularmedia', 'PopularMedia', 0, 'web', 'San Francisco', 'CA', '2007-03-01 00:00:00', 4250000, 'USD', 'unattributed'),\
    ('popularmedia', 'PopularMedia', 0, 'web', 'San Francisco', 'CA', '2008-07-28 00:00:00', 8000000, 'USD', 'c'),\
    ('clarizen', 'Clarizen', 5, 'web', 'San Mateo', 'CA', '2008-05-01 00:00:00', 9000000, 'USD', 'b'),\
    ('sellpoint', 'SellPoint', 0, 'web', 'San Ramon', 'CA', '2008-02-27 00:00:00', 7000000, 'USD', 'a'),\
    ('livedeal', 'LiveDeal', 0, 'web', 'Santa Clara', 'CA', '2005-10-26 00:00:00', 4900000, 'USD', 'a'),\
    ('neoedge-networks', 'NeoEdge Networks', 0, 'web', 'Mountain View', 'CA', '2007-06-08 00:00:00', 3000000, 'USD', 'b'),\
    ('zuora', 'Zuora', 30, 'web', 'Redwood City', 'CA', '2008-03-13 00:00:00', 6500000, 'USD', 'a'),\
    ('jivox', 'Jivox', 0, 'web', 'San Mateo', 'CA', '2008-03-10 00:00:00', 2700000, 'USD', 'seed'),\
    ('jivox', 'Jivox', 0, 'web', 'San Mateo', 'CA', '2008-06-16 00:00:00', 10700000, 'USD', 'a'),\
    ('elastra', 'Elastra', 0, 'unknown', 'San Francisco', 'CA', '2007-08-15 00:00:00', 2600000, 'USD', 'a'),\
    ('kwiry', 'kwiry', 3, 'mobile', 'San Francisco', 'CA', '2007-03-20 00:00:00', 1000000, 'USD', 'a'),\
    ('supplyframe', 'SupplyFrame', 25, 'web', 'Pasadena', 'CA', '2007-06-21 00:00:00', 7000000, 'USD', 'b'),\
    ('inmage-systems', 'InMage Systems', 0, 'software', 'Santa Clara', 'CA', '2007-07-10 00:00:00', 10000000, 'USD', 'b'),\
    ('authenticlick', 'Authenticlick', 20, 'web', 'Los Angeles', 'CA', '2006-02-01 00:00:00', 5000000, 'USD', 'a'),\
    ('interneer', 'Interneer', 6, 'web', 'Agoura Hills', 'CA', '2007-06-06 00:00:00', 2000000, 'USD', 'angel'),\
    ('interneer', 'Interneer', 6, 'web', 'Agoura Hills', 'CA', '2007-03-11 00:00:00', 1200000, 'USD', 'a'),\
    ('handipoints', 'Handipoints', 0, 'web', 'Oakland', 'CA', '2007-05-02 00:00:00', 250000, 'USD', 'seed'),\
    ('handipoints', 'Handipoints', 0, 'web', 'Oakland', 'CA', '2008-05-02 00:00:00', 550000, 'USD', 'angel'),\
    ('adconion-media-group', 'Adconion Media Group', 150, 'web', 'Santa Monica', 'CA', '2008-02-24 00:00:00', 80000000, 'USD', 'c'),\
    ('zend-technologies', 'Zend Technologies', 0, 'unknown', 'Cupertino', 'CA', '2006-08-28 00:00:00', 20000000, 'USD', 'd'),\
    ('zend-technologies', 'Zend Technologies', 0, 'unknown', 'Cupertino', 'CA', '2004-07-19 00:00:00', 8000000, 'USD', 'c'),\
    ('slideshare', 'SlideShare', 10, 'web', 'San Francisco', 'CA', '2008-05-07 00:00:00', 3000000, 'USD', 'a'),\
    ('intent', 'Intent', 6, 'web', 'Santa Monica', 'CA', '2008-02-01 00:00:00', 250000, 'USD', 'angel'),\
    ('wikimedia-foundation', 'Wikimedia Foundation', 0, 'web', 'San Francisco', 'CA', '2008-03-25 00:00:00', 3000000, 'USD', 'unattributed'),\
    ('clearwell-systems', 'Clearwell Systems', 75, 'software', 'Mountain View', 'CA', '2007-08-22 00:00:00', 17000000, 'USD', 'c'),\
    ('become', 'Become', 0, 'web', 'Mountain View', 'CA', '2008-03-25 00:00:00', 17500000, 'USD', 'c'),\
    ('become', 'Become', 0, 'web', 'Mountain View', 'CA', '2008-07-17 00:00:00', 8000000, 'USD', 'd'),\
    ('bubble-motion', 'Bubble Motion', 0, 'mobile', 'Mountain View', 'CA', '2008-03-26 00:00:00', 14000000, 'USD', 'b'),\
    ('bubble-motion', 'Bubble Motion', 0, 'mobile', 'Mountain View', 'CA', '2006-09-11 00:00:00', 10000000, 'USD', 'a'),\
    ('perfect-market', 'Perfect Market', 0, 'web', 'Pasadena', 'CA', '2008-07-09 00:00:00', 15600000, 'USD', 'a'),\
    ('glassdoor', 'Glassdoor', 12, 'web', 'Sausalito', 'CA', '2008-03-27 00:00:00', 3000000, 'USD', 'b'),\
    ('support-space', 'Support Space', 0, 'web', 'Redwood City', 'CA', '2006-10-24 00:00:00', 4250000, 'USD', 'a'),\
    ('xoopit', 'Xoopit', 0, 'web', 'San Francisco', 'CA', '2006-12-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('xoopit', 'Xoopit', 0, 'web', 'San Francisco', 'CA', '2008-04-02 00:00:00', 5000000, 'USD', 'a'),\
    ('reality-digital', 'Reality Digital', 0, 'software', 'San Francisco', 'CA', '2005-11-01 00:00:00', 2000000, 'USD', 'a'),\
    ('reality-digital', 'Reality Digital', 0, 'software', 'San Francisco', 'CA', '2008-03-31 00:00:00', 6300000, 'USD', 'b'),\
    ('cloud9-analytics', 'Cloud9 Analytics', 0, 'web', 'San Mateo', 'CA', '2007-07-25 00:00:00', 5000000, 'USD', 'a'),\
    ('netcipia', 'Netcipia', 3, 'web', 'Palo Alto', 'CA', '2007-08-30 00:00:00', 200000, 'USD', 'angel'),\
    ('icontrol', 'iControl', 0, 'web', 'Palo Alto', 'CA', '2008-04-02 00:00:00', 15500000, 'USD', 'b'),\
    ('icontrol', 'iControl', 0, 'web', 'Palo Alto', 'CA', '2006-04-26 00:00:00', 5000000, 'USD', 'a'),\
    ('frengo', 'Frengo', 0, 'web', 'San Mateo', 'CA', '2006-12-01 00:00:00', 2300000, 'USD', 'a'),\
    ('frengo', 'Frengo', 0, 'web', 'San Mateo', 'CA', '2007-05-03 00:00:00', 5700000, 'USD', 'b'),\
    ('mblox', 'mBlox', 0, 'mobile', 'Sunnyvale', 'CA', '2006-02-23 00:00:00', 25000000, 'USD', 'd'),\
    ('mblox', 'mBlox', 0, 'mobile', 'Sunnyvale', 'CA', '2005-03-09 00:00:00', 7500000, 'USD', 'c'),\
    ('mblox', 'mBlox', 0, 'mobile', 'Sunnyvale', 'CA', '2004-07-09 00:00:00', 10000000, 'USD', 'b'),\
    ('mblox', 'mBlox', 0, 'mobile', 'Sunnyvale', 'CA', '2008-01-28 00:00:00', 22000000, 'USD', 'e'),\
    ('esolar', 'eSolar', 10, 'hardware', 'Pasadena', 'CA', '2008-01-17 00:00:00', 10000000, 'USD', 'a'),\
    ('esolar', 'eSolar', 10, 'hardware', 'Pasadena', 'CA', '2008-04-21 00:00:00', 130000000, 'USD', 'b'),\
    ('marin-software', 'Marin Software', 0, 'software', 'San Francisco', 'CA', '2006-10-05 00:00:00', 2000000, 'USD', 'a'),\
    ('marin-software', 'Marin Software', 0, 'software', 'San Francisco', 'CA', '2008-04-09 00:00:00', 7250000, 'USD', 'b'),\
    ('sibeam', 'SiBEAM', 0, 'hardware', 'Sunnyvale', 'CA', '2004-12-01 00:00:00', 1250000, 'USD', 'seed'),\
    ('sibeam', 'SiBEAM', 0, 'hardware', 'Sunnyvale', 'CA', '2005-05-01 00:00:00', 15000000, 'USD', 'a'),\
    ('sibeam', 'SiBEAM', 0, 'hardware', 'Sunnyvale', 'CA', '2006-08-01 00:00:00', 21000000, 'USD', 'b'),\
    ('sibeam', 'SiBEAM', 0, 'hardware', 'Sunnyvale', 'CA', '2008-04-07 00:00:00', 40000000, 'USD', 'c'),\
    ('lumosity', 'Lumos Labs', 0, 'web', 'San Francisco', 'CA', '2007-06-11 00:00:00', 400000, 'USD', 'angel'),\
    ('lumosity', 'Lumos Labs', 0, 'web', 'San Francisco', 'CA', '2008-06-03 00:00:00', 3000000, 'USD', 'a'),\
    ('irise', 'iRise', 150, 'software', 'El Segundo', 'CA', '2008-03-01 00:00:00', 20000000, 'USD', 'unattributed'),\
    ('richrelevance', 'richrelevance', 0, 'web', 'San Francisco', 'CA', '2008-04-08 00:00:00', 4200000, 'USD', 'b'),\
    ('labmeeting', 'Labmeeting', 0, 'web', 'unknown', 'CA', '2008-05-01 00:00:00', 500000, 'USD', 'seed'),\
    ('shopflick', 'Shopflick', 10, 'web', 'Los Angeles', 'CA', '2008-03-15 00:00:00', 1000000, 'USD', 'angel'),\
    ('shopflick', 'Shopflick', 10, 'web', 'Los Angeles', 'CA', '2008-07-01 00:00:00', 7000000, 'USD', 'a'),\
    ('turnhere', 'TurnHere', 35, 'web', 'Emeryville', 'CA', '2008-02-15 00:00:00', 7500000, 'USD', 'a'),\
    ('turnhere', 'TurnHere', 35, 'web', 'Emeryville', 'CA', '2006-11-01 00:00:00', 1100000, 'USD', 'seed'),\
    ('coupa', 'Coupa', 0, 'web', 'Foster City', 'CA', '2007-03-13 00:00:00', 1500000, 'USD', 'a'),\
    ('coupa', 'Coupa', 0, 'web', 'Foster City', 'CA', '2008-04-09 00:00:00', 6000000, 'USD', 'b'),\
    ('squaretrade', 'SquareTrade', 0, 'hardware', 'San Francisco', 'CA', '1999-10-01 00:00:00', 400000, 'USD', 'seed'),\
    ('squaretrade', 'SquareTrade', 0, 'hardware', 'San Francisco', 'CA', '2008-04-10 00:00:00', 9000000, 'USD', 'b'),\
    ('v-enable', 'V-Enable', 0, 'mobile', 'San Diego', 'CA', '2006-02-28 00:00:00', 6000000, 'USD', 'c'),\
    ('v-enable', 'V-Enable', 0, 'mobile', 'San Diego', 'CA', '2003-09-21 00:00:00', 3750000, 'USD', 'b'),\
    ('aeria', 'Aeria', 0, 'web', 'Santa Clara', 'CA', '2007-10-01 00:00:00', 288000, 'USD', 'seed'),\
    ('heysan', 'Heysan', 6, 'mobile', 'San Francisco', 'CA', '2007-01-24 00:00:00', 20000, 'USD', 'seed'),\
    ('memeo', 'Memeo', 0, 'software', 'Aliso Viejo', 'CA', '2008-01-07 00:00:00', 8100000, 'USD', 'b'),\
    ('imageshack', 'imageshack', 0, 'web', 'Los Gatos', 'CA', '2007-05-01 00:00:00', 10000000, 'USD', 'a'),\
    ('pluggedin', 'PluggedIn', 0, 'unknown', 'Santa Monica', 'CA', '2008-04-15 00:00:00', 2000000, 'USD', 'unattributed'),\
    ('cellspin', 'CellSpin', 12, 'mobile', 'San Jose', 'CA', '2006-12-01 00:00:00', 1100000, 'USD', 'angel'),\
    ('remixation', 'Remixation', 5, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 1000000, 'USD', 'a'),\
    ('the-auteurs', 'The Auteurs', 9, 'web', 'Palo Alto', 'CA', '2007-08-01 00:00:00', 750000, 'USD', 'a'),\
    ('modern-feed', 'Modern Feed', 0, 'web', 'Los Angeles', 'CA', '2007-04-01 00:00:00', 3000000, 'USD', 'seed'),\
    ('grou-ps', 'GROU.PS', 0, 'web', 'San Francisco', 'CA', '2008-06-01 00:00:00', 1100000, 'USD', 'a'),\
    ('triggit', 'Triggit', 5, 'web', 'San Francisco', 'CA', '2007-03-01 00:00:00', 350000, 'USD', 'seed'),\
    ('triggit', 'Triggit', 5, 'web', 'San Francisco', 'CA', '2007-07-01 00:00:00', 500000, 'USD', 'angel'),\
    ('serious-business', 'Serious Business', 6, 'web', 'San Francisco', 'CA', '2008-04-25 00:00:00', 4000000, 'USD', 'unattributed'),\
    ('presdo', 'Presdo', 1, 'web', 'Mountain View', 'CA', '2007-12-01 00:00:00', 35000, 'USD', 'seed'),\
    ('nile-guide', 'Nile Guide', 0, 'web', 'San Francisco', 'CA', '2008-06-05 00:00:00', 8000000, 'USD', 'b'),\
    ('adify', 'Adify', 10, 'web', 'San Bruno', 'CA', '2006-08-01 00:00:00', 8000000, 'USD', 'a'),\
    ('adify', 'Adify', 10, 'web', 'San Bruno', 'CA', '2007-04-18 00:00:00', 19000000, 'USD', 'b'),\
    ('invensense', 'Invensense', 0, 'hardware', 'Sunnyvale', 'CA', '2008-04-28 00:00:00', 19000000, 'USD', 'c'),\
    ('centerd', 'Center''d', 0, 'web', 'Menlo Park', 'CA', '2007-02-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('centerd', 'Center''d', 0, 'web', 'Menlo Park', 'CA', '2007-10-01 00:00:00', 5500000, 'USD', 'a'),\
    ('centerd', 'Center''d', 0, 'web', 'Menlo Park', 'CA', '2007-05-01 00:00:00', 500000, 'USD', 'debt_round'),\
    ('new-relic', 'New Relic', 0, 'web', 'unknown', 'CA', '2008-04-01 00:00:00', 3500000, 'USD', 'a'),\
    ('gridstone-research', 'Gridstone Research', 0, 'software', 'San Mateo', 'CA', '2008-04-04 00:00:00', 10000000, 'USD', 'b'),\
    ('fusionops', 'FusionOps', 10, 'software', 'Sunnyvale', 'CA', '2005-03-01 00:00:00', 4000000, 'USD', 'a'),\
    ('marketlive', 'MarketLive', 0, 'software', 'Foster City', 'CA', '2008-05-05 00:00:00', 20000000, 'USD', 'e'),\
    ('snaplogic', 'SnapLogic', 0, 'software', 'San Mateo', 'CA', '2007-05-22 00:00:00', 2500000, 'USD', 'a'),\
    ('boardwalktech', 'Boardwalktech', 20, 'software', 'Palo Alto', 'CA', '2006-10-01 00:00:00', 500000, 'USD', 'angel'),\
    ('rosum', 'Rosum', 0, 'hardware', 'Mountain View', 'CA', '2008-04-15 00:00:00', 15000000, 'USD', 'b'),\
    ('citizenhawk', 'CitizenHawk', 0, 'unknown', 'Aliso Viejo', 'CA', '2008-05-12 00:00:00', 3000000, 'USD', 'unattributed'),\
    ('dilithium-networks', 'Dilithium Networks', 0, 'web', 'Petaluma', 'CA', '2003-04-23 00:00:00', 10000000, 'USD', 'b'),\
    ('dilithium-networks', 'Dilithium Networks', 0, 'web', 'Petaluma', 'CA', '2005-03-14 00:00:00', 18000000, 'USD', 'c'),\
    ('dilithium-networks', 'Dilithium Networks', 0, 'web', 'Petaluma', 'CA', '2007-07-03 00:00:00', 33000000, 'USD', 'd'),\
    ('passenger', 'Passenger', 0, 'unknown', 'Los Angeles', 'CA', '2007-11-05 00:00:00', 8300000, 'USD', 'b'),\
    ('passenger', 'Passenger', 0, 'unknown', 'Los Angeles', 'CA', '2008-03-14 00:00:00', 12200000, 'USD', 'c'),\
    ('experience-project', 'Experience Project', 0, 'web', 'San Francisco', 'CA', '2008-05-13 00:00:00', 3000000, 'USD', 'a'),\
    ('mozes', 'Mozes', 0, 'web', 'Palo Alto', 'CA', '2007-02-22 00:00:00', 5000000, 'USD', 'a'),\
    ('mozes', 'Mozes', 0, 'web', 'Palo Alto', 'CA', '2008-05-01 00:00:00', 11500000, 'USD', 'b'),\
    ('rvita', 'rVita', 6, 'software', 'Santa Clara', 'CA', '2007-10-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('mefeedia', 'Mefeedia', 6, 'web', 'Burbank', 'CA', '2008-03-19 00:00:00', 250000, 'USD', 'angel'),\
    ('wavemaker-software', 'Wavemaker Software', 0, 'software', 'San Francisco', 'CA', '2008-04-15 00:00:00', 4500000, 'USD', 'a'),\
    ('virtuallogix', 'VirtualLogix', 0, 'software', 'Sunnyvale', 'CA', '2007-07-11 00:00:00', 16000000, 'USD', 'b'),\
    ('fonemesh', 'Fonemesh', 7, 'software', 'San Francisco', 'CA', '2008-06-01 00:00:00', 100000, 'USD', 'angel'),\
    ('fonemesh', 'Fonemesh', 7, 'software', 'San Francisco', 'CA', '2008-05-01 00:00:00', 150000, 'USD', 'seed'),\
    ('cognition-technologies', 'Cognition Technologies', 20, 'software', 'Culver City', 'CA', '2008-07-15 00:00:00', 2700000, 'USD', 'unattributed'),\
    ('sometrics', 'Sometrics', 0, 'web', 'Los Angeles', 'CA', '2008-05-14 00:00:00', 1000000, 'USD', 'a'),\
    ('socialvibe', 'SocialVibe', 0, 'web', 'Los Angeles', 'CA', '2007-12-01 00:00:00', 4200000, 'USD', 'a'),\
    ('deviantart', 'deviantART', 0, 'web', 'Hollywood', 'CA', '2007-06-21 00:00:00', 3500000, 'USD', 'a'),\
    ('mythings', 'MyThings', 0, 'web', 'Menlo Park', 'CA', '2006-04-01 00:00:00', 8000000, 'USD', 'a'),\
    ('mywaves', 'mywaves', 35, 'mobile', 'Sunnyvale', 'CA', '2006-12-08 00:00:00', 6000000, 'USD', 'a'),\
    ('litescape', 'litescape', 0, 'web', 'Redwood Shores', 'CA', '2007-08-06 00:00:00', 14000000, 'USD', 'b'),\
    ('nextbio', 'nextbio', 0, 'unknown', 'Cupertino', 'CA', '2007-06-06 00:00:00', 7000000, 'USD', 'b'),\
    ('parascale', 'Parascale', 0, 'software', 'Cupertino', 'CA', '2008-06-23 00:00:00', 11370000, 'USD', 'a'),\
    ('row44', 'Row44', 0, 'hardware', 'Westlake Village', 'CA', '2008-05-20 00:00:00', 21000000, 'USD', 'a'),\
    ('jellycloud', 'JellyCloud', 0, 'web', 'Redwood City', 'CA', '2008-05-16 00:00:00', 6600000, 'USD', 'a'),\
    ('aster-data-systems', 'Aster Data Systems', 50, 'software', 'Redwood City', 'CA', '2005-11-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('aster-data-systems', 'Aster Data Systems', 50, 'software', 'Redwood City', 'CA', '2007-05-01 00:00:00', 5000000, 'USD', 'a'),\
    ('pixim', 'Pixim', 0, 'hardware', 'Mountain View', 'CA', '2007-03-07 00:00:00', 15000000, 'USD', 'b'),\
    ('pixim', 'Pixim', 0, 'hardware', 'Mountain View', 'CA', '2005-06-09 00:00:00', 12000000, 'USD', 'a'),\
    ('pixim', 'Pixim', 0, 'hardware', 'Mountain View', 'CA', '2007-06-12 00:00:00', 5100000, 'USD', 'b'),\
    ('funambol', 'Funambol', 0, 'mobile', 'Redwood City', 'CA', '2005-08-08 00:00:00', 5000000, 'USD', 'a'),\
    ('funambol', 'Funambol', 0, 'mobile', 'Redwood City', 'CA', '2006-12-01 00:00:00', 5500000, 'USD', 'a'),\
    ('funambol', 'Funambol', 0, 'mobile', 'Redwood City', 'CA', '2008-06-17 00:00:00', 12500000, 'USD', 'b'),\
    ('funambol', 'Funambol', 0, 'mobile', 'Redwood City', 'CA', '2008-06-01 00:00:00', 2000000, 'USD', 'debt_round'),\
    ('moblyng', 'Moblyng', 0, 'web', 'Menlo Park', 'CA', '2008-05-21 00:00:00', 5700000, 'USD', 'b'),\
    ('moblyng', 'Moblyng', 0, 'web', 'Menlo Park', 'CA', '2007-03-28 00:00:00', 1530000, 'USD', 'a'),\
    ('zecter', 'Zecter', 3, 'web', 'Mountain View', 'CA', '2007-06-01 00:00:00', 15000, 'USD', 'seed'),\
    ('votigo', 'Votigo', 0, 'web', 'Emeryville', 'CA', '2007-12-31 00:00:00', 1265000, 'USD', 'a'),\
    ('zinio', 'Zinio', 0, 'web', 'San Francisco', 'CA', '2005-09-12 00:00:00', 7000000, 'USD', 'a'),\
    ('dreamfactory', 'Dreamfactory', 0, 'web', 'Mountain View', 'CA', '2006-05-08 00:00:00', 5800000, 'USD', 'a'),\
    ('greennote', 'GreenNote', 15, 'web', 'Redwood City', 'CA', '2007-10-01 00:00:00', 4200000, 'USD', 'a'),\
    ('skyfire', 'Skyfire', 0, 'software', 'Mountain View', 'CA', '2007-06-01 00:00:00', 4800000, 'USD', 'a'),\
    ('skyfire', 'Skyfire', 0, 'software', 'Mountain View', 'CA', '2008-05-28 00:00:00', 13000000, 'USD', 'b'),\
    ('edufire', 'eduFire', 0, 'web', 'Santa Monica', 'CA', '2008-04-09 00:00:00', 400000, 'USD', 'angel'),\
    ('b-hive-networks', 'B-hive Networks', 0, 'software', 'San Mateo', 'CA', '2006-08-25 00:00:00', 7000000, 'USD', 'a'),\
    ('vmware', 'VMware', 5000, 'unknown', 'Palo Alto', 'CA', '2007-07-30 00:00:00', 150000000, 'USD', 'unattributed'),\
    ('katalyst-media', 'Katalyst Media', 11, 'unknown', 'Los Angeles', 'CA', '2008-01-28 00:00:00', 10000000, 'USD', 'a'),\
    ('neurovigil', 'NeuroVigil', 5, 'biotech', 'La Jolla', 'CA', '2008-05-30 00:00:00', 250000, 'USD', 'seed'),\
    ('sylantro', 'Sylantro', 0, 'web', 'Campbell', 'CA', '2006-04-01 00:00:00', 11000000, 'USD', 'e'),\
    ('sylantro', 'Sylantro', 0, 'web', 'Campbell', 'CA', '2003-11-01 00:00:00', 4500000, 'USD', 'd'),\
    ('sylantro', 'Sylantro', 0, 'web', 'Campbell', 'CA', '2000-10-01 00:00:00', 55000000, 'USD', 'c'),\
    ('teleflip', 'TeleFlip', 0, 'mobile', 'Santa Monica', 'CA', '2008-01-01 00:00:00', 4900000, 'USD', 'b'),\
    ('vidshadow', 'Vidshadow', 16, 'web', 'Placentia', 'CA', '2007-02-15 00:00:00', 2000000, 'USD', 'a'),\
    ('jigsaw', 'Jigsaw', 0, 'web', 'San Mateo', 'CA', '2003-12-01 00:00:00', 750000, 'USD', 'a'),\
    ('jigsaw', 'Jigsaw', 0, 'web', 'San Mateo', 'CA', '2004-09-01 00:00:00', 5200000, 'USD', 'b'),\
    ('jigsaw', 'Jigsaw', 0, 'web', 'San Mateo', 'CA', '2006-03-01 00:00:00', 12000000, 'USD', 'c'),\
    ('ozmo-devices', 'Ozmo Devices', 0, 'hardware', 'Palo Alto', 'CA', '2006-03-30 00:00:00', 12550000, 'USD', 'a'),\
    ('cooliris', 'Cooliris', 0, 'web', 'Menlo Park', 'CA', '2007-07-01 00:00:00', 3000000, 'USD', 'a'),\
    ('gamook', 'Gamook', 0, 'web', 'Menlo Park', 'CA', '2008-03-16 00:00:00', 1500000, 'USD', 'a'),\
    ('vindicia', 'Vindicia', 0, 'web', 'Redwood City', 'CA', '2008-03-31 00:00:00', 5600000, 'USD', 'c'),\
    ('dizzywood', 'Dizzywood', 0, 'web', 'unknown', 'CA', '2008-02-07 00:00:00', 1000000, 'USD', 'a'),\
    ('limbo', 'Limbo', 37, 'web', 'Burlingame', 'CA', '2007-04-25 00:00:00', 8000000, 'USD', 'b'),\
    ('startforce', 'StartForce', 21, 'web', 'San Francisco', 'CA', '2007-09-01 00:00:00', 1000000, 'USD', 'a'),\
    ('coolearth', 'coolearth', 9, 'cleantech', 'Livermore', 'CA', '2008-02-20 00:00:00', 21000000, 'USD', 'a'),\
    ('coolearth', 'coolearth', 9, 'cleantech', 'Livermore', 'CA', '2007-06-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('yield-software', 'Yield Software', 0, 'web', 'San Mateo', 'CA', '2008-06-04 00:00:00', 6000000, 'USD', 'b'),\
    ('tapulous', 'Tapulous', 9, 'web', 'Palo Alto', 'CA', '2008-07-01 00:00:00', 1800000, 'USD', 'angel'),\
    ('codefast', 'Codefast', 0, 'software', 'San Jose', 'CA', '2005-03-15 00:00:00', 6500000, 'USD', 'a'),\
    ('hyperic', 'Hyperic', 0, 'web', 'San Francisco', 'CA', '2006-05-10 00:00:00', 3800000, 'USD', 'a'),\
    ('ipling-2', 'iPling :))', 7, 'mobile', 'San Francisco', 'CA', '2008-03-01 00:00:00', 300000, 'USD', 'angel'),\
    ('intacct', 'Intacct', 100, 'software', 'San Jose', 'CA', '2008-04-29 00:00:00', 15000000, 'USD', 'unattributed'),\
    ('intacct', 'Intacct', 100, 'software', 'San Jose', 'CA', '2007-06-27 00:00:00', 14000000, 'USD', 'unattributed'),\
    ('vivaty', 'Vivaty', 25, 'web', 'Menlo Park', 'CA', '2007-08-01 00:00:00', 9400000, 'USD', 'a'),\
    ('aurora-biofuels', 'Aurora Biofuels', 0, 'cleantech', 'Alameda', 'CA', '2008-06-10 00:00:00', 20000000, 'USD', 'b'),\
    ('eeye', 'eEye', 0, 'unknown', 'Irvine', 'CA', '2004-06-01 00:00:00', 15000000, 'USD', 'd'),\
    ('eeye', 'eEye', 0, 'unknown', 'Irvine', 'CA', '2002-12-01 00:00:00', 9000000, 'USD', 'c'),\
    ('allbusiness-com', 'AllBusiness.com', 50, 'other', 'San Francisco', 'CA', '2004-07-01 00:00:00', 10000000, 'USD', 'b'),\
    ('allbusiness-com', 'AllBusiness.com', 50, 'other', 'San Francisco', 'CA', '2006-02-01 00:00:00', 12400000, 'USD', 'c'),\
    ('insideview', 'InsideView', 40, 'software', 'San Francisco', 'CA', '2007-06-01 00:00:00', 7400000, 'USD', 'a'),\
    ('clickability', 'Clickability', 0, 'software', 'San Francisco', 'CA', '2008-07-14 00:00:00', 3500000, 'USD', 'debt_round'),\
    ('keibi-technologies', 'Keibi Technologies', 30, 'software', 'San Francisco', 'CA', '2007-05-01 00:00:00', 5000000, 'USD', 'b'),\
    ('robodynamics', 'RoboDynamics', 5, 'hardware', 'Santa Monica', 'CA', '2003-09-01 00:00:00', 100000, 'USD', 'seed'),\
    ('robodynamics', 'RoboDynamics', 5, 'hardware', 'Santa Monica', 'CA', '2006-05-01 00:00:00', 500000, 'USD', 'angel'),\
    ('imagespan', 'ImageSpan', 0, 'web', 'Sausalito', 'CA', '2008-06-23 00:00:00', 11000000, 'USD', 'b'),\
    ('solarflare', 'Solarflare', 0, 'hardware', 'Irvine', 'CA', '2008-06-18 00:00:00', 26000000, 'USD', 'unattributed'),\
    ('clupedia', 'Clupedia', 0, 'web', 'Santa Ana', 'CA', '2007-05-22 00:00:00', 1300000, 'USD', 'a'),\
    ('zigabid', 'Zigabid', 10, 'other', 'La Canada', 'CA', '2006-12-01 00:00:00', 500000, 'USD', 'seed'),\
    ('zigabid', 'Zigabid', 10, 'other', 'La Canada', 'CA', '2008-02-01 00:00:00', 500000, 'USD', 'angel'),\
    ('inthrma', 'InThrMa', 3, 'software', 'Oakland', 'CA', '2007-10-01 00:00:00', 10000, 'USD', 'seed'),\
    ('webaroo', 'Webaroo', 0, 'unknown', 'Santa Clara', 'CA', '2006-12-22 00:00:00', 10000000, 'USD', 'b'),\
    ('webaroo', 'Webaroo', 0, 'unknown', 'Santa Clara', 'CA', '2008-07-05 00:00:00', 10000000, 'USD', 'c'),\
    ('fluid-entertainment', 'Fluid Entertainment', 0, 'other', 'Mill Valley', 'CA', '2008-03-12 00:00:00', 3200000, 'USD', 'a'),\
    ('unisfair', 'Unisfair', 0, 'other', 'Menlo Park', 'CA', '2008-01-21 00:00:00', 10000000, 'USD', 'b'),\
    ('on24', 'ON24', 0, 'web', 'San Francisco', 'CA', '2008-07-11 00:00:00', 8000000, 'USD', 'unattributed'),\
    ('trueanthem', 'trueAnthem', 10, 'other', 'Newport Beach', 'CA', '2008-07-28 00:00:00', 2000000, 'USD', 'angel'),\
    ('colizer', 'Colizer', 0, 'web', 'San Diego', 'CA', '2004-04-01 00:00:00', 120000, 'USD', 'a'),\
    ('coremetrics', 'Coremetrics', 0, 'software', 'San Mateo', 'CA', '2006-03-09 00:00:00', 31000000, 'USD', 'd'),\
    ('coremetrics', 'Coremetrics', 0, 'software', 'San Mateo', 'CA', '2008-04-04 00:00:00', 60000000, 'USD', 'e'),\
    ('minted', 'Minted', 0, 'unknown', 'San Francisco', 'CA', '2008-07-18 00:00:00', 2500000, 'USD', 'seed'),\
    ('appirio', 'Appirio', 0, 'software', 'San Mateo', 'CA', '2008-03-13 00:00:00', 1100000, 'USD', 'a'),\
    ('appirio', 'Appirio', 0, 'software', 'San Mateo', 'CA', '2008-07-21 00:00:00', 10000000, 'USD', 'b'),\
    ('samfind', 'samfind', 3, 'web', 'Los Angeles', 'CA', '2006-01-01 00:00:00', 20000, 'USD', 'seed'),\
    ('eye-fi', 'Eye-Fi', 0, 'hardware', 'Mountain View', 'CA', '2007-06-11 00:00:00', 5500000, 'USD', 'a'),\
    ('responselogix', 'ResponseLogix', 0, 'web', 'Sunnyvale', 'CA', '2008-01-23 00:00:00', 8000000, 'USD', 'a'),\
    ('qumu', 'Qumu', 0, 'software', 'Emeryville', 'CA', '2008-07-21 00:00:00', 10700000, 'USD', 'c'),\
    ('carbonflow', 'CarbonFlow', 0, 'cleantech', 'San Francsico', 'CA', '2008-07-23 00:00:00', 2900000, 'USD', 'a'),\
    ('allvoices', 'Allvoices', 8, 'web', 'San Francisco', 'CA', '2007-07-31 00:00:00', 4500000, 'USD', 'a'),\
    ('moxsie', 'Moxsie', 0, 'web', 'Palo Alto', 'CA', '2008-06-27 00:00:00', 1000000, 'USD', 'a'),\
    ('service-now-com', 'Service-now.com', 80, 'web', 'Solana Beach', 'CA', '2005-07-05 00:00:00', 2500000, 'USD', 'a'),\
    ('service-now-com', 'Service-now.com', 80, 'web', 'Solana Beach', 'CA', '2006-12-05 00:00:00', 5000000, 'USD', 'b'),\
    ('anvato', 'Anvato', 0, 'web', 'Mountain View', 'CA', '2008-07-29 00:00:00', 550000, 'USD', 'seed'),\
    ('vantage-media', 'Vantage Media', 150, 'web', 'El Segundo', 'CA', '2007-02-28 00:00:00', 70000000, 'USD', 'a'),\
    ('entone-technologies', 'Entone Technologies', 80, 'hardware', 'San Mateo', 'CA', '2008-08-01 00:00:00', 14500000, 'USD', 'b'),\
    ('entone-technologies', 'Entone Technologies', 80, 'hardware', 'San Mateo', 'CA', '2003-05-02 00:00:00', 9500000, 'USD', 'a'),\
    ('750-industries', '750 Industries', 7, 'web', 'San Francisco', 'CA', '2008-08-01 00:00:00', 1000000, 'USD', 'a'),\
    ('plastic-logic', 'Plastic Logic', 0, 'hardware', 'Mountain View', 'CA', '2002-04-20 00:00:00', 13700000, 'USD', 'a'),\
    ('plastic-logic', 'Plastic Logic', 0, 'hardware', 'Mountain View', 'CA', '2005-01-05 00:00:00', 8000000, 'USD', 'b'),\
    ('plastic-logic', 'Plastic Logic', 0, 'hardware', 'Mountain View', 'CA', '2005-11-30 00:00:00', 24000000, 'USD', 'c'),\
    ('plastic-logic', 'Plastic Logic', 0, 'hardware', 'Mountain View', 'CA', '2007-01-06 00:00:00', 100000000, 'USD', 'd'),\
    ('plastic-logic', 'Plastic Logic', 0, 'hardware', 'Mountain View', 'CA', '2008-08-04 00:00:00', 50000000, 'USD', 'e'),\
    ('paymo', 'Paymo', 0, 'mobile', 'San Francisco', 'CA', '2008-08-01 00:00:00', 5000000, 'USD', 'seed'),\
    ('brilliant-telecom', 'Brilliant Telecom', 0, 'unknown', 'Campbell', 'CA', '2008-08-04 00:00:00', 11000000, 'USD', 'a'),\
    ('skygrid', 'SkyGrid', 0, 'web', 'Sunnyvale', 'CA', '2008-08-06 00:00:00', 11000000, 'USD', 'b'),\
    ('intensedebate', 'Intense Debate', 4, 'web', 'Boulder', 'CO', '2007-05-01 00:00:00', 15000, 'USD', 'seed'),\
    ('associatedcontent', 'Associated Content', 0, 'web', 'Denver', 'CO', '2006-01-01 00:00:00', 5400000, 'USD', 'a'),\
    ('associatedcontent', 'Associated Content', 0, 'web', 'Denver', 'CO', '2007-07-01 00:00:00', 10000000, 'USD', 'b'),\
    ('madkast', 'madKast', 0, 'web', 'Boulder', 'CO', '2007-03-01 00:00:00', 15000, 'USD', 'seed'),\
    ('madkast', 'madKast', 0, 'web', 'Boulder', 'CO', '2007-10-01 00:00:00', 300000, 'USD', 'a'),\
    ('eventvue', 'EventVue', 0, 'web', 'Boulder', 'CO', '2007-08-01 00:00:00', 15000, 'USD', 'seed'),\
    ('eventvue', 'EventVue', 0, 'web', 'Boulder', 'CO', '2007-09-01 00:00:00', 250000, 'USD', 'a'),\
    ('socialthing', 'Socialthing!', 6, 'web', 'Boulder', 'CO', '2007-05-18 00:00:00', 15000, 'USD', 'seed'),\
    ('socialthing', 'Socialthing!', 6, 'web', 'Boulder', 'CO', '2007-10-01 00:00:00', 500000, 'USD', 'debt_round'),\
    ('jsquaredmedia', 'J Squared Media', 0, 'web', 'Boulder', 'CO', '2007-08-01 00:00:00', 15000, 'USD', 'seed'),\
    ('searchtophone', 'Search to Phone', 0, 'web', 'Boulder', 'CO', '2007-08-01 00:00:00', 15000, 'USD', 'seed'),\
    ('filtrbox', 'Filtrbox', 0, 'web', 'Boulder', 'CO', '2007-08-01 00:00:00', 15000, 'USD', 'seed'),\
    ('filtrbox', 'Filtrbox', 0, 'web', 'Boulder', 'CO', '2008-02-28 00:00:00', 500000, 'USD', 'seed'),\
    ('brightkite', 'Brightkite', 0, 'web', 'Denver', 'CO', '2007-08-01 00:00:00', 15000, 'USD', 'seed'),\
    ('brightkite', 'Brightkite', 0, 'web', 'Denver', 'CO', '2008-03-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('newsgator', 'NewsGator', 0, 'web', 'Denver', 'CO', '2004-06-23 00:00:00', 1400000, 'USD', 'a'),\
    ('newsgator', 'NewsGator', 0, 'web', 'Denver', 'CO', '2007-12-01 00:00:00', 12000000, 'USD', 'c'),\
    ('newsgator', 'NewsGator', 0, 'web', 'Denver', 'CO', '2004-12-01 00:00:00', 8600000, 'USD', 'b'),\
    ('me-dium', 'Me.dium', 0, 'web', 'Boulder', 'CO', '2007-06-01 00:00:00', 15000000, 'USD', 'b'),\
    ('buzzwire', 'Buzzwire', 0, 'web', 'Denver', 'CO', '2007-12-01 00:00:00', 8000000, 'USD', 'b'),\
    ('hivelive', 'HiveLive', 0, 'web', 'Boulder', 'CO', '2008-02-01 00:00:00', 2200000, 'USD', 'angel'),\
    ('hivelive', 'HiveLive', 0, 'web', 'Boulder', 'CO', '2008-02-27 00:00:00', 5600000, 'USD', 'a'),\
    ('xaware', 'XAware', 0, 'web', 'Colorado Springs', 'CO', '2008-01-01 00:00:00', 7400000, 'USD', 'b'),\
    ('xaware', 'XAware', 0, 'web', 'Colorado Springs', 'CO', '2002-11-01 00:00:00', 2100000, 'USD', 'a'),\
    ('lijit', 'Lijit', 0, 'web', 'Boulder', 'CO', '2007-01-01 00:00:00', 900000, 'USD', 'a'),\
    ('lijit', 'Lijit', 0, 'web', 'Boulder', 'CO', '2007-06-01 00:00:00', 3300000, 'USD', 'b'),\
    ('hubbuzz', 'hubbuzz', 8, 'web', 'Centennial', 'CO', '2007-03-01 00:00:00', 750000, 'USD', 'angel'),\
    ('gnip', 'Gnip', 6, 'web', 'Boulder', 'CO', '2008-03-01 00:00:00', 1100000, 'USD', 'a'),\
    ('collective-intellect', 'Collective Intellect', 0, 'unknown', 'Boulder', 'CO', '2006-02-01 00:00:00', 2600000, 'USD', 'a'),\
    ('collective-intellect', 'Collective Intellect', 0, 'unknown', 'Boulder', 'CO', '2008-04-15 00:00:00', 6600000, 'USD', 'b'),\
    ('rally-software', 'Rally Software', 0, 'software', 'Boulder', 'CO', '2008-06-04 00:00:00', 16850000, 'USD', 'c'),\
    ('rally-software', 'Rally Software', 0, 'software', 'Boulder', 'CO', '2006-06-14 00:00:00', 8000000, 'USD', 'b'),\
    ('symplified', 'Symplified', 0, 'software', 'Boulder', 'CO', '2008-06-09 00:00:00', 6000000, 'USD', 'a'),\
    ('indeed', 'Indeed', 0, 'web', 'Stamford', 'CT', '2005-08-01 00:00:00', 5000000, 'USD', 'a'),\
    ('geezeo', 'Geezeo', 0, 'web', 'Hartford', 'CT', '2008-04-24 00:00:00', 1200000, 'USD', 'unattributed'),\
    ('entrecard', 'Entrecard', 1, 'web', 'Hamden', 'CT', '2007-01-01 00:00:00', 30000, 'USD', 'seed'),\
    ('entrecard', 'Entrecard', 1, 'web', 'Hamden', 'CT', '2007-11-01 00:00:00', 40000, 'USD', 'seed'),\
    ('americantowns-com', 'AmericanTowns.com', 10, 'web', 'Fairfield', 'CT', '2006-07-01 00:00:00', 1100000, 'USD', 'a'),\
    ('americantowns-com', 'AmericanTowns.com', 10, 'web', 'Fairfield', 'CT', '2007-09-01 00:00:00', 3300000, 'USD', 'b'),\
    ('kayak', 'Kayak', 58, 'web', 'Norwalk', 'CT', '2006-05-01 00:00:00', 11500000, 'USD', 'c'),\
    ('kayak', 'Kayak', 58, 'web', 'Norwalk', 'CT', '2004-12-01 00:00:00', 7000000, 'USD', 'b'),\
    ('kayak', 'Kayak', 58, 'web', 'Norwalk', 'CT', '2007-12-01 00:00:00', 196000000, 'USD', 'd'),\
    ('gocrosscampus', 'GoCrossCampus', 25, 'web', 'New Haven', 'CT', '2007-09-01 00:00:00', 375000, 'USD', 'seed'),\
    ('health-plan-one', 'Health Plan One', 0, 'web', 'Shelton', 'CT', '2008-04-21 00:00:00', 6500000, 'USD', 'a'),\
    ('your-survival', 'Your Survival', 10, 'web', 'Westport', 'CT', '2007-11-01 00:00:00', 350000, 'USD', 'angel'),\
    ('media-lantern', 'Media Lantern', 9, 'web', 'New London', 'CT', '2008-03-30 00:00:00', 250000, 'USD', 'seed'),\
    ('gridpoint', 'GridPoint', 0, 'cleantech', 'Washington', 'DC', '2007-09-01 00:00:00', 32000000, 'USD', 'd'),\
    ('gridpoint', 'GridPoint', 0, 'cleantech', 'Washington', 'DC', '2006-05-01 00:00:00', 16000000, 'USD', 'b'),\
    ('gridpoint', 'GridPoint', 0, 'cleantech', 'Washington', 'DC', '2006-09-01 00:00:00', 21000000, 'USD', 'c'),\
    ('gridpoint', 'GridPoint', 0, 'cleantech', 'Washington', 'DC', '2008-03-28 00:00:00', 15000000, 'USD', 'd'),\
    ('hotpads-com', 'HotPads', 11, 'web', 'Washington', 'DC', '2007-03-01 00:00:00', 2300000, 'USD', 'a'),\
    ('launchbox', 'LaunchBox', 0, 'web', 'Washinton', 'DC', '2008-02-01 00:00:00', 250000, 'USD', 'a'),\
    ('cogent', 'Cogent', 0, 'web', 'Washington', 'DC', '2008-02-12 00:00:00', 510000, 'USD', 'seed'),\
    ('swapdrive', 'SwapDrive', 0, 'web', 'Washington', 'DC', '2000-05-01 00:00:00', 2650000, 'USD', 'a'),\
    ('swapdrive', 'SwapDrive', 0, 'web', 'Washington', 'DC', '2001-05-01 00:00:00', 2000000, 'USD', 'a'),\
    ('searchles', 'Searchles', 0, 'web', 'Washington', 'DC', '2008-07-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('payperpost', 'PayPerPost', 0, 'web', 'Orlando', 'FL', '2007-06-01 00:00:00', 7000000, 'USD', 'b'),\
    ('payperpost', 'PayPerPost', 0, 'web', 'Orlando', 'FL', '2006-10-01 00:00:00', 3000000, 'USD', 'a'),\
    ('affinityinternet', 'Affinity Internet', 0, 'web', 'Fort Lauderdale', 'FL', '1999-10-01 00:00:00', 60000000, 'USD', 'a'),\
    ('affinityinternet', 'Affinity Internet', 0, 'web', 'Fort Lauderdale', 'FL', '2002-01-01 00:00:00', 25000000, 'USD', 'a'),\
    ('multiply', 'Multiply', 0, 'web', 'Boca Raton', 'FL', '2005-11-01 00:00:00', 2000000, 'USD', 'a'),\
    ('multiply', 'Multiply', 0, 'web', 'Boca Raton', 'FL', '2006-07-01 00:00:00', 3000000, 'USD', 'a'),\
    ('multiply', 'Multiply', 0, 'web', 'Boca Raton', 'FL', '2007-08-01 00:00:00', 16600000, 'USD', 'b'),\
    ('revolutionmoney', 'Revolution Money', 0, 'web', 'Largo', 'FL', '2007-09-01 00:00:00', 50000000, 'USD', 'b'),\
    ('batanga', 'Batanga', 0, 'web', 'Miami', 'FL', '2007-08-01 00:00:00', 30000000, 'USD', 'c'),\
    ('batanga', 'Batanga', 0, 'web', 'Miami', 'FL', '2006-04-01 00:00:00', 5000000, 'USD', 'b'),\
    ('divorce360', 'divorce360', 6, 'web', 'North Palm Beach', 'FL', '2007-09-01 00:00:00', 2500000, 'USD', 'a'),\
    ('infinitybox', 'Infinity Box', 3, 'web', 'Tampa', 'FL', '2006-01-01 00:00:00', 18000, 'USD', 'seed'),\
    ('infinitybox', 'Infinity Box', 3, 'web', 'Tampa', 'FL', '2006-04-01 00:00:00', 100000, 'USD', 'angel'),\
    ('moli', 'MOLI', 0, 'web', 'West Palm Beach', 'FL', '2008-01-01 00:00:00', 29600000, 'USD', 'b'),\
    ('global-roaming', 'Global Roaming', 29, 'mobile', 'Miami', 'FL', '2007-02-01 00:00:00', 7500000, 'USD', 'a'),\
    ('global-roaming', 'Global Roaming', 29, 'mobile', 'Miami', 'FL', '2008-02-01 00:00:00', 23000000, 'USD', 'b'),\
    ('slingpage', 'Slingpage', 8, 'software', 'Estero', 'FL', '2007-12-01 00:00:00', 2250000, 'USD', 'angel'),\
    ('wrapmail', 'WrapMail', 10, 'web', 'Fort Lauderdale', 'FL', '2007-01-06 00:00:00', 1100000, 'USD', 'seed'),\
    ('ejamming', 'eJamming', 0, 'web', 'Boca Raton', 'FL', '2008-06-02 00:00:00', 150000, 'USD', 'unattributed'),\
    ('lehigh-technologies', 'Lehigh Technologies', 0, 'cleantech', 'Naples', 'FL', '2008-06-23 00:00:00', 34500000, 'USD', 'unattributed'),\
    ('tournease', 'TournEase', 10, 'web', 'Lakeland', 'FL', '2006-04-01 00:00:00', 3000000, 'USD', 'angel'),\
    ('tournease', 'TournEase', 10, 'web', 'Lakeland', 'FL', '2008-03-01 00:00:00', 500000, 'USD', 'angel'),\
    ('vitrue', 'Vitrue', 0, 'web', 'Atlanta', 'GA', '2007-10-01 00:00:00', 10000000, 'USD', 'b'),\
    ('vitrue', 'Vitrue', 0, 'web', 'Atlanta', 'GA', '2006-05-01 00:00:00', 2200000, 'USD', 'seed'),\
    ('screamingsports', 'Screaming Sports', 0, 'web', 'Atlanta', 'GA', '2007-07-01 00:00:00', 1250000, 'USD', 'a'),\
    ('berecruited', 'beRecruited', 0, 'web', 'Atlanta', 'GA', '2007-11-01 00:00:00', 1200000, 'USD', 'a'),\
    ('mfg', 'MFG', 0, 'web', 'Atlanta', 'GA', '2005-09-01 00:00:00', 14000000, 'USD', 'a'),\
    ('mfg', 'MFG', 0, 'web', 'Atlanta', 'GA', '2007-01-01 00:00:00', 4000000, 'USD', 'c'),\
    ('mfg', 'MFG', 0, 'web', 'Atlanta', 'GA', '2008-01-01 00:00:00', 26000000, 'USD', 'd'),\
    ('scintella-solutions', 'Scintella Solutions', 2, 'consulting', 'Atlanta', 'GA', '2008-04-30 00:00:00', 10000, 'USD', 'seed'),\
    ('scribestorm', 'ScribeStorm', 12, 'web', 'Fairfield', 'IA', '2006-04-24 00:00:00', 225000, 'USD', 'angel'),\
    ('balihoo', 'Balihoo', 0, 'mobile', 'Boise', 'ID', '2007-07-01 00:00:00', 1500000, 'USD', 'a'),\
    ('balihoo', 'Balihoo', 0, 'mobile', 'Boise', 'ID', '2008-01-01 00:00:00', 4000000, 'USD', 'b'),\
    ('info', 'Info', 0, 'web', 'Chicago', 'IL', '2004-11-01 00:00:00', 8400000, 'USD', 'a'),\
    ('feedburner', 'FeedBurner', 0, 'web', 'Chicago', 'IL', '2004-06-01 00:00:00', 1000000, 'USD', 'a'),\
    ('feedburner', 'FeedBurner', 0, 'web', 'Chicago', 'IL', '2005-04-01 00:00:00', 7000000, 'USD', 'b'),\
    ('viewpoints', 'Viewpoints', 0, 'web', 'Chicago', 'IL', '2007-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('grubhub', 'GrubHub', 0, 'web', 'Chicago', 'IL', '2007-11-01 00:00:00', 1100000, 'USD', 'a'),\
    ('ticketsnow', 'TicketsNow', 0, 'web', 'Rolling Meadows', 'IL', '2007-01-01 00:00:00', 34000000, 'USD', 'a'),\
    ('the-point', 'The Point', 0, 'web', 'Chicago', 'IL', '2007-11-01 00:00:00', 2500000, 'USD', 'angel'),\
    ('the-point', 'The Point', 0, 'web', 'Chicago', 'IL', '2008-02-01 00:00:00', 4800000, 'USD', 'a'),\
    ('inkling', 'Inkling', 2, 'web', 'Chicago', 'IL', '2007-01-01 00:00:00', 20000, 'USD', 'seed'),\
    ('crowdspring', 'crowdSPRING', 6, 'web', 'Chicago', 'IL', '2008-05-23 00:00:00', 3000000, 'USD', 'angel'),\
    ('ifbyphone', 'Ifbyphone', 16, 'web', 'Skokie', 'IL', '2007-07-01 00:00:00', 1500000, 'USD', 'a'),\
    ('fave-media', 'Fave Media', 18, 'web', 'Chicago', 'IL', '2008-01-17 00:00:00', 1600000, 'USD', 'a'),\
    ('accertify', 'Accertify', 0, 'web', 'Schaumburg', 'IL', '2008-06-03 00:00:00', 4000000, 'USD', 'a'),\
    ('firefly-energy', 'Firefly Energy', 0, 'cleantech', 'Peoria', 'IL', '2004-10-14 00:00:00', 4000000, 'USD', 'a'),\
    ('firefly-energy', 'Firefly Energy', 0, 'cleantech', 'Peoria', 'IL', '2008-06-10 00:00:00', 15000000, 'USD', 'c'),\
    ('firefly-energy', 'Firefly Energy', 0, 'cleantech', 'Peoria', 'IL', '2006-11-13 00:00:00', 10000000, 'USD', 'b'),\
    ('savo', 'SAVO', 68, 'software', 'Chicago', 'IL', '2005-09-01 00:00:00', 10000000, 'USD', 'a'),\
    ('chacha', 'ChaCha', 75, 'web', 'Carmel', 'IN', '2007-01-01 00:00:00', 6000000, 'USD', 'a'),\
    ('chacha', 'ChaCha', 75, 'web', 'Carmel', 'IN', '2007-11-01 00:00:00', 10000000, 'USD', 'b'),\
    ('betterworld', 'Better World Books', 0, 'web', 'Mishawaka', 'IN', '2008-04-07 00:00:00', 4500000, 'USD', 'a'),\
    ('compendium-blogware', 'Compendium Blogware', 25, 'software', 'Indianapolis', 'IN', '2007-05-01 00:00:00', 1100000, 'USD', 'angel'),\
    ('compendium-blogware', 'Compendium Blogware', 25, 'software', 'Indianapolis', 'IN', '2008-05-01 00:00:00', 1600000, 'USD', 'angel'),\
    ('instagarage', 'Instagarage', 4, 'web', 'New Orleans', 'LA', '2007-05-10 00:00:00', 25000, 'USD', 'seed'),\
    ('iskoot', 'iSkoot', 0, 'web', 'Cambridge', 'MA', '2007-02-27 00:00:00', 7000000, 'USD', 'b'),\
    ('iskoot', 'iSkoot', 0, 'web', 'Cambridge', 'MA', '2006-11-29 00:00:00', 6200000, 'USD', 'a'),\
    ('gotuitmedia', 'Gotuit Media', 0, 'web', 'Woburn', 'MA', '2004-12-01 00:00:00', 10000000, 'USD', 'b'),\
    ('gotuitmedia', 'Gotuit Media', 0, 'web', 'Woburn', 'MA', '2002-12-01 00:00:00', 6000000, 'USD', 'a'),\
    ('carbonite', 'Carbonite', 68, 'web', 'Boston', 'MA', '2006-03-01 00:00:00', 2500000, 'USD', 'a'),\
    ('carbonite', 'Carbonite', 68, 'web', 'Boston', 'MA', '2006-12-01 00:00:00', 3500000, 'USD', 'a'),\
    ('carbonite', 'Carbonite', 68, 'web', 'Boston', 'MA', '2007-05-01 00:00:00', 15000000, 'USD', 'b'),\
    ('carbonite', 'Carbonite', 68, 'web', 'Boston', 'MA', '2007-12-28 00:00:00', 5000000, 'USD', 'b'),\
    ('scanscout', 'ScanScout', 40, 'web', 'Boston', 'MA', '2007-03-01 00:00:00', 7000000, 'USD', 'a'),\
    ('scanscout', 'ScanScout', 40, 'web', 'Boston', 'MA', '2007-04-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('going', 'Going', 0, 'web', 'Boston', 'MA', '2007-07-01 00:00:00', 5000000, 'USD', 'a'),\
    ('brightcove', 'Brightcove', 0, 'web', 'Cambridge', 'MA', '2005-03-01 00:00:00', 5500000, 'USD', 'a'),\
    ('brightcove', 'Brightcove', 0, 'web', 'Cambridge', 'MA', '2005-11-01 00:00:00', 16200000, 'USD', 'b'),\
    ('brightcove', 'Brightcove', 0, 'web', 'Cambridge', 'MA', '2007-01-01 00:00:00', 59500000, 'USD', 'c'),\
    ('brightcove', 'Brightcove', 0, 'web', 'Cambridge', 'MA', '2006-09-01 00:00:00', 5000000, 'USD', 'b'),\
    ('brightcove', 'Brightcove', 0, 'web', 'Cambridge', 'MA', '2008-05-01 00:00:00', 4900000, 'USD', 'd'),\
    ('permissiontv', 'PermissionTV', 0, 'web', 'Waltham', 'MA', '2007-06-01 00:00:00', 9000000, 'USD', 'c'),\
    ('bountii', 'Bountii', 0, 'web', 'Boston', 'MA', '2007-06-01 00:00:00', 15000, 'USD', 'seed'),\
    ('conduitlabs', 'Conduit Labs', 0, 'web', 'Cambridge', 'MA', '2007-08-01 00:00:00', 5500000, 'USD', 'a'),\
    ('zoominfo', 'ZoomInfo', 80, 'web', 'Waltham', 'MA', '2004-07-01 00:00:00', 7000000, 'USD', 'a'),\
    ('zoominfo', 'ZoomInfo', 80, 'web', 'Waltham', 'MA', '2004-07-01 00:00:00', 7000000, 'USD', 'a'),\
    ('vlingo', 'Vlingo', 0, 'web', 'Cambridge', 'MA', '2008-04-02 00:00:00', 20000000, 'USD', 'b'),\
    ('vtap', 'Vtap', 0, 'web', 'Andover', 'MA', '2007-05-01 00:00:00', 14000000, 'USD', 'b'),\
    ('mocospace', 'MocoSpace', 25, 'mobile', 'Boston', 'MA', '2007-01-01 00:00:00', 3000000, 'USD', 'a'),\
    ('quattro-wireless', 'Quattro Wireless', 0, 'web', 'Waltham', 'MA', '2007-05-01 00:00:00', 6000000, 'USD', 'a'),\
    ('quattro-wireless', 'Quattro Wireless', 0, 'web', 'Waltham', 'MA', '2007-09-05 00:00:00', 12300000, 'USD', 'b'),\
    ('care-com', 'Care.com', 0, 'web', 'Waltham', 'MA', '2007-05-01 00:00:00', 3100000, 'USD', 'a'),\
    ('onforce', 'OnForce', 0, 'web', 'Lexington', 'MA', '2007-09-01 00:00:00', 6750000, 'USD', 'a'),\
    ('onforce', 'OnForce', 0, 'web', 'Lexington', 'MA', '2006-01-12 00:00:00', 15000000, 'USD', 'a'),\
    ('ratepoint', 'RatePoint', 0, 'web', 'Needham', 'MA', '2007-06-01 00:00:00', 6500000, 'USD', 'a'),\
    ('updown', 'UpDown', 5, 'web', 'Cambridge', 'MA', '2008-01-29 00:00:00', 750000, 'USD', 'angel'),\
    ('eons', 'Eons', 0, 'web', 'Boston', 'MA', '2007-03-01 00:00:00', 22000000, 'USD', 'b'),\
    ('eons', 'Eons', 0, 'web', 'Boston', 'MA', '2006-04-01 00:00:00', 10000000, 'USD', 'a'),\
    ('gamerdna', 'GamerDNA', 0, 'web', 'Cambridge', 'MA', '2008-04-18 00:00:00', 3000000, 'USD', 'a'),\
    ('mixandmeet', 'Mix & Meet', 2, 'web', 'Cambridge', 'MA', '2008-03-29 00:00:00', 400000, 'USD', 'angel'),\
    ('sermo', 'Sermo', 0, 'web', 'Cambridge', 'MA', '2007-09-01 00:00:00', 25000000, 'USD', 'c'),\
    ('sermo', 'Sermo', 0, 'web', 'Cambridge', 'MA', '2007-01-01 00:00:00', 9500000, 'USD', 'b'),\
    ('sermo', 'Sermo', 0, 'web', 'Cambridge', 'MA', '2006-09-01 00:00:00', 3000000, 'USD', 'a'),\
    ('worklight', 'WorkLight', 30, 'web', 'Newton', 'MA', '2006-09-01 00:00:00', 5100000, 'USD', 'a'),\
    ('worklight', 'WorkLight', 30, 'web', 'Newton', 'MA', '2008-04-30 00:00:00', 12000000, 'USD', 'b'),\
    ('matchmine', 'MatchMine', 0, 'web', 'Needham', 'MA', '2007-09-01 00:00:00', 10000000, 'USD', 'a'),\
    ('condodomain', 'CondoDomain', 4, 'other', 'Boston', 'MA', '2005-06-01 00:00:00', 300000, 'USD', 'angel'),\
    ('utest', 'uTest', 10, 'web', 'Ashland', 'MA', '2007-10-01 00:00:00', 2300000, 'USD', 'a'),\
    ('compete', 'Compete', 0, 'web', 'Boston', 'MA', '2007-08-01 00:00:00', 10000000, 'USD', 'c'),\
    ('compete', 'Compete', 0, 'web', 'Boston', 'MA', '2003-10-06 00:00:00', 13000000, 'USD', 'b'),\
    ('azuki', 'Azuki Systems', 30, 'web', 'Acton', 'MA', '2007-09-01 00:00:00', 6000000, 'USD', 'a'),\
    ('acinion', 'Acinion', 0, 'web', 'Acton', 'MA', '2006-12-01 00:00:00', 5000000, 'USD', 'a'),\
    ('acinion', 'Acinion', 0, 'web', 'Acton', 'MA', '2007-10-01 00:00:00', 16000000, 'USD', 'b'),\
    ('xkoto', 'xkoto', 45, 'software', 'Waltham', 'MA', '2007-11-01 00:00:00', 7500000, 'USD', 'b'),\
    ('hubspot', 'HubSpot', 30, 'web', 'Cambridge', 'MA', '2007-09-01 00:00:00', 5000000, 'USD', 'a'),\
    ('hubspot', 'HubSpot', 30, 'web', 'Cambridge', 'MA', '2008-05-16 00:00:00', 12000000, 'USD', 'b'),\
    ('hubspot', 'HubSpot', 30, 'web', 'Cambridge', 'MA', '2006-05-01 00:00:00', 500000, 'USD', 'seed'),\
    ('utterz', 'Utterz', 3, 'mobile', 'Maynard', 'MA', '2007-09-01 00:00:00', 4000000, 'USD', 'a'),\
    ('intronis', 'Intronis', 0, 'web', 'Boston', 'MA', '2007-10-01 00:00:00', 5000000, 'USD', 'a'),\
    ('acquia', 'Acquia', 11, 'web', 'Andover', 'MA', '2007-12-01 00:00:00', 7000000, 'USD', 'a'),\
    ('a123systems', 'A123Systems', 0, 'web', 'Watertown', 'MA', '2007-10-01 00:00:00', 30000000, 'USD', 'd'),\
    ('a123systems', 'A123Systems', 0, 'web', 'Watertown', 'MA', '2007-01-01 00:00:00', 40000000, 'USD', 'c'),\
    ('a123systems', 'A123Systems', 0, 'web', 'Watertown', 'MA', '2006-02-01 00:00:00', 30000000, 'USD', 'b'),\
    ('bostonpower', 'Boston Power', 55, 'web', 'Westborough', 'MA', '2006-11-01 00:00:00', 8000000, 'USD', 'a'),\
    ('bostonpower', 'Boston Power', 55, 'web', 'Westborough', 'MA', '2008-01-01 00:00:00', 45000000, 'USD', 'c'),\
    ('bostonpower', 'Boston Power', 55, 'web', 'Westborough', 'MA', '2007-01-01 00:00:00', 15600000, 'USD', 'b'),\
    ('visiblemeasures', 'Visible Measures', 0, 'web', 'Boston', 'MA', '2007-03-01 00:00:00', 5000000, 'USD', 'a'),\
    ('visiblemeasures', 'Visible Measures', 0, 'web', 'Boston', 'MA', '2008-01-01 00:00:00', 13500000, 'USD', 'b'),\
    ('visiblemeasures', 'Visible Measures', 0, 'web', 'Boston', 'MA', '2006-01-01 00:00:00', 800000, 'USD', 'seed'),\
    ('endeca', 'Endeca', 500, 'web', 'Cambridge', 'MA', '2004-06-01 00:00:00', 15000000, 'USD', 'c'),\
    ('endeca', 'Endeca', 500, 'web', 'Cambridge', 'MA', '2008-01-01 00:00:00', 15000000, 'USD', 'd'),\
    ('endeca', 'Endeca', 500, 'web', 'Cambridge', 'MA', '2001-11-01 00:00:00', 15000000, 'USD', 'b'),\
    ('endeca', 'Endeca', 500, 'web', 'Cambridge', 'MA', '2001-01-01 00:00:00', 10000000, 'USD', 'a'),\
    ('choicestream', 'ChoiceStream', 70, 'software', 'Cambridge', 'MA', '2000-01-01 00:00:00', 15000000, 'USD', 'seed'),\
    ('choicestream', 'ChoiceStream', 70, 'software', 'Cambridge', 'MA', '2005-02-22 00:00:00', 7000000, 'USD', 'a'),\
    ('choicestream', 'ChoiceStream', 70, 'software', 'Cambridge', 'MA', '2006-06-01 00:00:00', 13100000, 'USD', 'b'),\
    ('choicestream', 'ChoiceStream', 70, 'software', 'Cambridge', 'MA', '2007-04-04 00:00:00', 26500000, 'USD', 'c'),\
    ('youcastr', 'YouCastr', 5, 'web', 'Cambridge', 'MA', '2008-02-01 00:00:00', 100000, 'USD', 'angel'),\
    ('jackpot-rewards', 'JackPot Rewards', 0, 'web', 'Newton', 'MA', '2008-02-01 00:00:00', 16700000, 'USD', 'a'),\
    ('mzinga', 'Mzinga', 0, 'web', 'Burlington', 'MA', '2008-03-03 00:00:00', 32500000, 'USD', 'd'),\
    ('dimdim', 'Dimdim', 0, 'web', 'Burlington', 'MA', '2007-02-01 00:00:00', 2400000, 'USD', 'a'),\
    ('dimdim', 'Dimdim', 0, 'web', 'Burlington', 'MA', '2008-07-01 00:00:00', 6000000, 'USD', 'b'),\
    ('utoopia', 'utoopia', 2, 'web', 'Boston', 'MA', '2007-03-01 00:00:00', 100000, 'USD', 'seed'),\
    ('retail-convergence', 'Retail Convergence', 0, 'web', 'Boston', 'MA', '2008-04-24 00:00:00', 25000000, 'USD', 'a'),\
    ('frame-media', 'Frame Media', 0, 'web', 'Wellesley', 'MA', '2007-11-09 00:00:00', 2000000, 'USD', 'a'),\
    ('frame-media', 'Frame Media', 0, 'web', 'Wellesley', 'MA', '2008-05-02 00:00:00', 3000000, 'USD', 'unattributed'),\
    ('turbine', 'Turbine', 200, 'web', 'Westwood', 'MA', '2008-04-29 00:00:00', 40000000, 'USD', 'c'),\
    ('turbine', 'Turbine', 200, 'web', 'Westwood', 'MA', '2005-05-09 00:00:00', 30000000, 'USD', 'b'),\
    ('zeer', 'Zeer', 5, 'web', 'Boston', 'MA', '2007-08-01 00:00:00', 1050000, 'USD', 'angel'),\
    ('simpletuition', 'SimpleTuition', 0, 'web', 'Newton', 'MA', '2006-04-14 00:00:00', 4400000, 'USD', 'a'),\
    ('simpletuition', 'SimpleTuition', 0, 'web', 'Newton', 'MA', '2006-12-18 00:00:00', 7500000, 'USD', 'b'),\
    ('sirtris-pharmaceuticals', 'Sirtris Pharmaceuticals', 0, 'biotech', 'Cambridge', 'MA', '2004-12-01 00:00:00', 13000000, 'USD', 'a'),\
    ('sirtris-pharmaceuticals', 'Sirtris Pharmaceuticals', 0, 'biotech', 'Cambridge', 'MA', '2005-03-08 00:00:00', 27000000, 'USD', 'b'),\
    ('sirtris-pharmaceuticals', 'Sirtris Pharmaceuticals', 0, 'biotech', 'Cambridge', 'MA', '2006-04-19 00:00:00', 37000000, 'USD', 'c'),\
    ('good-data', 'Good Data', 30, 'web', 'Cambridge', 'MA', '2008-07-23 00:00:00', 2000000, 'USD', 'seed'),\
    ('navic-networks', 'Navic Networks', 0, 'web', 'Waltham', 'MA', '2000-02-14 00:00:00', 2000000, 'USD', 'a'),\
    ('navic-networks', 'Navic Networks', 0, 'web', 'Waltham', 'MA', '2001-02-26 00:00:00', 20000000, 'USD', 'c'),\
    ('navic-networks', 'Navic Networks', 0, 'web', 'Waltham', 'MA', '2000-06-07 00:00:00', 20000000, 'USD', 'b'),\
    ('pivot', 'Pivot', 40, 'software', 'Cambridge', 'MA', '2004-12-01 00:00:00', 5000000, 'USD', 'a'),\
    ('pivot', 'Pivot', 40, 'software', 'Cambridge', 'MA', '2006-04-01 00:00:00', 8000000, 'USD', 'b'),\
    ('optaros', 'Optaros', 200, 'consulting', 'Boston', 'MA', '2008-06-01 00:00:00', 12000000, 'USD', 'c'),\
    ('optaros', 'Optaros', 200, 'consulting', 'Boston', 'MA', '2006-11-20 00:00:00', 13000000, 'USD', 'b'),\
    ('optaros', 'Optaros', 200, 'consulting', 'Boston', 'MA', '2005-03-09 00:00:00', 7000000, 'USD', 'a'),\
    ('astaro', 'Astaro', 0, 'hardware', 'Burlington', 'MA', '2003-05-01 00:00:00', 6200000, 'USD', 'a'),\
    ('astaro', 'Astaro', 0, 'hardware', 'Burlington', 'MA', '2004-05-01 00:00:00', 6700000, 'USD', 'b'),\
    ('pangea-media', 'Pangea Media', 25, 'web', 'Watertown', 'MA', '2007-04-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('posterous', 'Posterous', 0, 'web', 'Boston', 'MA', '2008-05-01 00:00:00', 15000, 'USD', 'seed'),\
    ('spire', 'Spire', 0, 'web', 'Boston', 'MA', '2008-07-08 00:00:00', 9000000, 'USD', 'a'),\
    ('neosaej', 'neoSaej', 0, 'web', 'Burlington', 'MA', '2008-07-14 00:00:00', 7000000, 'USD', 'unattributed'),\
    ('genarts', 'GenArts', 0, 'software', 'Cambridge', 'MA', '2008-07-17 00:00:00', 22400000, 'USD', 'unattributed'),\
    ('ordermotion', 'OrderMotion', 0, 'web', 'Boston', 'MA', '2008-07-21 00:00:00', 1400000, 'USD', 'unattributed'),\
    ('ham-it', 'HAM-IT', 7, 'web', 'North Andover', 'MA', '2007-10-01 00:00:00', 340000, 'USD', 'seed'),\
    ('freewebs', 'Freewebs', 35, 'web', 'Silver Spring', 'MD', '2006-08-01 00:00:00', 12000000, 'USD', 'a'),\
    ('mptrax', 'MPTrax', 4, 'web', 'Baltimore', 'MD', '2007-01-01 00:00:00', 350000, 'USD', 'angel'),\
    ('jackbe', 'JackBe', 0, 'web', 'Chevy Chase', 'MD', '2007-10-01 00:00:00', 9500000, 'USD', 'c'),\
    ('zenimax', 'ZeniMax', 0, 'web', 'Rockville', 'MD', '2007-10-01 00:00:00', 300000000, 'USD', 'a'),\
    ('zenimax', 'ZeniMax', 0, 'web', 'Rockville', 'MD', '2008-05-30 00:00:00', 9900000, 'USD', 'a'),\
    ('intelliworks', 'Intelliworks', 24, 'software', 'Rockville', 'MD', '2005-02-14 00:00:00', 6000000, 'USD', 'a'),\
    ('intelliworks', 'Intelliworks', 24, 'software', 'Rockville', 'MD', '2006-10-10 00:00:00', 10000000, 'USD', 'b'),\
    ('intelliworks', 'Intelliworks', 24, 'software', 'Rockville', 'MD', '2008-04-07 00:00:00', 4000000, 'USD', 'c'),\
    ('hexio', 'HEXIO', 1, 'web', 'Kennebunk', 'ME', '2008-01-10 00:00:00', 20000, 'USD', 'seed'),\
    ('foneshow', 'Foneshow', 8, 'web', 'Portland', 'ME', '2007-09-01 00:00:00', 1050000, 'USD', 'a'),\
    ('foneshow', 'Foneshow', 8, 'web', 'Portland', 'ME', '2006-09-01 00:00:00', 25000, 'USD', 'seed'),\
    ('zattoo', 'Zattoo', 45, 'web', 'Ann Arbor', 'MI', '2007-11-01 00:00:00', 10000000, 'USD', 'b'),\
    ('loudclick', 'LoudClick', 9, 'web', 'Richfield', 'MN', '2008-03-31 00:00:00', 600000, 'USD', 'angel'),\
    ('agilis-systems', 'Agilis Systems', 0, 'mobile', 'St. Louis', 'MO', '2008-05-16 00:00:00', 5000000, 'USD', 'b'),\
    ('international-liars-poker-association', 'International Liars Poker Association', 24, 'other', 'St. Louis', 'MO', '2007-11-01 00:00:00', 1250000, 'USD', 'seed'),\
    ('channeladvisor', 'ChannelAdvisor', 300, 'web', 'Morrisville', 'NC', '2007-05-01 00:00:00', 30000000, 'USD', 'c'),\
    ('channeladvisor', 'ChannelAdvisor', 300, 'web', 'Morrisville', 'NC', '2004-01-22 00:00:00', 7000000, 'USD', 'a'),\
    ('channeladvisor', 'ChannelAdvisor', 300, 'web', 'Morrisville', 'NC', '2005-04-28 00:00:00', 18000000, 'USD', 'b'),\
    ('yap', 'Yap', 0, 'mobile', 'Charlotte', 'NC', '2007-03-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('yap', 'Yap', 0, 'mobile', 'Charlotte', 'NC', '2008-06-10 00:00:00', 6500000, 'USD', 'a'),\
    ('prepchamps', 'PrepChamps', 10, 'web', 'Durham', 'NC', '2008-04-22 00:00:00', 1200000, 'USD', 'a'),\
    ('silkroad-technology', 'SilkRoad technology', 0, 'unknown', 'Winston-Salem', 'NC', '2008-02-19 00:00:00', 10000000, 'USD', 'b'),\
    ('silkroad-technology', 'SilkRoad technology', 0, 'unknown', 'Winston-Salem', 'NC', '2008-05-15 00:00:00', 54000000, 'USD', 'c'),\
    ('brightdoor-systems', 'BrightDoor Systems', 26, 'web', 'Cary', 'NC', '2007-04-13 00:00:00', 200000, 'USD', 'a'),\
    ('drifttoit', 'DriftToIt', 4, 'web', 'Raleigh', 'NC', '2007-06-01 00:00:00', 300000, 'USD', 'angel'),\
    ('rpath', 'rPath', 45, 'other', 'Raleigh', 'NC', '2006-01-24 00:00:00', 6400000, 'USD', 'a'),\
    ('rpath', 'rPath', 45, 'other', 'Raleigh', 'NC', '2007-01-24 00:00:00', 9100000, 'USD', 'b'),\
    ('rpath', 'rPath', 45, 'other', 'Raleigh', 'NC', '2008-06-24 00:00:00', 10000000, 'USD', 'c'),\
    ('ntractive', 'Ntractive', 6, 'web', 'Grand Forks', 'ND', '2008-07-08 00:00:00', 570000, 'USD', 'a'),\
    ('billmelater', 'Bill Me Later', 0, 'web', 'Omaha', 'NE', '2006-03-01 00:00:00', 27400000, 'USD', 'a'),\
    ('adaptiveblue', 'AdaptiveBlue', 6, 'web', 'Livingston', 'NJ', '2007-02-15 00:00:00', 1500000, 'USD', 'a'),\
    ('phanfare', 'Phanfare', 0, 'web', 'Metuchen', 'NJ', '2007-11-01 00:00:00', 2500000, 'USD', 'c'),\
    ('phanfare', 'Phanfare', 0, 'web', 'Metuchen', 'NJ', '2006-07-01 00:00:00', 2000000, 'USD', 'b'),\
    ('enterprisedb', 'EnterpriseDB', 0, 'software', 'Edison', 'NJ', '2005-09-07 00:00:00', 7000000, 'USD', 'a'),\
    ('enterprisedb', 'EnterpriseDB', 0, 'software', 'Edison', 'NJ', '2006-08-01 00:00:00', 20000000, 'USD', 'b'),\
    ('enterprisedb', 'EnterpriseDB', 0, 'software', 'Edison', 'NJ', '2008-03-25 00:00:00', 10000000, 'USD', 'c'),\
    ('neocleus', 'Neocleus', 0, 'web', 'Jersey City', 'NJ', '2008-06-19 00:00:00', 11000000, 'USD', 'b'),\
    ('datapipe', 'Datapipe', 0, 'web', 'Jersey City', 'NJ', '2008-07-08 00:00:00', 75000000, 'USD', 'unattributed'),\
    ('switch2health', 'Switch2Health', 0, 'web', 'North Brunswick', 'NJ', '2008-07-16 00:00:00', 200000, 'USD', 'unattributed'),\
    ('voltaix', 'Voltaix', 0, 'hardware', 'N. Branch', 'NJ', '2008-07-29 00:00:00', 12500000, 'USD', 'unattributed'),\
    ('spaboom', 'SpaBoom', 9, 'web', 'Albuquerque', 'NM', '2006-07-15 00:00:00', 700000, 'USD', 'a'),\
    ('spaboom', 'SpaBoom', 9, 'web', 'Albuquerque', 'NM', '2007-09-15 00:00:00', 600000, 'USD', 'b'),\
    ('novint', 'Novint', 0, 'other', 'Albuquerque', 'NM', '2008-06-17 00:00:00', 5200000, 'USD', 'unattributed'),\
    ('meetmoi', 'MeetMoi', 7, 'web', 'New York City', 'NY', '2007-06-01 00:00:00', 1500000, 'USD', 'a'),\
    ('meetup', 'Meetup', 1, 'mobile', 'New York', 'NY', '2002-12-01 00:00:00', 2800000, 'USD', 'a'),\
    ('meetup', 'Meetup', 1, 'mobile', 'New York', 'NY', '2003-11-01 00:00:00', 5300000, 'USD', 'b'),\
    ('meetup', 'Meetup', 1, 'mobile', 'New York', 'NY', '2008-07-23 00:00:00', 7500000, 'USD', 'd'),\
    ('mogulus', 'Mogulus', 20, 'web', 'New York', 'NY', '2007-05-01 00:00:00', 1200000, 'USD', 'angel'),\
    ('mogulus', 'Mogulus', 20, 'web', 'New York', 'NY', '2008-01-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('mogulus', 'Mogulus', 20, 'web', 'New York', 'NY', '2008-07-01 00:00:00', 10000000, 'USD', 'a'),\
    ('pando', 'Pando', 23, 'software', 'New York', 'NY', '2007-01-01 00:00:00', 11000000, 'USD', 'b'),\
    ('pando', 'Pando', 23, 'software', 'New York', 'NY', '2008-03-01 00:00:00', 20900000, 'USD', 'c'),\
    ('outside-in', 'Outside.in', 17, 'web', 'Brooklyn', 'NY', '2007-02-01 00:00:00', 900000, 'USD', 'angel'),\
    ('outside-in', 'Outside.in', 17, 'web', 'Brooklyn', 'NY', '2007-10-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('outside-in', 'Outside.in', 17, 'web', 'Brooklyn', 'NY', '2008-05-19 00:00:00', 3000000, 'USD', 'a'),\
    ('selectminds', 'SelectMinds', 55, 'web', 'New York', 'NY', '2007-08-01 00:00:00', 5500000, 'USD', 'a'),\
    ('veotag', 'Veotag', 0, 'web', 'New York', 'NY', '2007-05-01 00:00:00', 750000, 'USD', 'angel'),\
    ('roo', 'KIT digital', 100, 'web', 'New York', 'NY', '2008-05-08 00:00:00', 15000000, 'USD', 'b'),\
    ('roo', 'KIT digital', 100, 'web', 'New York', 'NY', '2008-04-18 00:00:00', 5000000, 'USD', 'a'),\
    ('contextweb', 'ContextWeb', 0, 'web', 'New York', 'NY', '2004-06-01 00:00:00', 3000000, 'USD', 'a'),\
    ('contextweb', 'ContextWeb', 0, 'web', 'New York', 'NY', '2005-06-01 00:00:00', 9000000, 'USD', 'b'),\
    ('contextweb', 'ContextWeb', 0, 'web', 'New York', 'NY', '2008-07-20 00:00:00', 26000000, 'USD', 'd'),\
    ('datranmedia', 'Datran Media', 0, 'web', 'New York', 'NY', '2005-03-01 00:00:00', 60000000, 'USD', 'a'),\
    ('eyeblaster', 'Eyeblaster', 0, 'web', 'New York', 'NY', '2003-12-01 00:00:00', 8000000, 'USD', 'a'),\
    ('eyeblaster', 'Eyeblaster', 0, 'web', 'New York', 'NY', '2007-03-21 00:00:00', 30000000, 'USD', 'b'),\
    ('covestor', 'Covestor', 0, 'web', 'New York', 'NY', '2007-06-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('covestor', 'Covestor', 0, 'web', 'New York', 'NY', '2008-04-07 00:00:00', 6500000, 'USD', 'a'),\
    ('globalgrind', 'Global Grind', 20, 'web', 'New York', 'NY', '2007-08-01 00:00:00', 4500000, 'USD', 'b'),\
    ('heavy-com', 'Heavy.com', 0, 'web', 'New York', 'NY', '2001-03-01 00:00:00', 3000000, 'USD', 'a'),\
    ('spiralfrog', 'SpiralFrog', 15, 'web', 'New York', 'NY', '2007-12-01 00:00:00', 2000000, 'USD', 'debt_round'),\
    ('broadbandenterprises', 'Broadband Enterprises', 0, 'web', 'New York', 'NY', '2008-01-01 00:00:00', 10000000, 'USD', 'a'),\
    ('thumbplay', 'Thumbplay', 0, 'mobile', 'New York', 'NY', '2008-03-13 00:00:00', 18000000, 'USD', 'e'),\
    ('waterfrontmedia', 'Waterfront Media', 0, 'web', 'Brooklyn New York', 'NY', '2007-09-01 00:00:00', 25000000, 'USD', 'd'),\
    ('waterfrontmedia', 'Waterfront Media', 0, 'web', 'Brooklyn New York', 'NY', '2004-03-01 00:00:00', 4000000, 'USD', 'a'),\
    ('waterfrontmedia', 'Waterfront Media', 0, 'web', 'Brooklyn New York', 'NY', '2006-03-01 00:00:00', 6000000, 'USD', 'b'),\
    ('waterfrontmedia', 'Waterfront Media', 0, 'web', 'Brooklyn New York', 'NY', '2007-04-01 00:00:00', 8000000, 'USD', 'c'),\
    ('tutor', 'Tutor', 55, 'web', 'New Yorl', 'NY', '2007-05-01 00:00:00', 13500000, 'USD', 'b'),\
    ('daylife', 'Daylife', 0, 'web', 'New York', 'NY', '2007-06-01 00:00:00', 8300000, 'USD', 'b'),\
    ('teachthepeople', 'Teach The People', 1, 'web', 'Astoria', 'NY', '2007-01-01 00:00:00', 300000, 'USD', 'angel'),\
    ('healthcare-health-human-powered-search', 'OrganizedWisdom', 18, 'web', 'New York', 'NY', '2008-06-01 00:00:00', 2300000, 'USD', 'a'),\
    ('snooth', 'Snooth', 0, 'web', 'New  York', 'NY', '2006-12-01 00:00:00', 300000, 'USD', 'angel'),\
    ('snooth', 'Snooth', 0, 'web', 'New  York', 'NY', '2007-11-01 00:00:00', 1000000, 'USD', 'a'),\
    ('5min', '5min', 8, 'web', 'New York', 'NY', '2007-04-01 00:00:00', 300000, 'USD', 'angel'),\
    ('5min', '5min', 8, 'web', 'New York', 'NY', '2007-11-01 00:00:00', 5000000, 'USD', 'a'),\
    ('kaltura', 'Kaltura', 20, 'web', 'Brooklyn', 'NY', '2007-05-01 00:00:00', 2100000, 'USD', 'a'),\
    ('mimeo', 'Mimeo', 0, 'web', 'New York', 'NY', '2007-09-01 00:00:00', 25000000, 'USD', 'a'),\
    ('rayv', 'RayV', 0, 'web', 'New York', 'NY', '2007-10-01 00:00:00', 8000000, 'USD', 'b'),\
    ('tumblr', 'Tumblr', 0, 'web', 'New York', 'NY', '2007-10-31 00:00:00', 750000, 'USD', 'a'),\
    ('payoneer', 'Payoneer', 50, 'web', 'New York', 'NY', '2008-07-24 00:00:00', 8000000, 'USD', 'b'),\
    ('exelate', 'eXelate', 0, 'web', 'New York', 'NY', '2007-10-01 00:00:00', 4000000, 'USD', 'a'),\
    ('quigo', 'Quigo', 0, 'web', 'New York', 'NY', '2004-03-01 00:00:00', 5000000, 'USD', 'a'),\
    ('peer39', 'Peer39', 50, 'web', 'New York', 'NY', '2007-02-01 00:00:00', 3000000, 'USD', 'a'),\
    ('peer39', 'Peer39', 50, 'web', 'New York', 'NY', '2007-09-01 00:00:00', 8200000, 'USD', 'b'),\
    ('peer39', 'Peer39', 50, 'web', 'New York', 'NY', '2006-03-01 00:00:00', 500000, 'USD', 'seed'),\
    ('rebelmonkey', 'Rebel Monkey', 15, 'web', 'New York', 'NY', '2008-02-05 00:00:00', 1000000, 'USD', 'a'),\
    ('answers', 'Answers Corporation', 0, 'web', 'New York', 'NY', '1999-01-01 00:00:00', 300000, 'USD', 'a'),\
    ('answers', 'Answers Corporation', 0, 'web', 'New York', 'NY', '1999-04-01 00:00:00', 1360000, 'USD', 'b'),\
    ('answers', 'Answers Corporation', 0, 'web', 'New York', 'NY', '1999-09-01 00:00:00', 2750000, 'USD', 'c'),\
    ('answers', 'Answers Corporation', 0, 'web', 'New York', 'NY', '2000-05-01 00:00:00', 28000000, 'USD', 'd'),\
    ('answers', 'Answers Corporation', 0, 'web', 'New York', 'NY', '2008-06-17 00:00:00', 13000000, 'USD', 'debt_round'),\
    ('seetoo', 'SeeToo', 0, 'web', 'New York', 'NY', '2007-10-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('drop-io-2', 'drop.io', 0, 'web', 'Brooklyn', 'NY', '2007-11-01 00:00:00', 1250000, 'USD', 'a'),\
    ('drop-io-2', 'drop.io', 0, 'web', 'Brooklyn', 'NY', '2008-03-10 00:00:00', 2700000, 'USD', 'b'),\
    ('motionbox', 'Motionbox', 0, 'web', 'New York', 'NY', '2006-09-01 00:00:00', 4200000, 'USD', 'a'),\
    ('motionbox', 'Motionbox', 0, 'web', 'New York', 'NY', '2007-12-01 00:00:00', 7000000, 'USD', 'b'),\
    ('dgplabs', 'DGP Labs', 0, 'web', 'New York', 'NY', '2007-11-01 00:00:00', 4750000, 'USD', 'a'),\
    ('ideeli', 'Ideeli', 0, 'web', 'New York', 'NY', '2007-12-01 00:00:00', 3800000, 'USD', 'a'),\
    ('livegamer', 'Live Gamer', 0, 'web', 'New York', 'NY', '2007-12-01 00:00:00', 24000000, 'USD', 'a'),\
    ('weshow', 'WeShow', 0, 'web', 'New York', 'NY', '2007-06-01 00:00:00', 5000000, 'USD', 'a'),\
    ('etsy', 'Etsy', 45, 'web', 'Brooklyn', 'NY', '2006-06-01 00:00:00', 315000, 'USD', 'angel'),\
    ('etsy', 'Etsy', 45, 'web', 'Brooklyn', 'NY', '2006-11-01 00:00:00', 1000000, 'USD', 'a'),\
    ('etsy', 'Etsy', 45, 'web', 'Brooklyn', 'NY', '2007-07-01 00:00:00', 3250000, 'USD', 'b'),\
    ('etsy', 'Etsy', 45, 'web', 'Brooklyn', 'NY', '2008-01-01 00:00:00', 27000000, 'USD', 'c'),\
    ('phonetag', 'PhoneTag', 10, 'web', 'New York', 'NY', '2007-02-01 00:00:00', 3500000, 'USD', 'angel'),\
    ('phonetag', 'PhoneTag', 10, 'web', 'New York', 'NY', '2004-01-01 00:00:00', 200000, 'USD', 'seed'),\
    ('phonetag', 'PhoneTag', 10, 'web', 'New York', 'NY', '2007-12-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('tremormedia', 'Tremor Media', 0, 'web', 'New York', 'NY', '2006-09-01 00:00:00', 8400000, 'USD', 'a'),\
    ('tremormedia', 'Tremor Media', 0, 'web', 'New York', 'NY', '2008-01-01 00:00:00', 11000000, 'USD', 'b'),\
    ('yoonew', 'YooNew', 0, 'web', 'New York', 'NY', '2005-08-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('fifthgenerationsystems', 'Fifth Generation Systems', 0, 'web', 'Roslyn Heights', 'NY', '2008-01-01 00:00:00', 5300000, 'USD', 'b'),\
    ('fifthgenerationsystems', 'Fifth Generation Systems', 0, 'web', 'Roslyn Heights', 'NY', '2006-11-01 00:00:00', 5250000, 'USD', 'a'),\
    ('igaworldwide', 'IGA Worldwide', 0, 'web', 'New York', 'NY', '2006-02-01 00:00:00', 12000000, 'USD', 'a'),\
    ('igaworldwide', 'IGA Worldwide', 0, 'web', 'New York', 'NY', '2007-07-01 00:00:00', 25000000, 'USD', 'b'),\
    ('igaworldwide', 'IGA Worldwide', 0, 'web', 'New York', 'NY', '2008-01-01 00:00:00', 5000000, 'USD', 'b'),\
    ('theladders', 'TheLadders', 200, 'web', 'New York', 'NY', '2004-11-08 00:00:00', 7250000, 'USD', 'a'),\
    ('adotube', 'Adotube', 30, 'web', 'New York', 'NY', '2007-07-31 00:00:00', 630000, 'USD', 'seed'),\
    ('adotube', 'Adotube', 30, 'web', 'New York', 'NY', '2008-04-01 00:00:00', 600000, 'USD', 'seed'),\
    ('kluster', 'Kluster', 0, 'web', 'New York', 'NY', '2008-01-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('blog-talk-radio', 'Blog Talk Radio', 0, 'web', 'New York', 'NY', '2008-06-25 00:00:00', 4600000, 'USD', 'a'),\
    ('pingg', 'Pingg', 7, 'web', 'New York', 'NY', '2007-01-01 00:00:00', 500000, 'USD', 'seed'),\
    ('pingg', 'Pingg', 7, 'web', 'New York', 'NY', '2007-03-01 00:00:00', 800000, 'USD', 'angel'),\
    ('outbrain', 'Outbrain', 0, 'web', 'New York City', 'NY', '2007-01-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('outbrain', 'Outbrain', 0, 'web', 'New York City', 'NY', '2008-02-25 00:00:00', 5000000, 'USD', 'a'),\
    ('mochila', 'Mochila', 52, 'web', 'New York', 'NY', '2007-01-02 00:00:00', 8000000, 'USD', 'b'),\
    ('olx', 'OLX', 85, 'web', 'New York City', 'NY', '2007-09-01 00:00:00', 10000000, 'USD', 'a'),\
    ('olx', 'OLX', 85, 'web', 'New York City', 'NY', '2008-04-11 00:00:00', 13500000, 'USD', 'b'),\
    ('boonty', 'Boonty', 150, 'web', 'New York', 'NY', '2005-07-07 00:00:00', 10000000, 'USD', 'b'),\
    ('tripology', 'Tripology', 0, 'web', 'New York', 'NY', '2007-01-01 00:00:00', 1250000, 'USD', 'a'),\
    ('the-feedroom', 'The Feedroom', 0, 'web', 'New York', 'NY', '2008-07-09 00:00:00', 12000000, 'USD', 'unattributed'),\
    ('next-new-networks', 'Next New Networks', 15, 'web', 'New York', 'NY', '2007-01-01 00:00:00', 8000000, 'USD', 'a'),\
    ('next-new-networks', 'Next New Networks', 15, 'web', 'New York', 'NY', '2008-03-12 00:00:00', 15000000, 'USD', 'b'),\
    ('riverwired', 'RiverWired', 0, 'web', 'New York', 'NY', '2008-03-01 00:00:00', 1500000, 'USD', 'seed'),\
    ('fynanz', 'Fynanz', 10, 'web', 'New York', 'NY', '2007-10-01 00:00:00', 500000, 'USD', 'angel'),\
    ('digital-railroad', 'Digital Railroad', 0, 'web', 'New York', 'NY', '2005-06-14 00:00:00', 5200000, 'USD', 'a'),\
    ('digital-railroad', 'Digital Railroad', 0, 'web', 'New York', 'NY', '2007-02-05 00:00:00', 10000000, 'USD', 'b'),\
    ('silicon-alley-insider', 'Silicon Alley Insider', 0, 'unknown', 'New York', 'NY', '2008-07-16 00:00:00', 900000, 'USD', 'seed'),\
    ('undertone-networks', 'Undertone Networks', 0, 'web', 'New York', 'NY', '2008-03-31 00:00:00', 40000000, 'USD', 'a'),\
    ('someecards', 'Someecards', 0, 'web', 'New York', 'NY', '2008-04-01 00:00:00', 350000, 'USD', 'seed'),\
    ('trunkt', 'Trunkt', 2, 'web', 'New York', 'NY', '2007-05-15 00:00:00', 350000, 'USD', 'seed'),\
    ('cutcaster-2', 'Cutcaster', 6, 'web', 'New York City', 'NY', '2008-01-01 00:00:00', 150000, 'USD', 'seed'),\
    ('visible-world', 'Visible World', 0, 'software', 'New York', 'NY', '2008-04-14 00:00:00', 25000000, 'USD', 'c'),\
    ('visible-world', 'Visible World', 0, 'software', 'New York', 'NY', '2003-11-17 00:00:00', 8000000, 'USD', 'b'),\
    ('mad-mimi', 'Mad Mimi', 3, 'web', 'unknown', 'NY', '2007-04-29 00:00:00', 200000, 'USD', 'angel'),\
    ('social-median', 'Social Median', 0, 'web', 'New York', 'NY', '2008-03-07 00:00:00', 500000, 'USD', 'seed'),\
    ('scanbuy', 'Scanbuy', 0, 'software', 'New York', 'NY', '2006-12-12 00:00:00', 9000000, 'USD', 'b'),\
    ('scanbuy', 'Scanbuy', 0, 'software', 'New York', 'NY', '2007-11-01 00:00:00', 6800000, 'USD', 'b'),\
    ('pontiflex', 'Pontiflex', 0, 'web', 'Brooklyn', 'NY', '2008-04-30 00:00:00', 2500000, 'USD', 'a'),\
    ('chic-tv', 'CHIC.TV', 12, 'web', 'New York', 'NY', '2006-01-01 00:00:00', 500000, 'USD', 'angel'),\
    ('expertflyer', 'ExpertFlyer', 3, 'web', 'Patchogue', 'NY', '2004-03-01 00:00:00', 30000, 'USD', 'seed'),\
    ('x-1', 'x+1', 0, 'web', 'New York', 'NY', '2008-06-01 00:00:00', 16000000, 'USD', 'a'),\
    ('media6', 'Media6¡', 0, 'web', 'New York', 'NY', '2008-05-01 00:00:00', 2000000, 'USD', 'debt_round'),\
    ('knewton', 'Knewton', 0, 'web', 'New York', 'NY', '2008-05-21 00:00:00', 2500000, 'USD', 'a'),\
    ('sense-networks', 'Sense Networks', 0, 'mobile', 'New York City', 'NY', '2008-04-01 00:00:00', 3000000, 'USD', 'a'),\
    ('targetspot', 'TargetSpot', 20, 'web', 'New York', 'NY', '2008-03-11 00:00:00', 8600000, 'USD', 'b'),\
    ('bountyjobs', 'BountyJobs', 31, 'web', 'New York', 'NY', '2008-07-07 00:00:00', 12000000, 'USD', 'b'),\
    ('vbs-tv', 'vbs tv', 40, 'other', 'Brooklyn', 'NY', '2006-12-01 00:00:00', 10000000, 'USD', 'seed'),\
    ('instinctiv', 'Instinctiv', 0, 'mobile', 'Ithaca', 'NY', '2008-06-01 00:00:00', 750000, 'USD', 'seed'),\
    ('visualplant', 'VISUALPLANT', 6, 'web', 'New York', 'NY', '2007-03-01 00:00:00', 500000, 'USD', 'seed'),\
    ('xunlight', 'XunLight', 0, 'hardware', 'Toledo', 'OH', '2008-05-07 00:00:00', 22000000, 'USD', 'b'),\
    ('strands', 'Strands', 150, 'web', 'Corvallis', 'OR', '2006-03-01 00:00:00', 6000000, 'USD', 'a'),\
    ('strands', 'Strands', 150, 'web', 'Corvallis', 'OR', '2007-06-18 00:00:00', 25000000, 'USD', 'b'),\
    ('strands', 'Strands', 150, 'web', 'Corvallis', 'OR', '2007-12-01 00:00:00', 24000000, 'USD', 'b'),\
    ('jivesoftware', 'Jive Software', 0, 'web', 'Portland', 'OR', '2007-08-01 00:00:00', 15000000, 'USD', 'a'),\
    ('platial', 'Platial', 0, 'web', 'Portland', 'OR', '2005-10-01 00:00:00', 800000, 'USD', 'angel'),\
    ('platial', 'Platial', 0, 'web', 'Portland', 'OR', '2007-02-01 00:00:00', 2600000, 'USD', 'a'),\
    ('garagegames', 'GarageGames', 0, 'web', 'Eugene', 'OR', '2007-09-01 00:00:00', 50000000, 'USD', 'a'),\
    ('iovation', 'iovation', 50, 'web', 'Portland', 'OR', '2008-03-27 00:00:00', 15000000, 'USD', 'a'),\
    ('ingrid', 'InGrid', 0, 'web', 'Berwyn', 'PA', '2007-07-01 00:00:00', 13500000, 'USD', 'c'),\
    ('ingrid', 'InGrid', 0, 'web', 'Berwyn', 'PA', '2004-09-01 00:00:00', 6600000, 'USD', 'a'),\
    ('ingrid', 'InGrid', 0, 'web', 'Berwyn', 'PA', '2006-06-01 00:00:00', 8100000, 'USD', 'b'),\
    ('ingrid', 'InGrid', 0, 'web', 'Berwyn', 'PA', '2006-06-01 00:00:00', 1500000, 'USD', 'debt_round'),\
    ('sleep-fm', 'Sleep.FM', 4, 'web', 'Philadelphia', 'PA', '2008-03-30 00:00:00', 15000, 'USD', 'seed'),\
    ('boomi', 'Boomi', 0, 'web', 'Berwyn', 'PA', '2008-07-01 00:00:00', 4000000, 'USD', 'a'),\
    ('styky', 'Styky', 3, 'web', 'Philadelphia', 'PA', '2007-01-01 00:00:00', 100000, 'USD', 'seed'),\
    ('totaltakeout', 'TotalTakeout', 25, 'web', 'Allentown', 'PA', '2007-10-01 00:00:00', 150000, 'USD', 'seed'),\
    ('redlasso', 'RedLasso', 0, 'web', 'King of Prussia', 'PA', '2007-11-01 00:00:00', 6500000, 'CAD', 'a'),\
    ('qpondirect', 'QponDirect', 10, 'web', 'Pittsburgh', 'PA', '2008-03-01 00:00:00', 300000, 'USD', 'angel'),\
    ('energyweb-solutions', 'EnergyWeb Solutions', 3, 'web', 'Allentown', 'PA', '2006-06-15 00:00:00', 100000, 'USD', 'seed'),\
    ('aria-systems', 'Aria Systems', 0, 'software', 'Media', 'PA', '2007-11-07 00:00:00', 4000000, 'USD', 'a'),\
    ('vivsimo', 'Viv’simo', 0, 'software', 'Pittsburgh', 'PA', '2008-03-17 00:00:00', 4000000, 'USD', 'a'),\
    ('vivsimo', 'Viv’simo', 0, 'software', 'Pittsburgh', 'PA', '2003-07-12 00:00:00', 960000, 'USD', 'seed'),\
    ('vivsimo', 'Viv’simo', 0, 'software', 'Pittsburgh', 'PA', '2002-07-04 00:00:00', 500000, 'USD', 'seed'),\
    ('vivsimo', 'Viv’simo', 0, 'software', 'Pittsburgh', 'PA', '2000-06-01 00:00:00', 100000, 'USD', 'seed'),\
    ('vivsimo', 'Viv’simo', 0, 'software', 'Pittsburgh', 'PA', '2001-01-29 00:00:00', 100000, 'USD', 'seed'),\
    ('showclix', 'ShowClix', 0, 'web', 'Oakmont', 'PA', '2007-10-01 00:00:00', 150000, 'USD', 'seed'),\
    ('showclix', 'ShowClix', 0, 'web', 'Oakmont', 'PA', '2008-06-01 00:00:00', 250000, 'USD', 'seed'),\
    ('biowizard', 'BioWizard', 8, 'web', 'Wayne', 'PA', '2008-01-03 00:00:00', 1000000, 'USD', 'a'),\
    ('ticketleap', 'TicketLeap', 4, 'web', 'Philadelphia', 'PA', '2008-07-22 00:00:00', 2000000, 'USD', 'a'),\
    ('tizra', 'Tizra', 0, 'web', 'Providence', 'RI', '2007-05-01 00:00:00', 500000, 'USD', 'seed'),\
    ('quickoffice', 'Quickoffice', 0, 'software', 'Plano', 'TN', '2007-02-05 00:00:00', 7000000, 'USD', 'b'),\
    ('quickoffice', 'Quickoffice', 0, 'software', 'Plano', 'TN', '2006-01-01 00:00:00', 11500000, 'USD', 'a'),\
    ('quickoffice', 'Quickoffice', 0, 'software', 'Plano', 'TN', '2008-05-23 00:00:00', 3000000, 'USD', 'c'),\
    ('thoof', 'Thoof', 0, 'web', 'Austin', 'TX', '2007-06-01 00:00:00', 1000000, 'USD', 'seed'),\
    ('bazaarvoice', 'Bazaarvoice', 0, 'web', 'Austin', 'TX', '2006-05-01 00:00:00', 4000000, 'USD', 'a'),\
    ('bazaarvoice', 'Bazaarvoice', 0, 'web', 'Austin', 'TX', '2007-09-01 00:00:00', 8800000, 'USD', 'b'),\
    ('bazaarvoice', 'Bazaarvoice', 0, 'web', 'Austin', 'TX', '2008-06-18 00:00:00', 7100000, 'USD', 'c'),\
    ('smallworldlabs', 'Small World Labs', 25, 'web', 'Austin', 'TX', '2008-03-19 00:00:00', 1000000, 'USD', 'a'),\
    ('pluck', 'Pluck', 0, 'web', 'Austin', 'TX', '2004-10-06 00:00:00', 8500000, 'USD', 'b'),\
    ('pluck', 'Pluck', 0, 'web', 'Austin', 'TX', '2006-11-14 00:00:00', 7000000, 'USD', 'c'),\
    ('spiceworks', 'Spiceworks', 0, 'web', 'Austin', 'TX', '2007-08-01 00:00:00', 8000000, 'USD', 'b'),\
    ('spendview', 'SpendView', 3, 'web', 'Houston', 'TX', '2008-01-01 00:00:00', 2000000, 'USD', 'a'),\
    ('shangby', 'Shangby', 0, 'web', 'Austin', 'TX', '2007-06-01 00:00:00', 1000000, 'USD', 'a'),\
    ('onnetworks', 'On Networks', 20, 'web', 'Austin', 'TX', '2006-11-01 00:00:00', 4000000, 'USD', 'a'),\
    ('onnetworks', 'On Networks', 20, 'web', 'Austin', 'TX', '2007-11-01 00:00:00', 12000000, 'USD', 'b'),\
    ('careflash', 'CareFlash', 0, 'web', 'Houston', 'TX', '2007-08-01 00:00:00', 600000, 'USD', 'seed'),\
    ('heliovolt', 'HelioVolt', 0, 'web', 'Austin', 'TX', '2007-08-01 00:00:00', 77000000, 'USD', 'b'),\
    ('challenge-games', 'Challenge Games', 0, 'web', 'Austin', 'TX', '2008-07-10 00:00:00', 4500000, 'USD', 'a'),\
    ('woot', 'Woot', 100, 'web', 'Carrollton', 'TX', '2008-01-01 00:00:00', 4000000, 'USD', 'a'),\
    ('mumboe', 'Mumboe', 17, 'software', 'Austin', 'TX', '2006-10-01 00:00:00', 4500000, 'USD', 'seed'),\
    ('breakingpoint-systems', 'BreakingPoint Systems', 54, 'hardware', 'Austin', 'TX', '2007-11-12 00:00:00', 15000000, 'USD', 'c'),\
    ('famecast', 'FameCast', 0, 'web', 'Austin', 'TX', '2007-06-01 00:00:00', 4500000, 'USD', 'a'),\
    ('click-forensics', 'Click Forensics', 0, 'web', 'Austin', 'TX', '2007-01-01 00:00:00', 5000000, 'USD', 'a'),\
    ('click-forensics', 'Click Forensics', 0, 'web', 'Austin', 'TX', '2008-03-01 00:00:00', 10000000, 'USD', 'b'),\
    ('godtube', 'GodTube', 0, 'web', 'Plano', 'TX', '2008-05-04 00:00:00', 30000000, 'USD', 'b'),\
    ('ibiz-software', 'iBiz Software', 50, 'software', 'Dallas', 'TX', '2008-05-20 00:00:00', 250000, 'USD', 'angel'),\
    ('cosential', 'Cosential', 10, 'web', 'Austin', 'TX', '2002-06-01 00:00:00', 250000, 'USD', 'b'),\
    ('jad-tech-consulting', 'JAD Tech Consulting', 6, 'consulting', 'Richardson', 'TX', '1993-09-23 00:00:00', 125000, 'USD', 'seed'),\
    ('mozy', 'Mozy', 26, 'web', 'American Fork', 'UT', '2005-05-01 00:00:00', 1900000, 'USD', 'a'),\
    ('mozy', 'Mozy', 26, 'web', 'American Fork', 'UT', '2005-05-01 00:00:00', 1900000, 'USD', 'a'),\
    ('ancestry-com', 'Ancestry.com', 0, 'web', 'Provo', 'UT', '1999-09-01 00:00:00', 33200000, 'USD', 'b'),\
    ('movenetworks', 'Move Networks', 0, 'web', 'American Fork', 'UT', '2006-12-01 00:00:00', 11300000, 'USD', 'a'),\
    ('movenetworks', 'Move Networks', 0, 'web', 'American Fork', 'UT', '2007-10-01 00:00:00', 34000000, 'USD', 'b'),\
    ('movenetworks', 'Move Networks', 0, 'web', 'American Fork', 'UT', '2008-04-14 00:00:00', 46000000, 'USD', 'c'),\
    ('worldvitalrecords', 'World Vital Records', 0, 'web', 'Provo', 'UT', '2007-09-01 00:00:00', 1200000, 'USD', 'a'),\
    ('bungee-labs', 'Bungee Labs', 38, 'web', 'Orem', 'UT', '2008-03-14 00:00:00', 8000000, 'USD', 'c'),\
    ('footnote', 'Footnote', 31, 'web', 'Lindon', 'UT', '2008-01-01 00:00:00', 8000000, 'USD', 'a'),\
    ('mediaforge', 'mediaFORGE', 0, 'web', 'Salt Lake City', 'UT', '2006-07-01 00:00:00', 1500000, 'USD', 'a'),\
    ('teamstreamz', 'TeamStreamz', 5, 'web', 'Lehi', 'UT', '2007-01-10 00:00:00', 80000, 'USD', 'seed'),\
    ('exinda', 'Exinda', 0, 'software', 'Sandy', 'UT', '2004-05-22 00:00:00', 6000000, 'USD', 'a'),\
    ('clearspring', 'Clearspring', 0, 'web', 'McLean', 'VA', '2006-05-01 00:00:00', 2500000, 'USD', 'a'),\
    ('clearspring', 'Clearspring', 0, 'web', 'McLean', 'VA', '2007-02-01 00:00:00', 5500000, 'USD', 'b'),\
    ('clearspring', 'Clearspring', 0, 'web', 'McLean', 'VA', '2008-05-20 00:00:00', 18000000, 'USD', 'c'),\
    ('clearspring', 'Clearspring', 0, 'web', 'McLean', 'VA', '2007-07-27 00:00:00', 10000000, 'USD', 'b'),\
    ('availmedia', 'Avail Media', 0, 'web', 'Reston', 'VA', '2007-05-01 00:00:00', 17000000, 'USD', 'b'),\
    ('availmedia', 'Avail Media', 0, 'web', 'Reston', 'VA', '2007-11-01 00:00:00', 25000000, 'USD', 'c'),\
    ('fortiusone', 'FortiusOne', 0, 'web', 'Arlington', 'VA', '2007-05-01 00:00:00', 5450000, 'USD', 'b'),\
    ('mixx', 'Mixx', 0, 'web', 'unknown', 'VA', '2007-09-01 00:00:00', 1500000, 'USD', 'a'),\
    ('mixx', 'Mixx', 0, 'web', 'unknown', 'VA', '2008-02-01 00:00:00', 2000000, 'USD', 'b'),\
    ('jobfox', 'Jobfox', 0, 'web', 'McLean', 'VA', '2008-01-01 00:00:00', 20000000, 'USD', 'c'),\
    ('jobfox', 'Jobfox', 0, 'web', 'McLean', 'VA', '2006-05-01 00:00:00', 13000000, 'USD', 'b'),\
    ('jobfox', 'Jobfox', 0, 'web', 'McLean', 'VA', '2005-04-01 00:00:00', 7000000, 'USD', 'a'),\
    ('healthcentral', 'HealthCentral', 0, 'web', 'Arlington', 'VA', '2008-01-01 00:00:00', 50000000, 'USD', 'a'),\
    ('comscore', 'comScore', 0, 'web', 'Reston', 'VA', '2002-06-06 00:00:00', 20000000, 'USD', 'd'),\
    ('comscore', 'comScore', 0, 'web', 'Reston', 'VA', '2001-08-13 00:00:00', 15000000, 'USD', 'c'),\
    ('visualcv', 'VisualCV', 20, 'web', 'Reston', 'VA', '2008-01-28 00:00:00', 2000000, 'USD', 'a'),\
    ('paxfire', 'Paxfire', 0, 'unknown', 'Sterling', 'VA', '2005-05-22 00:00:00', 2100000, 'USD', 'a'),\
    ('shoutwire', 'ShoutWire', 9, 'web', 'unknown', 'VA', '2007-01-01 00:00:00', 500000, 'USD', 'a'),\
    ('parature', 'Parature', 100, 'software', 'Vienna', 'VA', '2006-07-05 00:00:00', 13500000, 'USD', 'a'),\
    ('parature', 'Parature', 100, 'software', 'Vienna', 'VA', '2008-05-07 00:00:00', 16000000, 'USD', 'b'),\
    ('loladex', 'Loladex', 2, 'web', 'Leesburg', 'VA', '2007-11-01 00:00:00', 350000, 'USD', 'seed'),\
    ('appian', 'Appian', 120, 'software', 'Vienna', 'VA', '2008-07-21 00:00:00', 10000000, 'USD', 'a'),\
    ('fortisphere', 'Fortisphere', 50, 'software', 'Chantilly', 'VA', '2007-11-19 00:00:00', 10000000, 'USD', 'a'),\
    ('wetpaint', 'Wetpaint', 35, 'web', 'Seattle', 'WA', '2005-10-01 00:00:00', 5250000, 'USD', 'a'),\
    ('wetpaint', 'Wetpaint', 35, 'web', 'Seattle', 'WA', '2007-01-01 00:00:00', 9500000, 'USD', 'b'),\
    ('wetpaint', 'Wetpaint', 35, 'web', 'Seattle', 'WA', '2005-05-01 00:00:00', 25000000, 'USD', 'c'),\
    ('jobster', 'Jobster', 0, 'web', 'Seattle', 'WA', '2005-08-01 00:00:00', 19500000, 'USD', 'b'),\
    ('jobster', 'Jobster', 0, 'web', 'Seattle', 'WA', '2006-07-01 00:00:00', 18000000, 'USD', 'c'),\
    ('jobster', 'Jobster', 0, 'web', 'Seattle', 'WA', '2005-01-01 00:00:00', 8000000, 'USD', 'a'),\
    ('jobster', 'Jobster', 0, 'web', 'Seattle', 'WA', '2008-04-25 00:00:00', 7000000, 'USD', 'd'),\
    ('yapta', 'Yapta', 0, 'web', 'Seattle', 'WA', '2007-07-01 00:00:00', 2300000, 'USD', 'a'),\
    ('yapta', 'Yapta', 0, 'web', 'Seattle', 'WA', '2007-07-01 00:00:00', 700000, 'USD', 'seed'),\
    ('farecast', 'Farecast', 26, 'web', 'Seattle', 'WA', '2004-10-01 00:00:00', 1500000, 'USD', 'a'),\
    ('farecast', 'Farecast', 26, 'web', 'Seattle', 'WA', '2005-07-01 00:00:00', 7000000, 'USD', 'b'),\
    ('farecast', 'Farecast', 26, 'web', 'Seattle', 'WA', '2007-01-01 00:00:00', 12100000, 'USD', 'c'),\
    ('hautesecure', 'Haute Secure', 0, 'web', 'Seattle', 'WA', '2007-01-01 00:00:00', 500000, 'USD', 'a'),\
    ('newsvine', 'Newsvine', 6, 'web', 'Seattle', 'WA', '2005-07-01 00:00:00', 1250000, 'USD', 'a'),\
    ('ilike', 'iLike', 28, 'web', 'Seattle', 'WA', '2006-01-01 00:00:00', 2500000, 'USD', 'a'),\
    ('ilike', 'iLike', 28, 'web', 'Seattle', 'WA', '2006-01-01 00:00:00', 13300000, 'USD', 'b'),\
    ('redfin', 'Redfin', 100, 'web', 'Seattle', 'WA', '2007-07-01 00:00:00', 12000000, 'USD', 'c'),\
    ('redfin', 'Redfin', 100, 'web', 'Seattle', 'WA', '2006-05-01 00:00:00', 8000000, 'USD', 'b'),\
    ('redfin', 'Redfin', 100, 'web', 'Seattle', 'WA', '2005-09-01 00:00:00', 770000, 'USD', 'a'),\
    ('actionengine', 'Action Engine', 0, 'web', 'Bellevue', 'WA', '2000-05-01 00:00:00', 7700000, 'USD', 'a'),\
    ('actionengine', 'Action Engine', 0, 'web', 'Bellevue', 'WA', '2002-05-01 00:00:00', 5000000, 'USD', 'b'),\
    ('actionengine', 'Action Engine', 0, 'web', 'Bellevue', 'WA', '2005-01-01 00:00:00', 10000000, 'USD', 'd'),\
    ('actionengine', 'Action Engine', 0, 'web', 'Bellevue', 'WA', '2003-03-01 00:00:00', 15500000, 'USD', 'c'),\
    ('actionengine', 'Action Engine', 0, 'web', 'Bellevue', 'WA', '2007-07-01 00:00:00', 20000000, 'USD', 'e'),\
    ('wishpot', 'Wishpot', 0, 'web', 'Seattle', 'WA', '2008-04-29 00:00:00', 1000000, 'USD', 'a'),\
    ('payscale', 'PayScale', 0, 'web', 'Seattle', 'WA', '2004-10-01 00:00:00', 3200000, 'USD', 'a'),\
    ('payscale', 'PayScale', 0, 'web', 'Seattle', 'WA', '2005-10-01 00:00:00', 7000000, 'USD', 'b'),\
    ('payscale', 'PayScale', 0, 'web', 'Seattle', 'WA', '2007-07-01 00:00:00', 10300000, 'USD', 'c'),\
    ('buddytv', 'BuddyTV', 0, 'web', 'Seattle', 'WA', '2007-07-01 00:00:00', 2800000, 'USD', 'a'),\
    ('buddytv', 'BuddyTV', 0, 'web', 'Seattle', 'WA', '2007-05-01 00:00:00', 250000, 'USD', 'angel'),\
    ('buddytv', 'BuddyTV', 0, 'web', 'Seattle', 'WA', '2008-04-14 00:00:00', 6000000, 'USD', 'b'),\
    ('judysbook', 'Judys Book', 0, 'web', 'Seattle', 'WA', '2004-07-01 00:00:00', 2500000, 'USD', 'a'),\
    ('judysbook', 'Judys Book', 0, 'web', 'Seattle', 'WA', '2005-11-01 00:00:00', 8000000, 'USD', 'b'),\
    ('sampa', 'Sampa', 4, 'web', 'Redmond', 'WA', '2007-08-01 00:00:00', 310000, 'USD', 'angel'),\
    ('sampa', 'Sampa', 4, 'web', 'Redmond', 'WA', '2008-04-02 00:00:00', 1000000, 'USD', 'angel'),\
    ('zango', 'Zango', 225, 'web', 'Bellevue', 'WA', '2004-03-01 00:00:00', 40000000, 'USD', 'a'),\
    ('cdigix', 'Cdigix', 0, 'web', 'Seattle', 'WA', '2005-11-01 00:00:00', 4000000, 'USD', 'a'),\
    ('ripl', 'Ripl', 0, 'web', 'Seattle', 'WA', '2007-08-01 00:00:00', 4500000, 'USD', 'b'),\
    ('eyejot', 'EyeJot', 5, 'web', 'Seattle', 'WA', '2007-05-01 00:00:00', 750000, 'USD', 'seed'),\
    ('eyejot', 'EyeJot', 5, 'web', 'Seattle', 'WA', '2007-01-01 00:00:00', 400000, 'USD', 'angel'),\
    ('flowplay', 'FlowPlay', 6, 'web', 'Seattle', 'WA', '2007-05-01 00:00:00', 500000, 'USD', 'angel'),\
    ('flowplay', 'FlowPlay', 6, 'web', 'Seattle', 'WA', '2008-02-06 00:00:00', 3700000, 'USD', 'a'),\
    ('smartsheet', 'SmartSheet', 0, 'web', 'Bellevue', 'WA', '2007-06-01 00:00:00', 2690000, 'USD', 'a'),\
    ('visibletechnologies', 'Visible Technologies', 60, 'web', 'Seattle', 'WA', '2006-08-01 00:00:00', 3500000, 'USD', 'a'),\
    ('visibletechnologies', 'Visible Technologies', 60, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 12000000, 'USD', 'b'),\
    ('zillow', 'Zillow', 155, 'web', 'Seattle', 'WA', '2005-10-01 00:00:00', 32000000, 'USD', 'a'),\
    ('zillow', 'Zillow', 155, 'web', 'Seattle', 'WA', '2006-07-01 00:00:00', 25000000, 'USD', 'b'),\
    ('zillow', 'Zillow', 155, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 30000000, 'USD', 'c'),\
    ('seomoz', 'SEOMoz', 0, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 1250000, 'USD', 'a'),\
    ('docusign', 'DocuSign', 0, 'web', 'Seattle', 'WA', '2006-04-01 00:00:00', 10000000, 'USD', 'b'),\
    ('docusign', 'DocuSign', 0, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 12400000, 'USD', 'c'),\
    ('adready', 'AdReady', 33, 'web', 'Seattle', 'WA', '2007-12-01 00:00:00', 10000000, 'USD', 'b'),\
    ('treemo', 'Treemo', 0, 'web', 'Seattle', 'WA', '2007-10-01 00:00:00', 2550000, 'USD', 'a'),\
    ('gridnetworks', 'GridNetworks', 0, 'web', 'Seattle', 'WA', '2007-10-01 00:00:00', 9500000, 'USD', 'a'),\
    ('pelago', 'Pelago', 0, 'web', 'Seattle', 'WA', '2006-11-01 00:00:00', 7400000, 'USD', 'a'),\
    ('pelago', 'Pelago', 0, 'web', 'Seattle', 'WA', '2008-05-27 00:00:00', 15000000, 'USD', 'b'),\
    ('blist', 'Blist', 0, 'web', 'Seattle', 'WA', '2008-02-20 00:00:00', 6500000, 'USD', 'a'),\
    ('realself', 'RealSelf', 0, 'web', 'Seattle', 'WA', '2007-10-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('rescuetime', 'RescueTime', 0, 'web', 'Seattle', 'WA', '2007-10-14 00:00:00', 20000, 'USD', 'seed'),\
    ('zoji', 'Zoji', 0, 'web', 'Seattle', 'WA', '2007-10-01 00:00:00', 1500000, 'USD', 'angel'),\
    ('snapvine', 'Snapvine', 20, 'web', 'Seattle', 'WA', '2006-07-01 00:00:00', 2000000, 'USD', 'a'),\
    ('snapvine', 'Snapvine', 20, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 10000000, 'USD', 'b'),\
    ('jott', 'Jott', 0, 'web', 'Seattle', 'WA', '2007-05-01 00:00:00', 5400000, 'USD', 'b'),\
    ('earthclassmail', 'Earth Class Mail', 0, 'web', 'Seattle', 'WA', '2007-09-01 00:00:00', 7400000, 'USD', 'a'),\
    ('earthclassmail', 'Earth Class Mail', 0, 'web', 'Seattle', 'WA', '2008-01-01 00:00:00', 5900000, 'USD', 'a'),\
    ('smilebox', 'Smilebox', 0, 'web', 'Redmond', 'WA', '2006-02-01 00:00:00', 5000000, 'USD', 'a'),\
    ('smilebox', 'Smilebox', 0, 'web', 'Redmond', 'WA', '2007-12-01 00:00:00', 7000000, 'USD', 'b'),\
    ('fyreball', 'Fyreball', 0, 'web', 'Bellevue', 'WA', '2007-12-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('delve-networks', 'Delve Networks', 0, 'web', 'Seattle', 'WA', '2006-12-01 00:00:00', 1650000, 'USD', 'seed'),\
    ('delve-networks', 'Delve Networks', 0, 'web', 'Seattle', 'WA', '2007-08-01 00:00:00', 8000000, 'USD', 'a'),\
    ('livemocha', 'LiveMocha', 0, 'web', 'Bellevue', 'WA', '2008-01-01 00:00:00', 6000000, 'USD', 'a'),\
    ('mercentcorporation', 'Mercent Corporation', 50, 'web', 'Seattle', 'WA', '2008-01-01 00:00:00', 6500000, 'USD', 'b'),\
    ('cleverset', 'CleverSet', 0, 'web', 'Seattle', 'WA', '2006-08-01 00:00:00', 1400000, 'USD', 'angel'),\
    ('cleverset', 'CleverSet', 0, 'web', 'Seattle', 'WA', '2005-09-01 00:00:00', 1200000, 'USD', 'a'),\
    ('cleverset', 'CleverSet', 0, 'web', 'Seattle', 'WA', '2007-12-01 00:00:00', 500000, 'USD', 'angel'),\
    ('liquidplanner', 'LiquidPlanner', 11, 'web', 'Bellevue', 'WA', '2008-01-01 00:00:00', 1200000, 'USD', 'angel'),\
    ('limeade', 'Limeade', 10, 'web', 'Bellevue', 'WA', '2007-01-01 00:00:00', 1000000, 'USD', 'angel'),\
    ('yodio', 'Yodio', 4, 'web', 'Bellevue', 'WA', '2008-02-01 00:00:00', 850000, 'USD', 'a'),\
    ('tastemakers', 'Tastemakers', 11, 'web', 'Seattle', 'WA', '2007-02-01 00:00:00', 1000000, 'USD', 'a'),\
    ('whitepages-com', 'WhitePages.com', 125, 'web', 'Seattle', 'WA', '2005-08-01 00:00:00', 45000000, 'USD', 'a'),\
    ('revenuescience', 'RevenueScience', 0, 'web', 'Seattle', 'WA', '2005-12-19 00:00:00', 24000000, 'USD', 'e'),\
    ('gotvoice', 'GotVoice', 15, 'mobile', 'Kirkland', 'WA', '2006-10-25 00:00:00', 3000000, 'USD', 'a'),\
    ('cardomain-network', 'CarDomain Network', 50, 'web', 'Seattle', 'WA', '2007-06-29 00:00:00', 3000000, 'USD', 'a'),\
    ('mpire', 'mpire', 0, 'web', 'Seattle', 'WA', '2007-02-01 00:00:00', 9800000, 'USD', 'a'),\
    ('mpire', 'mpire', 0, 'web', 'Seattle', 'WA', '2008-06-11 00:00:00', 10000000, 'USD', 'b'),\
    ('teachstreet', 'TeachStreet', 8, 'web', 'Seattle', 'WA', '2008-03-01 00:00:00', 2250000, 'USD', 'a'),\
    ('estately', 'Estately', 0, 'web', 'unknown', 'WA', '2008-04-24 00:00:00', 450000, 'USD', 'angel'),\
    ('infinia', 'Infinia', 0, 'cleantech', 'Kennewick', 'WA', '2008-04-22 00:00:00', 57000000, 'USD', 'b'),\
    ('infinia', 'Infinia', 0, 'cleantech', 'Kennewick', 'WA', '2007-06-01 00:00:00', 9500000, 'USD', 'a'),\
    ('m-metrics', 'M:Metrics', 0, 'web', 'Seattle', 'WA', '2005-10-16 00:00:00', 7000000, 'USD', 'b'),\
    ('cozi', 'Cozi', 26, 'software', 'Seattle', 'WA', '2007-07-11 00:00:00', 3000000, 'USD', 'a'),\
    ('cozi', 'Cozi', 26, 'software', 'Seattle', 'WA', '2008-06-01 00:00:00', 8000000, 'USD', 'c'),\
    ('trusera', 'Trusera', 15, 'web', 'Seattle', 'WA', '2007-06-01 00:00:00', 2000000, 'USD', 'angel'),\
    ('alerts-com', 'Alerts.com', 0, 'web', 'Bellevue', 'WA', '2008-07-08 00:00:00', 1200000, 'USD', 'a'),\
    ('myrio', 'Myrio', 75, 'software', 'Bothell', 'WA', '2001-01-01 00:00:00', 20500000, 'USD', 'unattributed'),\
    ('grid-networks', 'Grid Networks', 0, 'web', 'Seattle', 'WA', '2007-10-30 00:00:00', 9500000, 'USD', 'a'),\
    ('grid-networks', 'Grid Networks', 0, 'web', 'Seattle', 'WA', '2008-05-20 00:00:00', 10500000, 'USD', 'b')\
;")


# Insert new employee
cursor.execute(cf_tbl)
#emp_no = cursor.lastrowid

# Insert salary information
#data_salary = {
#  'emp_no': emp_no,
#  'salary': 50000,
#  'from_date': tomorrow,
#  'to_date': date(9999, 1, 1),\
#}
#cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()

## 1
- Show a pie chart on all the data ingested which depicts the total funding raised category wise.

```
SELECT   category, SUM(raisedAmt) AS `Total raisedAmt` FROM cf_tbl GROUP BY category;
```

## 2
- Show a pie chart on all the data ingested which depicts the total funding raised state wise.

```
SELECT state, SUM(raisedAmt) AS `Total raisedAmt` FROM cf_tbl GROUP BY state;
```

## 3
- Bar chart depicting total funding raised date wise.

```
SELECT fundedDate, SUM(raisedAmt) AS `Total raisedAmt` FROM cf_tbl GROUP BY fundedDate;
```

## 4

- Show and downloadable data in csv format contains all data (columns), round wise (a/b/c/seed etc) separately. i.e return all the rows which has gone under &quot;seed&quot; funding.

```
SELECT * FROM cf_tbl WHERE roundx='seed';
```

## 5
- Return the name of the company(s) which has made their way till &quot;round e&quot; funding. Similarly for other rounds as well.
- For e.g. For your reference sequence is: seed, angel, a, b, c, d, e

```
SELECT COUNT(roundx) AS Total, company 
  FROM (SELECT company, roundx 
  FROM (SELECT company, roundx FROM cf_tbl GROUP BY roundx, company) as comp 
 WHERE roundx IN ('a','b','c','d')) AS compcomp GROUP BY company 
HAVING Total=4;


SELECT company
     GROUP_CONCAT(DISTINCT roundx ORDER BY roundx) AS rounds
     COUNT(DISTINCT roundx) AS n
  FROM cf_tbl
 WHERE roundx IN ('a', 'b', 'c', 'd')
 GROUP BY company
HAVING n = 4

mysql> SELECT COUNT(roundx) AS Total, company 
    -> FROM (SELECT company, roundx 
    -> FROM (SELECT company, roundx FROM cf_tbl GROUP BY roundx, company) as comp 
    -> WHERE roundx IN ('a','b','c','d')) AS compcomp GROUP BY company HAVING Total=4;
+-------+---------------------+
| Total | company             |
+-------+---------------------+
|     4 | Plaxo               |
|     4 | Veoh                |
|     4 | MeeVee              |
|     4 | Yelp                |
|     4 | Glam Media          |
|     4 | LinkedIn            |
|     4 | obopay              |
|     4 | Friendster          |
|     4 | Trulia              |
|     4 | Demand Media        |
|     4 | Tesla Motors        |
|     4 | SugarCRM            |
|     4 | fabrik              |
|     4 | Teneros             |
|     4 | SearchMe            |
|     4 | Plastic Logic       |
|     4 | Brightcove          |
|     4 | Endeca              |
|     4 | Waterfront Media    |
|     4 | Answers Corporation |
|     4 | Jobster             |
|     4 | Action Engine       |
+-------+---------------------+
22 rows in set (0.00 sec)

```

```
mysql> SELECT company
    ->      , GROUP_CONCAT(DISTINCT roundx ORDER BY roundx) AS rounds
    ->      , COUNT(DISTINCT roundx) AS n
    ->   FROM cf_tbl
    ->  WHERE roundx IN ('a', 'b', 'c', 'd')
    ->  GROUP BY company
    -> HAVING n = 4;
+---------------------+---------+---+
| company             | rounds  | n |
+---------------------+---------+---+
| Action Engine       | a,b,c,d | 4 |
| Answers Corporation | a,b,c,d | 4 |
| Brightcove          | a,b,c,d | 4 |
| Demand Media        | a,b,c,d | 4 |
| Endeca              | a,b,c,d | 4 |
| fabrik              | a,b,c,d | 4 |
| Friendster          | a,b,c,d | 4 |
| Glam Media          | a,b,c,d | 4 |
| Jobster             | a,b,c,d | 4 |
| LinkedIn            | a,b,c,d | 4 |
| MeeVee              | a,b,c,d | 4 |
| obopay              | a,b,c,d | 4 |
| Plastic Logic       | a,b,c,d | 4 |
| Plaxo               | a,b,c,d | 4 |
| SearchMe            | a,b,c,d | 4 |
| SugarCRM            | a,b,c,d | 4 |
| Teneros             | a,b,c,d | 4 |
| Tesla Motors        | a,b,c,d | 4 |
| Trulia              | a,b,c,d | 4 |
| Veoh                | a,b,c,d | 4 |
| Waterfront Media    | a,b,c,d | 4 |
| Yelp                | a,b,c,d | 4 |
+---------------------+---------+---+
22 rows in set (0.07 sec)
```